In [447]:
from docplex.mp.model import Model
import cplex
from time import time

In [501]:
import tsplib95
import networkx as nx
import numpy as np
# load the tsplib problem

problem = tsplib95.load('Tnm118.tsp')

# convert into a networkx.Graph
graph = problem.get_graph()

# convert into a numpy distance matrix
distance_matrix = nx.to_numpy_array(graph)


c = distance_matrix
n = len(c)
B=nx.Graph()
for i in range(len(c)):
  for j in range(len(c)):
    if i<j:
      B.add_edge(i,j,weight=c[i][j])


def nodes2edge(u, v):
    return (min(u, v), max(u, v))

#import networkx as nx

# Define the file path
file_path = 'Tnm118_1.txt'  # Replace with your file's actual path if needed

# Create an empty graph
graph_new = nx.Graph()
graph_new.add_nodes_from(B.nodes())
# Read the file and add edges to the graph
with open(file_path, 'r') as file:
    for line in file:
        source, target, attributes = line.strip().split(maxsplit=2)
        source, target = int(source), int(target)
        attributes = eval(attributes)  # Convert string representation of dictionary to a Python dictionary
        graph_new.add_edge(source, target, **attributes)
n=len(graph_new.nodes)

mat =[[0 for i in range(n)] for j in range(n)]
def two_opt(tour,dist):
      """Improve the TSP tour using 2-opt heuristic."""
      best = tour
      improved = True
      best_distance = total_distance(tour, dist)

      while improved:
          improved = False
          for i in range(1, len(tour) - 2):
              for j in range(i + 1, len(tour)):
                  if j - i == 1:
                      continue  # Skip adjacent edges
                  new_tour = best[:i] + best[i:j][::-1] + best[j:]
                  new_distance = total_distance(new_tour, dist)
                  if new_distance < best_distance:
                      best = new_tour
                      best_distance = new_distance
                      improved = True
                      break  # Improvement found, restart outer loop
              if improved:
                  break
      return best
for i in range(n):
      for j in range(n):
          if i!=j and ((min(i,j),max(i,j)) in graph_new.edges):
              x = c[i][j]
              mat[i][j] = x
          if i!=j and ((min(i,j),max(i,j)) not in graph_new.edges):
              mat[i][j] = 10000000000
def total_distance(tour, dist):
      """Calculate total distance of the given tour using distance matrix."""
      return sum(dist[tour[i]][tour[(i + 1) % len(tour)]] for i in range(len(tour)))
  #print('Distances Matrix is :')
  #print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in mat]))
def nearest_neighbor(start,mat):
      tour=[start]
      n=len(mat)
      for i in range(n-1):

          m=next(x for x in list(range(0,n)) if x not in tour)

          for j in range(n):
              if j not in tour:
                  if mat[start][j] < mat[start][m]:
                      m=j


          tour.append(m)
          start = m


      tour.append(tour[0])
      return tour

In [503]:
from networkx.algorithms import bipartite
def edge_from_tour(tour):
    edge_set=[]
    for i in range(len(tour)):
        if i>=1:
            edge_set.append((min(tour[i-1],tour[i]),max(tour[i-1],tour[i])))
    return edge_set 


#print(max_b_match(solution))
def edges_to_vertices(edges):
    # Create adjacency list
    adj = {}
    for u, v in edges:
        adj.setdefault(u, []).append(v)
        adj.setdefault(v, []).append(u)

    # Find the start node (degree 1 node in the path)
    start = None
    for node in adj:
        if len(adj[node]) == 1:  # Degree 1 implies a start or end node
            start = node
            break

    # Traverse the path to reconstruct the vertex order
    path = [start]
    current = start
    prev = None
    while len(path) < len(edges) + 1:
        for neighbor in adj[current]:
            if neighbor != prev:  # Avoid going back to the previous node
                path.append(neighbor)
                prev, current = current, neighbor
                break

    return path

m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
ham_path=[]
edges_cycle_cover=[]
path_edges=[]
for e in graph_new.edges:
    if solution.get_value(x[e])>1e-7:
        edges_cycle_cover.append(e)
sum_path=0 
for component in nx.connected_components(graph_new.edge_subgraph(edges_cycle_cover)):
    G_comp=nx.Graph()
    G_tree=nx.Graph()
    for e in edges_cycle_cover:
        if e[0] in component and e[1] in component:
           
           G_comp.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
    T=nx.minimum_spanning_tree(G_comp, weight='weight')
    for e in T.edges:
        path_edges.append(e)
    ham_path.append(edges_to_vertices(T.edges()))
    print(edges_to_vertices(T.edges()))
    for e in T.edges():
            G_tree.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
            sum_path+=c[e[0]][e[1]]
def bipartite_edge_distance(G, u, v, U, V):
    if u not in G or v not in G:
        raise ValueError("Node not in graph.")
    
    # Check if u and v are on the same side
    if (u in U and v in U) or (u in V and v in V):
        return float('inf')

    # Check for direct connection (undirected graph)
    if G.has_edge(u, v):
        return c[u][v]  # default weight = 1
    else:
        return float('inf')
def mat_ham(G,paths,U,V):    
    ham_path_mat=np.zeros((len(paths),len(paths)))
    for i in range(len(paths)):
        for j in range(len(paths)):
            if i!=j:
                ham_path_mat[i][j]=min([bipartite_edge_distance(G, paths[i][0], paths[j][0], U, V),bipartite_edge_distance(G, paths[i][0], paths[j][-1], U, V),bipartite_edge_distance(G, paths[i][-1], paths[j][0], U, V),bipartite_edge_distance(G, paths[i][-1], paths[j][-1], U, V)])
            
    return ham_path_mat
def merge_hamiltonian_paths_fix_largest(paths, U, V): 
    # Sort paths by length and fix the largest path
    paths.sort(key=len, reverse=True)
    largest_path = paths[0]
    other_paths = paths[1:]

    # Initialize the cycle with the largest path
    cycle = largest_path[:]
    current_end = largest_path[-1]

    # Connect other paths to the cycle
    for path in other_paths:
        start, end = path[0], path[-1]

        # Determine if reversal is needed
        if current_end in U and start in U or current_end in V and start in V:
            path = path[::-1]  # Reverse the path
            start, end = end, start

        # Add the reversed or original path to the cycle
        cycle.extend(path)
        current_end = end  # Update the current endpoint
    
    # Close the cycle
    first_start = largest_path[0]
    if current_end in U and first_start in V or current_end in V and first_start in U:
        cycle.append(first_start)

    return cycle
bottom_nodes, top_nodes = bipartite.sets(graph_new)
print(mat_ham(graph_new,ham_path, bottom_nodes, top_nodes))
merge_path=merge_hamiltonian_paths_fix_largest(ham_path, bottom_nodes, top_nodes)

init_tour= two_opt(merge_path,mat)
print(init_tour)
print(total_distance(merge_path,mat))
print(total_distance(init_tour,mat))

[112, 115, 114, 111, 108, 105, 102, 99, 96, 93, 85, 88, 91, 2, 5, 8, 11, 14, 17, 23, 20, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 0, 3, 94, 97, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 95, 90, 87, 1, 4, 7, 10, 98, 101, 104, 107, 110, 113, 116, 117]
[16, 13, 19, 22]
[25, 28, 31, 34]
[37, 40, 43, 46]
[49, 52, 55, 58]
[64, 61, 67, 70]
[73, 76, 79, 82]
[100, 103, 106, 109]
[[     0. 130512. 110605.  92916.  90738. 123423. 130512.  19887.]
 [130512.      0.  10000.  50000.  90000. 160000. 170000. 176834.]
 [110605.  10000.      0.  10000.  50000. 120000. 130000. 162697.]
 [ 92916.  50000.  10000.      0.  10000.  80000.  90000. 151230.]
 [ 90738.  90000.  50000.  10000.      0.  40000.  50000. 149901.]
 [123423. 160000. 120000.  80000.  40000.      0.  29999. 171670.]
 [130512. 170000. 130000.  90000.  50000.  29999.      0. 176835.]
 [ 19887. 176834. 162697. 151230. 149901. 1

In [505]:
from cplex.callbacks import UserCutCallback, LazyConstraintCallback, BranchCallback, NodeCallback, Context
from collections import defaultdict
from time import time
from typing import *

import numpy as np

from docplex.mp.callbacks.cb_mixin import *
from collections import defaultdict, namedtuple
import cplex.callbacks as cpx_cb
import cplex._internal._constants as cpxcst
import math
import time
import random
import itertools
def similar_edges(edges):
  no_shared_vertices = True

# Iterate through all pairs of edges
  for edge1, edge2 in itertools.combinations(edges, 2):
      if set(edge1) & set(edge2):  # Check for intersection of vertices
          #print(f"Edges {edge1} and {edge2} share vertices.")
          #no_shared_vertices = False
          return set(edge1) & set(edge2)

  if no_shared_vertices:
      return True
def vertex_list(edges):
    vertices = set()
    for u, v in edges:
        vertices.add(u)
        vertices.add(v)
    
    # Optional: convert to sorted list
    vertex_list = sorted(vertices)
    return vertex_list

def has_edge(i,e,graph_new,solution):
    if graph_new.has_edge(i, e[0]) and solution.get_value(x[(min(i,e[0]),max(i,e[0]))])>1e-7:
       return (i, e[0]),solution.get_value(x[(min(i,e[0]),max(i,e[0]))])
        
    elif graph_new.has_edge(i, e[1]) and solution.get_value(x[(min(i,e[1]),max(i,e[1]))])>1e-7:
       return (i, e[1]), solution.get_value(x[(min(i,e[1]),max(i,e[1]))])
    else:
       return None 
        
def nodes2edge(u, v):
    return (min(u, v), max(u, v))
def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons
def k_factors(solution):
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    #k_factor(sep_graph, 2, matching_weight='weight')
    G=nx.k_factor(sep_graph, 2,matching_weight='weight')
    subtour=[]
    
    for component in nx.connected_components( sep_graph.edge_subgraph([e for e in G.edges() if G[e[0]][e[1]]['weight']>1/2+1e-7]) ):
        subtour.append(component)
    return subtour
from docplex.mp.callbacks.cb_mixin import *
import numpy as np
def add_violated_cuts(callback, solution):
    fracSubtours, distances = getFracSubtourCons(solution, callback.x, callback.nodes)
    for subtour in fracSubtours:
        expr = []
        for edge in subtour:
            expr.append("x.{}.{}".format(*edge))
        thecoefs = [1] * len(subtour) 
        callback.add(cplex.SparsePair(expr, thecoefs), sense="G", rhs=2)
    return fracSubtours, len((fracSubtours)), max(distances) if len(distances) > 0 else 0
class SubtourUserCallback(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           # var_pos_name=[]
           # edge_pos=[]
           #if self.get_num_nodes()%20==0:
               sol = self.make_solution_from_vars(x.values())
               subtourCons= getFracSubtourCons(sol)
               for subtour in subtourCons:
                       tour=0
                       for e in subtour:
                           tour+=x[e]
                       ct=tour>=2
                       unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                       if unsats:
                           for ct, cpx_lhs, sense, cpx_rhs in unsats:
                                self.add(cpx_lhs, sense, cpx_rhs)
               #             print('Skip cut',self.get_num_nodes())
               # comb_graph = nx.Graph()
               # comb_graph.add_nodes_from(graph_new.nodes)
               # solution=sol
               # for e in graph_new.edges:
               #     val = solution.get_value(x[e])
               #     if val > 1e-7:
               #              comb_graph.add_edge(e[0], e[1], weight=val)
                    
               #      # Step 2: Get max weight matching
               # raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
                    
               #      # Normalize edge representation
               # matching = []
               # for e in raw_matching:
               #      edge = (min(e), max(e))
               #      if edge in graph_new.edges and solution.get_value(x[edge])>0.5+1e-7:
               #              matching.append(edge)
                    
               #      # Step 3: If perfect matching, print
               #      #if nx.is_perfect_matching(comb_graph, matching):
                    
                    
               #          # Step 4: Build meta-graph where each node is a matching edge
               # meta_graph = nx.Graph()
               # meta_graph.add_nodes_from(matching)
                    
               #          # Step 5: Connect matching edges that share neighbors in original graph
               # for i in range(len(matching)):
               #      for j in range(i + 1, len(matching)):
               #          e1 = matching[i]
               #          e2 = matching[j]
                    
               #                  # Get all 4 nodes
               #          nodes_e1 = set(e1)
               #          nodes_e2 = set(e2)
                    
               #                  # Check all connections between nodes of e1 and e2
               #          connecting_weight = 0
               #          for u in nodes_e1:
               #              for v in nodes_e2:
               #                  if graph_new.has_edge(u, v):
               #                     connecting_weight += solution.get_value(x[(u, v)]) if (u, v) in x else solution.get_value(x[(v, u)])
                    
               #          if connecting_weight > 0:
               #              meta_graph.add_edge(e1, e2, weight=min(connecting_weight,1))
                    
               #          # Step 6: Print meta-graph edges
               # chosen_meta=[(u,v) for u, v, data in meta_graph.edges(data=True) if data['weight']<1-1e-7]
               # for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta) ):
               #                  handle_vertices=[]
               #                  handle_edges = []
               #                  teeth_edges =[]
               #                  teeth_vertex=[]
               #                  internal_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v in component)]
               #                  cut_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v not in component) or (u not in component and v in component)]
               #                  for (e1,e2) in internal_edge:
               #                      for u in e1:
               #                          for v in e2:
               #                              if graph_new.has_edge(u, v):
               #                                  handle_edges.append((min(u, v),max(u,v)))
               #                      handle_edges.append(e1)
               #                      handle_edges.append(e2)
               #                  for (e1,e2) in cut_edge:
               #                      teeth=[]
               #                      for u in e1:
               #                          for v in e2:
               #                              if graph_new.has_edge(u, v):
               #                                  teeth.append((min(u, v),max(u,v)))
               #                      teeth.append(e1)
               #                      teeth.append(e2)
               #                      teeth_edges.append(teeth)
               #                      teeth_vertex.append(vertex_list(teeth))
                                    
               #                  if len(cut_edge)%2==1 and len(cut_edge)>=3 and similar_edges(cut_edge)==True:
               #                      tour=0
               #                      teeth_sum=0
               #                      for teeth in teeth_vertex:
               #                          teeth_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in teeth and v in teeth]
               #                          for e in teeth_edges_in_graph:
               #                              teeth_sum+=x[e]
               #                      handle_vertices_set = vertex_list(handle_edges)
               #                      handle_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in handle_vertices_set and v in handle_vertices_set]
               #                      for e in handle_edges_in_graph :
               #                          #if e[0] in vertex_list(handle_edges) and e[1] in vertex_list(handle_edges):
               #                              tour+=x[e]
    
               #                      ct=tour+teeth_sum<= len(handle_vertices_set)+sum(len(teeth_vertices) for teeth_vertices in teeth_vertex )-0.5*(3*len(teeth_vertex)+1) 
               #                      unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               #                      for ct, cpx_lhs, sense, cpx_rhs in unsats:
               #                              self.add(cpx_lhs, sense, cpx_rhs)
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v>1e-7): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 if unsats:
                     for ct, cpx_lhs, sense, cpx_rhs in unsats:
        
                         self.add(cpx_lhs, sense, cpx_rhs)
                     #print('Lazy SEC')





def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph

def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          numb_U=0
          numb_V=0
          for i in component:
              if i in bottom_nodes:
                  numb_U+=1
              if i in top_nodes:
                  numb_V+=1
          if len(component) < n and len(component)%2==0 and numb_U==numb_V:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub




class SubtourUserCallback_heu(ConstraintCallbackMixin, UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)
        self.subtour_cut_count = 0
        
    def __call__(self):
           # var_pos_name=[]
           # edge_pos=[]
           #if self.get_num_nodes()%10==0:
              sol = self.make_solution_from_vars(x.values())
              subtourCons=  heuristic_SEC_GH(sol)
              for subtour in subtourCons:
                       tour=0
                       for e in subtour:
                           tour+=x[e]
                       ct=tour>=2
                       unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                       if unsats:
                           for ct, cpx_lhs, sense, cpx_rhs in unsats:
                                self.add(cpx_lhs, sense, cpx_rhs)
              # for i in range(len(component_sub)):
              #     tour=0
              #     for e in edges_sub[i]:
              #              tour+=x[e]
              #     ct=tour<=len(component_sub[i])-1
              #     unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
              #     for ct, cpx_lhs, sense, cpx_rhs in unsats:
              #               self.add(cpx_lhs, sense, cpx_rhs)
                       #print('Skip cut',self.get_num_nodes())
                                
           
            
                        # Get all 4 nodes
              # comb_graph = nx.Graph()
              # comb_graph.add_nodes_from(graph_new.nodes)
              # solution=sol
              # for e in graph_new.edges:
              #     val = solution.get_value(x[e])
              #     if val > 1e-7:
              #                comb_graph.add_edge(e[0], e[1], weight=val)
                    
              #  #      # Step 2: Get max weight matching
              # raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
                    
              #  #      # Normalize edge representation
              # matching = []
              # for e in raw_matching:
              #        edge = (min(e), max(e))
              #        if edge in graph_new.edges and solution.get_value(x[edge])>1-1e-7:
              #                matching.append(edge)
                    
              #  #      # Step 3: If perfect matching, print
              #        #if nx.is_perfect_matching(comb_graph, matching):
                    
                    
              #  #          # Step 4: Build meta-graph where each node is a matching edge
              # meta_graph = nx.Graph()
              # meta_graph.add_nodes_from(matching)
                    
              #  #          # Step 5: Connect matching edges that share neighbors in original graph
              # for i in range(len(matching)):
              #     for j in range(i + 1, len(matching)):
              #            e1 = matching[i]
              #            e2 = matching[j]
                    
              #  #                  # Get all 4 nodes
              #            nodes_e1 = set(e1)
              #            nodes_e2 = set(e2)
                    
              #  #                  # Check all connections between nodes of e1 and e2
              #            connecting_weight = 0
              #            for u in nodes_e1:
              #                for v in nodes_e2:
              #                    if graph_new.has_edge(u, v):
              #                       connecting_weight += solution.get_value(x[(u, v)]) if (u, v) in x else solution.get_value(x[(v, u)])
                   
              #            if connecting_weight > 0:
              #                meta_graph.add_edge(e1, e2, weight=connecting_weight)
                    
              #  #          # Step 6: Print meta-graph edges
              # chosen_meta=[(u,v) for u, v, data in meta_graph.edges(data=True) if data['weight']<1-1e-7]
              # for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta) ):
              #                    handle_vertices=[]
              #                    handle_edges = []
              #                    teeth_edges =[]
              #                    teeth_vertex=[]
              #                    internal_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v in component)]
              #                    cut_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v not in component) or (u not in component and v in component)]
              #                    for (e1,e2) in internal_edge:
              #                        for u in e1:
              #                            for v in e2:
              #                                if graph_new.has_edge(u, v):
              #                                    handle_edges.append((min(u, v),max(u,v)))
              #                        handle_edges.append(e1)
              #                        handle_edges.append(e2)
              #                    for (e1,e2) in cut_edge:
              #                        teeth=[]
              #                        for u in e1:
              #                            for v in e2:
              #                                if graph_new.has_edge(u, v):
              #                                    teeth.append((min(u, v),max(u,v)))
              #                        teeth.append(e1)
              #                        teeth.append(e2)
              #                        teeth_edges.append(teeth)
              #                        teeth_vertex.append(vertex_list(teeth))
                                    
              #                    if len(cut_edge)%2==1 and len(cut_edge)>=3 and similar_edges(cut_edge)==True:
              #                        tour=0
              #                        teeth_sum=0
              #                        for teeth in teeth_vertex:
              #                            teeth_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in teeth and v in teeth]
              #                            for e in teeth_edges_in_graph:
              #                                teeth_sum+=x[e]
              #                        handle_vertices_set = vertex_list(handle_edges)
              #                        handle_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in handle_vertices_set and v in handle_vertices_set]
              #                        for e in handle_edges_in_graph :
              #                            if e[0] in vertex_list(handle_edges) and e[1] in vertex_list(handle_edges):
              #                                tour+=x[e]
    
              #                        ct=tour+teeth_sum<= len(handle_vertices_set)+sum(len(teeth_vertices) for teeth_vertices in teeth_vertex )-0.5*(3*len(teeth_vertex)+1) 
              #                        unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
              #                        for ct, cpx_lhs, sense, cpx_rhs in unsats:
              #                                self.add(cpx_lhs, sense, cpx_rhs)
                                        
              comb_graph = nx.Graph()
              comb_graph.add_nodes_from(graph_new.nodes)
                    
              for e in graph_new.edges:
                        val = sol.get_value(x[e])
                        if val > 1e-7:
                            comb_graph.add_edge(e[0], e[1], weight=val)
                        
                        # Step 2: Get max weight matching
              raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
                        
                        # Step 3: Normalize and filter matching edges
              matching = []
              for e in raw_matching:
                        edge = (min(e), max(e))
                        if edge in graph_new.edges and sol.get_value(x[edge])>1-1e-7:
                                matching.append(edge)
                        
                        # Step 4: Print matching
                      
                        
                        # Step 5: Build meta-graph
              meta_graph = nx.Graph()
              normal_vertex=[]
              meta_graph.add_nodes_from(matching)
              for v in comb_graph.nodes():
                        if v not in vertex_list(matching):
                           meta_graph.add_node((v,v))
                           normal_vertex.append(v)
              for i in range(len(matching)):
                    for j in range(i + 1, len(matching)):
                            e1 = matching[i]
                            e2 = matching[j]
                            nodes_e1 = set(e1)
                            nodes_e2 = set(e2)
                            connecting_weight = 0
                            for u in nodes_e1:
                                for v in nodes_e2:
                                    if graph_new.has_edge(u, v):
                                            key = (u, v) if (u, v) in x else (v, u)
                                            connecting_weight += sol.get_value(x[key])
                            if connecting_weight > 0:
                                    meta_graph.add_edge(e1, e2, weight=connecting_weight)
              for e in comb_graph.edges:
                        if e[0] in normal_vertex and e[1] in normal_vertex:
                            meta_graph.add_edge((e[0],e[0]), (e[1],e[1]), weight=sol.get_value(x[e]))
              for i in normal_vertex:
                        for e in matching:
                            if has_edge(i,e,graph_new,sol)!=None:
                                edge,weight_sol=has_edge(i,e,graph_new,sol)
                                meta_graph.add_edge((i,i),e,weight=weight_sol )
                    #print(meta_graph.edges(data=True))
              chosen_meta = [(u, v) for u, v, data in meta_graph.edges(data=True) if data['weight'] < 1 - 1e-7]
              for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta) ):
                                handle_vertices=[]
                                handle_edges = []
                                teeth_edges =[]
                                teeth_vertex=[]
                                internal_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v in component)]
                                cut_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v not in component) or (u not in component and v in component)]
                                for (e1,e2) in internal_edge:
                                    for u in e1:
                                        for v in e2:
                                            if graph_new.has_edge(u, v):
                                                handle_edges.append((min(u, v),max(u,v)))
                                    handle_edges.append(e1)
                                    handle_edges.append(e2)
                                for (e1,e2) in cut_edge:
                                    teeth=[]
                                    for u in e1:
                                        for v in e2:
                                            if graph_new.has_edge(u, v):
                                                teeth.append((min(u, v),max(u,v)))
                                    teeth.append(e1)
                                    teeth.append(e2)
                                    teeth_edges.append(teeth)
                                    teeth_vertex.append(vertex_list(teeth))
                                    
                                if len(cut_edge)%2==1 and len(cut_edge)>=3 and similar_edges(cut_edge)==True:
                                    tour=0
                                    teeth_sum=0
                                    for teeth in teeth_vertex:
                                        teeth_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in teeth and v in teeth]
                                        for e in teeth_edges_in_graph:
                                            teeth_sum+=x[e]
                                    handle_vertices_set = vertex_list(handle_edges)
                                    handle_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in handle_vertices_set and v in handle_vertices_set]
                                    for e in handle_edges_in_graph :
                                        #if e[0] in vertex_list(handle_edges) and e[1] in vertex_list(handle_edges):
                                            tour+=x[e]
    
                                    ct=tour+teeth_sum<= len(handle_vertices_set)+sum(len(teeth_vertices) for teeth_vertices in teeth_vertex )-0.5*(3*len(teeth_vertex)+1) 
                                    unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                                    for ct, cpx_lhs, sense, cpx_rhs in unsats:
                                             self.add(cpx_lhs, sense, cpx_rhs)
class SubtourUserCallback_heu_fixed(ConstraintCallbackMixin, UserCutCallback):
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)
          # Dictionary: edge -> variable
        self.graph_new = graph_new
        self.bottom_nodes = bottom_nodes
        self.top_nodes = top_nodes

    def __call__(self):
        sol = self.make_solution_from_vars(x.values())
        
        # Heuristic separation (external method, user-defined)
        edges_sub, component_sub = separate_heuristic(sol)

        #if len(component_sub) > 0:
            # Subtour elimination cuts
        for i, component in enumerate(component_sub):
            tour_expr = sum(x[e] for e in edges_sub[i])
            ct = tour_expr <= len(component) - 1
            unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
            for ct, cpx_lhs, sense, cpx_rhs in unsats:
                self.add(cpx_lhs, sense, cpx_rhs)
                
        comb_graph = nx.Graph()
        comb_graph.add_nodes_from(graph_new.nodes)
                        
        for e in graph_new.edges:
                            val = sol.get_value(x[e])
                            if val > 1e-7:
                                comb_graph.add_edge(e[0], e[1], weight=val)
                            
                            # Step 2: Get max weight matching
        raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
                            
                            # Step 3: Normalize and filter matching edges
        matching = []
        for e in raw_matching:
            edge = (min(e), max(e))
            if edge in graph_new.edges and sol.get_value(x[edge])>1-1e-7:
                matching.append(edge)
                            
                            # Step 4: Print matching
                          
                            
                            # Step 5: Build meta-graph
        meta_graph = nx.Graph()
        normal_vertex=[]
        meta_graph.add_nodes_from(matching)
        for v in comb_graph.nodes():
            if v not in vertex_list(matching):
                meta_graph.add_node((v,v))
                normal_vertex.append(v)
        for i in range(len(matching)):
            for j in range(i + 1, len(matching)):
                                e1 = matching[i]
                                e2 = matching[j]
                                nodes_e1 = set(e1)
                                nodes_e2 = set(e2)
                                connecting_weight = 0
                                for u in nodes_e1:
                                    for v in nodes_e2:
                                        if graph_new.has_edge(u, v):
                                                key = (u, v) if (u, v) in x else (v, u)
                                                connecting_weight += sol.get_value(x[key])
                                if connecting_weight > 0:
                                        meta_graph.add_edge(e1, e2, weight=min(connecting_weight,1))
        for e in comb_graph.edges:
            if e[0] in normal_vertex and e[1] in normal_vertex:
                                meta_graph.add_edge((e[0],e[0]), (e[1],e[1]), weight=sol.get_value(x[e]))
        for i in normal_vertex:
            for e in matching:
                if has_edge(i,e,graph_new,sol)!=None:
                                    edge,weight_sol=has_edge(i,e,graph_new,sol)
                                    meta_graph.add_edge((i,i),e,weight=weight_sol )
                        #print(meta_graph.edges(data=True))
        chosen_meta = [(u, v) for u, v, data in meta_graph.edges(data=True) if data['weight'] < 1 - 1e-7]
        for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta) ):
                                    handle_vertices=[]
                                    handle_edges = []
                                    teeth_edges =[]
                                    teeth_vertex=[]
                                    internal_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v in component)]
                                    cut_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v not in component) or (u not in component and v in component)]
                                    for (e1,e2) in internal_edge:
                                        for u in e1:
                                            for v in e2:
                                                if graph_new.has_edge(u, v):
                                                    handle_edges.append((min(u, v),max(u,v)))
                                        handle_edges.append(e1)
                                        handle_edges.append(e2)
                                    for (e1,e2) in cut_edge:
                                        teeth=[]
                                        for u in e1:
                                            for v in e2:
                                                if graph_new.has_edge(u, v):
                                                    teeth.append((min(u, v),max(u,v)))
                                        teeth.append(e1)
                                        teeth.append(e2)
                                        teeth_edges.append(teeth)
                                        teeth_vertex.append(vertex_list(teeth))
                                        
                                    if len(cut_edge)%2==1 and len(cut_edge)>=3 and similar_edges(cut_edge)==True:
                                        tour=0
                                        teeth_sum=0
                                        for teeth in teeth_vertex:
                                            teeth_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in teeth and v in teeth]
                                            for e in teeth_edges_in_graph:
                                                teeth_sum+=x[e]
                                        handle_vertices_set = vertex_list(handle_edges)
                                        handle_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in handle_vertices_set and v in handle_vertices_set]
                                        for e in handle_edges_in_graph :
                                            #if e[0] in vertex_list(handle_edges) and e[1] in vertex_list(handle_edges):
                                                tour+=x[e]
        
                                        ct=tour+teeth_sum<= len(handle_vertices_set)+sum(len(teeth_vertices) for teeth_vertices in teeth_vertex )-0.5*(3*len(teeth_vertex)+1) 
                                        unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                                        for ct, cpx_lhs, sense, cpx_rhs in unsats:
                                                self.add(cpx_lhs, sense, cpx_rhs)
def find_max_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        has_edge_with_value_1 = any(self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1 for edge in graph.edges(vertex))
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = max(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1 for edge in graph.edges(vertex))
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))


def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_branch = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1 for edge in graph.edges(vertex))
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = max(edge[2] for edge in fractional_edges)
            #min_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            #min_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == min_weight]
            for e in max_weight_edges:
                fractional_edges_branch.append(e)
            #for e in min_weight_edges:
            #    fractional_edges_branch.append(e)

    return fractional_edges_branch

def find_fractional_edges_with_max_degree_and_edge_value_1(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    max_degree = max(dict(graph.degree).values(), default=0)
    max_degree_vertices = [node for node, degree in graph.degree if degree == max_degree]

    if not max_degree_vertices:
        return []

    fractional_edges_max = []

    for vertex in max_degree_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1 for edge in graph.edges(vertex))
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1/2+1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

def find_fractional_edges_with_max_deg(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    vertice_edge_1=[]
    fractional_edges_max = []

    for vertex in graph.nodes():
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1 for edge in graph.edges(vertex))
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition
        vertice_edge_1.append(vertex)
        # Process fractional edges
    max_deg_vertice=[vertex for vertex in vertice_edge_1 if graph.degree(vertex)==max([graph.degree(vertex) for vertex in vertice_edge_1])]
    for vertex in max_deg_vertice:
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1/2+1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         if len(find_max_min_weight_fractional_edges(self))>1e-7:
             for k in find_max_min_weight_fractional_edges(self):
                    if feas[x[k].index] == infeas:
                        xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                        if xj_inf > 0.5:
                            xj_inf = 1.0 - xj_inf
                            
                        if (xj_inf <= maxinf): 
                            bestj = x[k].index
                            maxinf = xj_inf
                            maxobj = abs(obj[x[k].index])
             if bestj < 0:
                    return
             xj_lo = math.floor(x_val[bestj])
                                    
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                    
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))





class MyBranch2(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         if len(find_fractional_edges_with_max_degree_and_edge_value_1(self))>1e-7:
             for k in find_fractional_edges_with_max_degree_and_edge_value_1(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         elif len(find_fractional_edges_with_max_deg(self))>1e-7:
             for k in find_fractional_edges_with_max_deg(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
           

         
             
        
  
         
        

from docplex.mp.model import Model
#from docplex.mp.model import Priority
m=Model()
x = {(i,j):m.binary_var(name="x_{0}_{1}".format(i,j)) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
#m.parameters.threads.set(1) 
#m.parameters.mip.strategy.search.set(1)
#m.parameters.mip.interval.set(1)
m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(7200)
branch_cb=m.register_callback(MyBranch2)
m.branch_callback=branch_cb
m.parameters.mip.tolerances.mipgap(1e-8)
cut_cb = m.register_callback(SubtourUserCallback_heu)
m.cut_callback = cut_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
values = {}
values.update({x[e]: 1 for e in edge_from_tour(init_tour)})
warmstart=m.new_solution()
warmstart.update( values )
m.add_mip_start(warmstart)
solution=m.solve(log_output=True) 

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_TimeLimit                               7200
CPXPARAM_MIP_Tolerances_MIPGap                   1e-08
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLUD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Processing 1 MIP starts.
MIP start 'm1' defined solution with objective 1748606.0000.
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 1748606.0000.
Dual steepest-edge pricing selected.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional bra

In [493]:
import time
import itertools
def similar_edges(edges):
  no_shared_vertices = True

# Iterate through all pairs of edges
  for edge1, edge2 in itertools.combinations(edges, 2):
      if set(edge1) & set(edge2):  # Check for intersection of vertices
          #print(f"Edges {edge1} and {edge2} share vertices.")
          #no_shared_vertices = False
          return set(edge1) & set(edge2)

  if no_shared_vertices:
      return True
def heuristic_SEC_GH(solution):
    comb_graph = nx.Graph()
    comb_graph.add_nodes_from(graph_new.nodes)
        
    for e in graph_new.edges:
            val = solution.get_value(x[e])
            if val > 1e-7:
                comb_graph.add_edge(e[0], e[1], weight=val)
            
            # Step 2: Get max weight matching
    raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
            
            # Step 3: Normalize and filter matching edges
    matching = []
    for e in raw_matching:
            edge = (min(e), max(e))
            if edge in graph_new.edges and solution.get_value(x[edge])>1-1e-7:
                    matching.append(edge)
            
            # Step 4: Print matching
          
            
            # Step 5: Build meta-graph
    meta_graph = nx.Graph()
    normal_vertex=[]
    meta_graph.add_nodes_from(matching)
    for v in comb_graph.nodes():
            if v not in vertex_list(matching):
               meta_graph.add_node((v,v))
               normal_vertex.append(v)
    for i in range(len(matching)):
        for j in range(i + 1, len(matching)):
                e1 = matching[i]
                e2 = matching[j]
                nodes_e1 = set(e1)
                nodes_e2 = set(e2)
                connecting_weight = 0
                for u in nodes_e1:
                    for v in nodes_e2:
                        if graph_new.has_edge(u, v):
                                key = (u, v) if (u, v) in x else (v, u)
                                connecting_weight += solution.get_value(x[key])
                if connecting_weight > 0:
                        meta_graph.add_edge(e1, e2, capacity=connecting_weight)
    for e in comb_graph.edges:
            if e[0] in normal_vertex and e[1] in normal_vertex:
                meta_graph.add_edge((e[0],e[0]), (e[1],e[1]), capacity=solution.get_value(x[e]))
    for i in normal_vertex:
            for e in matching:
                if has_edge(i,e,graph_new,solution)!=None:
                    edge,weight_sol=has_edge(i,e,graph_new,solution)
                    meta_graph.add_edge((i,i),e,capacity=weight_sol )
    tree = nx.gomory_hu_tree(meta_graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in vertex_list(V1):
                for v in vertex_list(V2):
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons
    
def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons
def has_edge(i,e,graph_new,solution):
    if graph_new.has_edge(i, e[0]) and solution.get_value(x[(min(i,e[0]),max(i,e[0]))])>1e-7:
       return (i, e[0]),solution.get_value(x[(min(i,e[0]),max(i,e[0]))])
        
    elif graph_new.has_edge(i, e[1]) and solution.get_value(x[(min(i,e[1]),max(i,e[1]))])>1e-7:
       return (i, e[1]), solution.get_value(x[(min(i,e[1]),max(i,e[1]))])
    else:
       return None 

def separate_heuristic(sol):
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in sep_graph.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if 2<len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
def vertex_list(edges):
    vertices = set()
    for u, v in edges:
        vertices.add(u)
        vertices.add(v)
    
    # Optional: convert to sorted list
    vertex_list = sorted(vertices)
    return vertex_list

m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve(log_output=True) 
print(solution.get_objective_value())
comb_constr=0
sec_constr=0
sep_time=0
iteration=1
def have_disjoint_edges(edges):
    sum_disjoint=0
    for i in range(len(edges)):
        for j in range(i + 1, len(edges)):
            u1, v1 = edges[i]
            u2, v2 = edges[j]
            if {u1, v1}.isdisjoint({u2, v2}):
                sum_disjoint+=1
    
    return False
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    
    for subtour in heuristic_SEC_GH(solution):
        #if sum(solution.get_value(x[e]) for e in subtour)<2-1e-7:
            m.add_constraint(m.sum(x[e] for e in subtour)>= 2 )
            sec_constr+=1
    end = time.time()
    sep_time+=end-start
    # for subtour in getFracSubtourCons(solution):
    #     m.add_constraint(m.sum(x[e] for e in subtour)>= 2 )
    #     sec_constr+=1
        
    #chosen_edges = [ e for e in graph_new.edges if solution.get_value(x[e]) > 0.5+1e-7 ]
    # edges_sub,component_sub=separate_heuristic(solution)
    # for i in range(len(component_sub)):#nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
    #     #if len(component) < n and len(component)%2==0:
    #        #boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
    #        if sum(solution.get_value(x[e]) for e in edges_sub[i])>len(component_sub[i])-1+1e-7:
    #           m.add_constraint(m.sum(x[e] for e in edges_sub[i])<= len(component_sub[i])-1 )
              #sec_constr+=1

#while m.solve_details.nb_iterations>1e-7:
    comb_graph = nx.Graph()
    comb_graph.add_nodes_from(graph_new.nodes)
        
    for e in graph_new.edges:
            val = solution.get_value(x[e])
            if val > 1e-7:
                comb_graph.add_edge(e[0], e[1], weight=val)
            
            # Step 2: Get max weight matching
    raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
            
            # Step 3: Normalize and filter matching edges
    matching = []
    for e in raw_matching:
            edge = (min(e), max(e))
            if edge in graph_new.edges and solution.get_value(x[edge])>1-1e-7:
                    matching.append(edge)
            
            # Step 4: Print matching
          
            
            # Step 5: Build meta-graph
    meta_graph = nx.Graph()
    normal_vertex=[]
    meta_graph.add_nodes_from(matching)
    for v in comb_graph.nodes():
            if v not in vertex_list(matching):
               meta_graph.add_node((v,v))
               normal_vertex.append(v)
    for i in range(len(matching)):
        for j in range(i + 1, len(matching)):
                e1 = matching[i]
                e2 = matching[j]
                nodes_e1 = set(e1)
                nodes_e2 = set(e2)
                connecting_weight = 0
                for u in nodes_e1:
                    for v in nodes_e2:
                        if graph_new.has_edge(u, v):
                                key = (u, v) if (u, v) in x else (v, u)
                                connecting_weight += solution.get_value(x[key])
                if connecting_weight > 0:
                        meta_graph.add_edge(e1, e2, weight=min(connecting_weight,1))
    for e in comb_graph.edges:
            if e[0] in normal_vertex and e[1] in normal_vertex:
                meta_graph.add_edge((e[0],e[0]), (e[1],e[1]), weight=solution.get_value(x[e]))
    for i in normal_vertex:
            for e in matching:
                if has_edge(i,e,graph_new,solution)!=None:
                    edge,weight_sol=has_edge(i,e,graph_new,solution)
                    meta_graph.add_edge((i,i),e,weight=weight_sol )
        #print(meta_graph.edges(data=True))
    chosen_meta = [(u, v) for u, v, data in meta_graph.edges(data=True) if data['weight'] < 1 - 1e-7]
    for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta)):
                 handle_edges = []
                 teeth_edges = []
                 teeth_vertex = []
            
                 internal_edges = [(u, v) for (u, v) in meta_graph.edges() if u in component and v in component]
                 cut_edges = [(u, v) for (u, v) in meta_graph.edges() if (u in component) != (v in component)]
            
            #     # Handle edges
                 for (e1, e2) in internal_edges:
                     for u in e1:
                         for v in e2:
                             if graph_new.has_edge(u, v):
                                 handle_edges.append((min(u, v), max(u, v)))
                     handle_edges.append(e1)
                     handle_edges.append(e2)
            
            #     # Teeth edges
                 for (e1, e2) in cut_edges:
                     teeth = []
                     for u in e1:
                         for v in e2:
                             if graph_new.has_edge(u, v):
                                 teeth.append((min(u, v), max(u, v)))
                     teeth.append(e1)
                     teeth.append(e2)
                     teeth_edges.append(teeth)
                     teeth_vertex.append(vertex_list(teeth))
                    
                 if len(cut_edges) % 2 == 1 and len(cut_edges) >= 3 and similar_edges(cut_edges)==True:
            #         # Comb constraint
                     teeth_sum = 0
                     teeth_sum_val =0
                     for teeth in teeth_vertex:
                         print('Teeth',teeth)
                         teeth_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in teeth and v in teeth]
                         teeth_sum += m.sum(x[e] for e in teeth_edges_in_graph if e in x)
                         teeth_sum_val+=sum(solution.get_value(x[e]) for e in teeth_edges_in_graph if e in x )
            
                     handle_vertices_set = vertex_list(handle_edges)
                     print('Handle',handle_vertices_set)
                     handle_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in handle_vertices_set and v in handle_vertices_set]
                     handle_sum = m.sum(x[e] for e in handle_edges_in_graph if e in x)
                     rhs = len(handle_vertices_set) + sum(len(tv) for tv in teeth_vertex) - 0.5 * (3 * len(teeth_vertex) + 1)
                     print('Comb RHS:',rhs)
                     if sum(solution.get_value(x[e]) for e in handle_edges_in_graph if e in x)+teeth_sum_val>rhs+1e-7:
                        m.add_constraint(handle_sum + teeth_sum <= rhs) 
        #if len(cut_edges) % 2 == 1 and len(cut_edges) >= 3 and similar_edges(cut_edges)!=True:
            
    solution=m.solve(log_output=True) 
    #for cons in m.iter_constraints():
    #    if cons.slack_value>1e-7:
    #        m.remove_constraint(cons)
    print(solution.get_objective_value())
    iteration+=1
print(sep_time) 
print(sec_constr)
print(iteration)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =       1434118.000000
Iteration:    62   Dual objective     =       1570289.000000
1572396.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1572966.000000
1572966.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1572966.000000
1572967.0
Versio

In [475]:
comb_graph = nx.Graph()
comb_graph.add_nodes_from(graph_new.nodes)
        
for e in graph_new.edges:
            val = solution.get_value(x[e])
            if val > 1e-7:
                comb_graph.add_edge(e[0], e[1], weight=val)
            
            # Step 2: Get max weight matching
raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
            
            # Step 3: Normalize and filter matching edges
matching = []
for e in raw_matching:
            edge = (min(e), max(e))
            if edge in graph_new.edges and solution.get_value(x[edge])>1-1e-7:
                    matching.append(edge)
            
            # Step 4: Print matching
          
            
            # Step 5: Build meta-graph
meta_graph = nx.Graph()
normal_vertex=[]
meta_graph.add_nodes_from(matching)
for v in comb_graph.nodes():
            if v not in vertex_list(matching):
               meta_graph.add_node((v,v))
               normal_vertex.append(v)
for i in range(len(matching)):
        for j in range(i + 1, len(matching)):
                e1 = matching[i]
                e2 = matching[j]
                nodes_e1 = set(e1)
                nodes_e2 = set(e2)
                connecting_weight = 0
                for u in nodes_e1:
                    for v in nodes_e2:
                        if graph_new.has_edge(u, v):
                                key = (u, v) if (u, v) in x else (v, u)
                                connecting_weight += solution.get_value(x[key])
                if connecting_weight > 0:
                        meta_graph.add_edge(e1, e2, weight=min(connecting_weight,1))
for e in comb_graph.edges:
            if e[0] in normal_vertex and e[1] in normal_vertex:
                meta_graph.add_edge((e[0],e[0]), (e[1],e[1]), weight=solution.get_value(x[e]))
for i in normal_vertex:
            for e in matching:
                if has_edge(i,e,graph_new,solution)!=None:
                    edge,weight_sol=has_edge(i,e,graph_new,solution)
                    meta_graph.add_edge((i,i),e,weight=weight_sol )
        #print(meta_graph.edges(data=True))
chosen_meta = [(u, v) for u, v, data in meta_graph.edges(data=True) if data['weight'] < 1 - 1e-7]
for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta)):
    print(component)
    for u, v, data in meta_graph.edges(data=True):
        if u in component and v in component:
            print((u, v), data['weight'])

{(56, 56), (63, 83), (84, 87), (70, 73), (38, 41), (29, 32), (57, 57), (66, 69), (53, 53), (85, 85), (62, 65), (16, 16), (35, 35), (26, 26), (44, 44), (3, 6), (22, 25), (86, 89), (8, 8), (68, 71), (96, 96), (82, 82), (18, 18), (50, 50), (59, 59), (64, 67), (60, 60), (23, 23), (5, 5), (78, 81), (0, 0), (9, 12), (10, 13), (74, 77), (19, 19), (92, 95), (79, 79), (47, 47), (15, 15)}
((86, 89), (63, 83)) 0.19999999999999996
((86, 89), (92, 95)) 0.8
((86, 89), (10, 13)) 0.19999999999999984
((86, 89), (19, 19)) 0.6000000000000001
((86, 89), (57, 57)) 0.19999999999999996
((68, 71), (74, 77)) 0.9700000000000002
((68, 71), (62, 65)) 0.5299999999999998
((68, 71), (79, 79)) 0.029999999999999805
((68, 71), (82, 82)) 0.4700000000000002
((22, 25), (16, 16)) 0.4
((22, 25), (19, 19)) 0.6
((78, 81), (64, 67)) 0.4
((78, 81), (70, 73)) 0.4
((78, 81), (84, 87)) 0.8
((78, 81), (5, 5)) 0.19999999999999996
((78, 81), (8, 8)) 0.19999999999999996
((64, 67), (70, 73)) 0.6
((9, 12), (3, 6)) 0.9700000000000002
((9

In [347]:
import time
import itertools
def similar_edges(edges):
  no_shared_vertices = True

# Iterate through all pairs of edges
  for edge1, edge2 in itertools.combinations(edges, 2):
      if set(edge1) & set(edge2):  # Check for intersection of vertices
          #print(f"Edges {edge1} and {edge2} share vertices.")
          #no_shared_vertices = False
          return set(edge1) & set(edge2)

  if no_shared_vertices:
      return True
def heuristic_SEC_GH(solution):
    comb_graph = nx.Graph()
    comb_graph.add_nodes_from(graph_new.nodes)
        
    for e in graph_new.edges:
            val = solution.get_value(x[e])
            if val > 1e-7:
                comb_graph.add_edge(e[0], e[1], weight=val)
            
            # Step 2: Get max weight matching
    raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
            
            # Step 3: Normalize and filter matching edges
    matching = []
    for e in raw_matching:
            edge = (min(e), max(e))
            if edge in graph_new.edges and solution.get_value(x[edge])>1-1e-7:
                    matching.append(edge)
            
            # Step 4: Print matching
          
            
            # Step 5: Build meta-graph
    meta_graph = nx.Graph()
    normal_vertex=[]
    meta_graph.add_nodes_from(matching)
    for v in comb_graph.nodes():
            if v not in vertex_list(matching):
               meta_graph.add_node((v,v))
               normal_vertex.append(v)
    for i in range(len(matching)):
        for j in range(i + 1, len(matching)):
                e1 = matching[i]
                e2 = matching[j]
                nodes_e1 = set(e1)
                nodes_e2 = set(e2)
                connecting_weight = 0
                for u in nodes_e1:
                    for v in nodes_e2:
                        if graph_new.has_edge(u, v):
                                key = (u, v) if (u, v) in x else (v, u)
                                connecting_weight += solution.get_value(x[key])
                if connecting_weight > 0:
                        meta_graph.add_edge(e1, e2, capacity=min(connecting_weight,1))
    for e in comb_graph.edges:
            if e[0] in normal_vertex and e[1] in normal_vertex:
                meta_graph.add_edge((e[0],e[0]), (e[1],e[1]), capacity=solution.get_value(x[e]))
    for i in normal_vertex:
            for e in matching:
                if has_edge(i,e,graph_new,solution)!=None:
                    edge,weight_sol=has_edge(i,e,graph_new,solution)
                    meta_graph.add_edge((i,i),e,capacity=weight_sol )
    tree = nx.gomory_hu_tree(meta_graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in vertex_list(V1):
                for v in vertex_list(V2):
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons
    
def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons
def has_edge(i,e,graph_new,solution):
    if graph_new.has_edge(i, e[0]) and solution.get_value(x[(min(i,e[0]),max(i,e[0]))])>1e-7:
       return (i, e[0]),solution.get_value(x[(min(i,e[0]),max(i,e[0]))])
        
    elif graph_new.has_edge(i, e[1]) and solution.get_value(x[(min(i,e[1]),max(i,e[1]))])>1e-7:
       return (i, e[1]), solution.get_value(x[(min(i,e[1]),max(i,e[1]))])
    else:
       return None 

def separate_heuristic(sol):
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in sep_graph.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if 2<len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
def vertex_list(edges):
    vertices = set()
    for u, v in edges:
        vertices.add(u)
        vertices.add(v)
    
    # Optional: convert to sorted list
    vertex_list = sorted(vertices)
    return vertex_list

m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve(log_output=True) 
comb_constr=0
sec_constr=0
sep_time=0
iteration=1
def have_disjoint_edges(edges):
    sum_disjoint=0
    for i in range(len(edges)):
        for j in range(i + 1, len(edges)):
            u1, v1 = edges[i]
            u2, v2 = edges[j]
            if {u1, v1}.isdisjoint({u2, v2}):
                sum_disjoint+=1
    
    return False
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    for subtour in getFracSubtourCons(solution):
        if sum(solution.get_value(x[e]) for e in subtour)<2-1e-7:
            m.add_constraint(m.sum(x[e] for e in subtour)>= 2 )
            sec_constr+=1
    end=time.time()
    sep_time+=end-start
    #chosen_edges = [ e for e in graph_new.edges if solution.get_value(x[e]) > 0.5+1e-7 ]
    # edges_sub,component_sub=separate_heuristic(solution)
    # for i in range(len(component_sub)):#nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
    #     #if len(component) < n and len(component)%2==0:
    #        #boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
    #        if sum(solution.get_value(x[e]) for e in edges_sub[i])>len(component_sub[i])-1+1e-7:
    #           m.add_constraint(m.sum(x[e] for e in edges_sub[i])<= len(component_sub[i])-1 )
              #sec_constr+=1

#while m.solve_details.nb_iterations>1e-7:
    # comb_graph = nx.Graph()
    # comb_graph.add_nodes_from(graph_new.nodes)
        
    # for e in graph_new.edges:
    #         val = solution.get_value(x[e])
    #         if val > 1e-7:
    #             comb_graph.add_edge(e[0], e[1], weight=val)
            
    #         # Step 2: Get max weight matching
    # raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
            
    #         # Step 3: Normalize and filter matching edges
    # matching = []
    # for e in raw_matching:
    #         edge = (min(e), max(e))
    #         if edge in graph_new.edges and solution.get_value(x[edge])>1-1e-7:
    #                 matching.append(edge)
            
    #         # Step 4: Print matching
          
            
    #         # Step 5: Build meta-graph
    # meta_graph = nx.Graph()
    # normal_vertex=[]
    # meta_graph.add_nodes_from(matching)
    # for v in comb_graph.nodes():
    #         if v not in vertex_list(matching):
    #            meta_graph.add_node((v,v))
    #            normal_vertex.append(v)
    # for i in range(len(matching)):
    #     for j in range(i + 1, len(matching)):
    #             e1 = matching[i]
    #             e2 = matching[j]
    #             nodes_e1 = set(e1)
    #             nodes_e2 = set(e2)
    #             connecting_weight = 0
    #             for u in nodes_e1:
    #                 for v in nodes_e2:
    #                     if graph_new.has_edge(u, v):
    #                             key = (u, v) if (u, v) in x else (v, u)
    #                             connecting_weight += solution.get_value(x[key])
    #             if connecting_weight > 0:
    #                     meta_graph.add_edge(e1, e2, weight=min(connecting_weight,1))
    # for e in comb_graph.edges:
    #         if e[0] in normal_vertex and e[1] in normal_vertex:
    #             meta_graph.add_edge((e[0],e[0]), (e[1],e[1]), weight=solution.get_value(x[e]))
    # for i in normal_vertex:
    #         for e in matching:
    #             if has_edge(i,e,graph_new,solution)!=None:
    #                 edge,weight_sol=has_edge(i,e,graph_new,solution)
    #                 meta_graph.add_edge((i,i),e,weight=weight_sol )
    #     #print(meta_graph.edges(data=True))
    # chosen_meta = [(u, v) for u, v, data in meta_graph.edges(data=True) if data['weight'] < 1 - 1e-7]
    # for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta)):
    #              handle_edges = []
    #              teeth_edges = []
    #              teeth_vertex = []
            
    #              internal_edges = [(u, v) for (u, v) in meta_graph.edges() if u in component and v in component]
    #              cut_edges = [(u, v) for (u, v) in meta_graph.edges() if (u in component) != (v in component)]
            
    #         #     # Handle edges
    #              for (e1, e2) in internal_edges:
    #                  for u in e1:
    #                      for v in e2:
    #                          if graph_new.has_edge(u, v):
    #                              handle_edges.append((min(u, v), max(u, v)))
    #                  handle_edges.append(e1)
    #                  handle_edges.append(e2)
            
    #         #     # Teeth edges
    #              for (e1, e2) in cut_edges:
    #                  teeth = []
    #                  for u in e1:
    #                      for v in e2:
    #                          if graph_new.has_edge(u, v):
    #                              teeth.append((min(u, v), max(u, v)))
    #                  teeth.append(e1)
    #                  teeth.append(e2)
    #                  teeth_edges.append(teeth)
    #                  teeth_vertex.append(vertex_list(teeth))
                    
    #              if len(cut_edges) % 2 == 1 and len(cut_edges) >= 3 and similar_edges(cut_edges)==True:
    #         #         # Comb constraint
    #                  teeth_sum = 0
    #                  teeth_sum_val =0
    #                  for teeth in teeth_vertex:
    #                      print('Teeth',teeth)
    #                      teeth_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in teeth and v in teeth]
    #                      teeth_sum += m.sum(x[e] for e in teeth_edges_in_graph if e in x)
    #                      teeth_sum_val+=sum(solution.get_value(x[e]) for e in teeth_edges_in_graph if e in x )
            
    #                  handle_vertices_set = vertex_list(handle_edges)
    #                  print('Handle',handle_vertices_set)
    #                  handle_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in handle_vertices_set and v in handle_vertices_set]
    #                  handle_sum = m.sum(x[e] for e in handle_edges_in_graph if e in x)
    #                  rhs = len(handle_vertices_set) + sum(len(tv) for tv in teeth_vertex) - 0.5 * (3 * len(teeth_vertex) + 1)
    #                  print('Comb RHS:',rhs)
    #                  if sum(solution.get_value(x[e]) for e in handle_edges_in_graph if e in x)+teeth_sum_val>rhs+1e-7:
    #                      m.add_constraint(handle_sum + teeth_sum <= rhs) 
        #if len(cut_edges) % 2 == 1 and len(cut_edges) >= 3 and similar_edges(cut_edges)!=True:
            
    solution=m.solve(log_output=True) 
    #for cons in m.iter_constraints():
    #    if cons.slack_value>1e-7:
    #        m.remove_constraint(cons)
    print(solution.get_objective_value())
    iteration+=1
print(sep_time) 
print(sec_constr+comb_constr)
print(iteration)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =       1300196.000000
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1458890.500000
1459795.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1460242.000000
1462119.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_

In [361]:
m.prettyprint()

// This file has been generated by DOcplex
// model name is: docplex_model112
// single vars section
dvar float x_0_2;
dvar float x_0_4;
dvar float x_0_5;
dvar float x_0_6;
dvar float x_0_10;
dvar float x_0_11;
dvar float x_0_13;
dvar float x_0_15;
dvar float x_0_17;
dvar float x_0_19;
dvar float x_0_21;
dvar float x_0_25;
dvar float x_0_26;
dvar float x_0_27;
dvar float x_0_29;
dvar float x_0_31;
dvar float x_0_33;
dvar float x_0_35;
dvar float x_0_39;
dvar float x_0_40;
dvar float x_0_42;
dvar float x_0_43;
dvar float x_0_44;
dvar float x_0_47;
dvar float x_0_48;
dvar float x_0_49;
dvar float x_0_52;
dvar float x_0_53;
dvar float x_0_56;
dvar float x_0_57;
dvar float x_0_61;
dvar float x_0_63;
dvar float x_0_65;
dvar float x_0_66;
dvar float x_0_67;
dvar float x_0_71;
dvar float x_0_73;
dvar float x_0_75;
dvar float x_0_77;
dvar float x_0_79;
dvar float x_0_80;
dvar float x_0_81;
dvar float x_0_85;
dvar float x_0_87;
dvar float x_0_89;
dvar float x_0_91;
dvar float x_0_92;
dvar float

In [495]:
comb_graph = nx.Graph()
comb_graph.add_nodes_from(graph_new.nodes)
for e in graph_new.edges:
    val = solution.get_value(x[e])
    if val > 1e-7:
       comb_graph.add_edge(e[0], e[1], weight=val)
degree_3_vertices = [node for node, degree in comb_graph.degree if degree >= 3]


fractional_edges_branch = []

for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(solution.get_value(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))]) == 1 for edge in comb_graph.edges(vertex))
        for edge in comb_graph.edges(vertex):
            print((min(edge[0], edge[1]), max(edge[0], edge[1])),solution.get_value(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))]))
            
mat_1 =[[0 for i in range(n)] for j in range(n)]
for i in range(n):
      for j in range(n):
          if i!=j and ((min(i,j),max(i,j)) in comb_graph.edges):
              mat_1[i][j] = c[i][j]
          if i!=j and ((min(i,j),max(i,j)) not in comb_graph.edges):
              mat_1[i][j] = 10000000000          
#nearest_neighbor(0,mat)
init_tour= two_opt(nearest_neighbor(0,mat_1),mat)
print(init_tour)
print(total_distance(init_tour,mat))

(4, 10) 0.3333333333333335
(4, 83) 0.9999999999999996
(4, 86) 0.6666666666666665
(5, 8) 1.0
(5, 73) 0.11111111111111094
(5, 81) 0.8888888888888891
(5, 8) 1.0
(8, 14) 0.722222222222222
(8, 84) 0.277777777777778
(4, 10) 0.3333333333333335
(7, 10) 1.0
(10, 13) 0.6666666666666665
(11, 14) 1.0
(11, 17) 0.722222222222222
(11, 84) 0.277777777777778
(10, 13) 0.6666666666666665
(13, 16) 1.0
(13, 22) 0.3333333333333335
(8, 14) 0.722222222222222
(11, 14) 1.0
(14, 20) 0.277777777777778
(13, 16) 1.0
(16, 19) 0.6666666666666671
(16, 89) 0.33333333333333315
(11, 17) 0.722222222222222
(17, 20) 1.0
(17, 26) 0.277777777777778
(16, 19) 0.6666666666666671
(19, 22) 1.0
(19, 25) 0.3333333333333329
(14, 20) 0.277777777777778
(17, 20) 1.0
(20, 23) 0.722222222222222
(13, 22) 0.3333333333333335
(19, 22) 1.0
(22, 28) 0.3333333333333337
(22, 89) 0.3333333333333328
(20, 23) 0.722222222222222
(23, 26) 1.0
(23, 32) 0.277777777777778
(19, 25) 0.3333333333333329
(25, 28) 1.0
(25, 31) 0.666666666666667
(17, 26) 0.27777

In [497]:
from cplex.callbacks import UserCutCallback, LazyConstraintCallback, BranchCallback
from docplex.mp.callbacks.cb_mixin import *
from collections import defaultdict, namedtuple
import cplex.callbacks as cpx_cb
import cplex._internal._constants as cpxcst
import math
import time
import random
def vertex_list(edges):
    vertices = set()
    for u, v in edges:
        vertices.add(u)
        vertices.add(v)
    
    # Optional: convert to sorted list
    vertex_list = sorted(vertices)
    return vertex_list
def match_sep(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
# m=Model()
# x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
# m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
# m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
# #m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
# m.parameters.preprocessing.presolve.set(0)
# m.parameters.threads.set(1) 
# solution=m.solve() 
# print(solution.get_objective_value())
# while m.solve_details.nb_iterations>1e-7:
#     start=time.time()
#     #chosen_edges = [ e for e in graph_new.edges if solution.get_value(x[e]) > 0.5+1e-7 ]
#     for component in match_sep(solution):#nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
#         if len(component) < n and len(component)%2==0:
#            boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
#            if sum(solution.get_value(x[e]) for e in boundary_edges)>len(component)-1+1e-7:
#               m.add_constraint(m.sum(x[e] for e in boundary_edges)<= len(component)-1 )
              
#     solution=m.solve(log_output=True)
              
    #comb_graph = nx.Graph()
    #comb_graph.add_nodes_from(graph_new.nodes)
    
    #for e in graph_new.edges:
    #    val = solution.get_value(x[e])
    #    if val > 1e-7:
    #        comb_graph.add_edge(e[0], e[1], weight=val)
    
    # Step 2: Get max weight matching
    #raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
    
    # Step 3: Normalize and filter matching edges
    #matching = []
    #for e in raw_matching:
    #    edge = (min(e), max(e))
    #    if edge in graph_new.edges and solution.get_value(x[edge])>0.5+1e-7:
    #        matching.append(edge)
    
    # Step 4: Print matching
    
    
    # Step 5: Build meta-graph
    #meta_graph = nx.Graph()
    #meta_graph.add_nodes_from(matching)
    
    #for i in range(len(matching)):
    #    for j in range(i + 1, len(matching)):
    #        e1 = matching[i]
    #        e2 = matching[j]
    #        nodes_e1 = set(e1)
    #        nodes_e2 = set(e2)
    #        connecting_weight = 0
    #        for u in nodes_e1:
    #            for v in nodes_e2:
    #                if graph_new.has_edge(u, v):
    #                    key = (u, v) if (u, v) in x else (v, u)
    #                    connecting_weight += solution.get_value(x[key])
    #        if connecting_weight > 0:
    #            meta_graph.add_edge(e1, e2, weight=min(connecting_weight,1))
    
    # Step 6: Process weakly connected meta components (comb candidates)
    # chosen_meta = [(u, v) for u, v, data in meta_graph.edges(data=True) if data['weight'] < 1 - 1e-7]
    
    # for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta)):
    #     handle_edges = []
    #     teeth_edges = []
    #     teeth_vertex = []
    
    #     internal_edges = [(u, v) for (u, v) in meta_graph.edges() if u in component and v in component]
    #     cut_edges = [(u, v) for (u, v) in meta_graph.edges() if (u in component) != (v in component)]
    
    #     # Handle edges
    #     for (e1, e2) in internal_edges:
    #         for u in e1:
    #             for v in e2:
    #                 if graph_new.has_edge(u, v):
    #                     handle_edges.append((min(u, v), max(u, v)))
    #         handle_edges.append(e1)
    #         handle_edges.append(e2)
    
    #     # Teeth edges
    #     for (e1, e2) in cut_edges:
    #         teeth = []
    #         for u in e1:
    #             for v in e2:
    #                 if graph_new.has_edge(u, v):
    #                     teeth.append((min(u, v), max(u, v)))
    #         teeth.append(e1)
    #         teeth.append(e2)
    #         teeth_edges.append(teeth)
    #         teeth_vertex.append(vertex_list(teeth))
            
    #     if len(cut_edges) % 2 == 1 and len(cut_edges) >= 3 and similar_edges(cut_edges)==True:
    #         # Comb constraint
    #         teeth_sum = 0
    #         teeth_sum_val =0
    #         for teeth in teeth_vertex:
    #             print('Teeth',teeth)
    #             teeth_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in teeth and v in teeth]
    #             teeth_sum += m.sum(x[e] for e in teeth_edges_in_graph if e in x)
    #             teeth_sum_val+=sum(solution.get_value(x[e]) for e in teeth_edges_in_graph if e in x )
    
    #         handle_vertices_set = vertex_list(handle_edges)
    #         print('Handle',handle_vertices_set)
    #         handle_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in handle_vertices_set and v in handle_vertices_set]
    #         handle_sum = m.sum(x[e] for e in handle_edges_in_graph if e in x)
    #         rhs = len(handle_vertices_set) + sum(len(tv) for tv in teeth_vertex) - 0.5 * (3 * len(teeth_vertex) + 1)
    #         if sum(solution.get_value(x[e]) for e in handle_edges_in_graph if e in x)+teeth_sum_val>rhs+1e-7:
    #             m.add_constraint(handle_sum + teeth_sum <= rhs)  
    #             comb_constr+=1
    #     #if len(cut_edges) % 2 == 1 and len(cut_edges) >= 3 and similar_edges(cut_edges)!=True:
            
    #solution=m.solve(log_output=True) 
    #for cons in m.iter_constraints():
    #     if cons.slack_value>1e-7:
    #         m.remove_constraint(cons)
    #print(solution.get_objective_value())
def removed_graph(G,edge_set):
    for e in edge_set:
        G.remove_edge(e[0],e[1])
    return G
        
def construct_hamiltonian_tour(cplex_solution, nodes):
    adjacency = {node: [] for node in nodes}
    for i, j in cplex_solution:
        adjacency[i].append(j)
        adjacency[j].append(i)
    
    # Start with an arbitrary node
    start_node = nodes[0]
    current_node = start_node
    tour = [current_node]
    visited = set([current_node])
    
    while len(visited) < len(nodes):
        # Visit the next unvisited node
        for neighbor in adjacency[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                tour.append(neighbor)
                current_node = neighbor
                break
    
    # Close the tour
    tour.append(start_node)
    return tour
def k_factor(solution):
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    #k_factor(sep_graph, 2, matching_weight='weight')
    print(k_factor(sep_graph, 2, matching_weight='weight'))
    
def match_sep_1(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                matching_1=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                for e in nx.max_weight_matching(removed_graph(G_comp,matching),weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching_1.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True or nx.is_perfect_matching(G_comp, matching_1)==True  and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets

def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree == 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph
def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_fractional_edges_with_max_degree_and_edge_value_1(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    max_degree = max(dict(graph.degree).values(), default=0)
    max_degree_vertices = [node for node, degree in graph.degree if degree == max_degree]

    if not max_degree_vertices:
        return []

    fractional_edges_max = []

    for vertex in max_degree_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1/2+1e-7< edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

def find_fractional_edges_with_max_deg(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    vertice_edge_1=[]
    fractional_edges_max = []

    for vertex in graph.nodes():
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition
        vertice_edge_1.append(vertex)
        # Process fractional edges
    max_deg_vertice=[vertex for vertex in vertice_edge_1 if graph.degree(vertex)==max([graph.degree(vertex) for vertex in vertice_edge_1])]
    for vertex in max_deg_vertice:
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1/2+1e-7< edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class CustomHeuristicCallback(ModelCallbackMixin, cpx_cb.HeuristicCallback):
    def __init__(self, env):
        # Initialize the parent classes
        cpx_cb.HeuristicCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_heuristics = 0

    def __call__(self):
        x_val= self.make_solution_from_vars(x.values())
        self.nb_heuristics += 1
        tour_edges=[]
        for e in graph_new.edges:
            if x_val.get_value(x[e])>1-1e-7:
                tour_edges.append(e)
        
        # Example heuristic to construct a feasible solution
        gap=self.get_MIP_relative_gap()
        nodes = graph_new.nodes()
        if len(tour_edges)==len(nodes):
            initial_tour = construct_hamiltonian_tour(tour_edges,list(nodes))
            obj_value = total_distance(initial_tour, mat)
    
            # Check feasibility of the solution (optional, depends on your logic)
            tour1=two_opt(initial_tour,mat)
            if total_distance(tour1,mat)<total_distance(initial_tour,mat):
                var_edges=edge_from_tour(tour1)
                
                var_index = [x[e] for e in var_edges]
                self.set_solution(var_index, [1] * len(var_index))
        
            else:
                var_edges=edge_from_tour(initial_tour)
                var_index = [x[e] for e in var_edges]
                self.set_solution(var_index, [1] * len(var_index))
class CustomIncumbentCallback(ModelCallbackMixin, cpx_cb.IncumbentCallback):

    incumbent_sources = {
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_NODESOLN: 'node',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_HEURSOLN: 'heuristic',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_USERSOLN: 'user',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_MIPSTART: 'mipstart'
    }

    def __init__(self, env):
        # non public...
        cpx_cb.IncumbentCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_incumbents = 0

    def __call__(self):
        self.nb_incumbents += 1
        obj = self.get_objective_value()
        src = self.get_solution_source()
        src_name = self.incumbent_sources.get(src, ' unknown???')
        print('{0}> found incumbent with objective value {1}, coming from: {2}'.format(self.nb_incumbents, obj, src_name))       
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         #if len(set(find_min_weight_fractional_edges(self))&set(min_matching(self)))!=0:
             
         #    for k in set(find_max_weight_fractional_edges(self))&set(max_matching(self)):
         #               if feas[x[k].index] == infeas:
         #                   xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
         #                   if xj_inf > 0.5:
         #                       xj_inf = 1.0 - xj_inf
                                
         #                   if (xj_inf <= maxinf): 
         #                       bestj = x[k].index
         #                       maxinf = xj_inf
         #                       maxobj = abs(obj[x[k].index])
         #    if bestj < 0: 
         #             return 
         #    xj_lo = math.floor(x_val[bestj])
                                        
         #    dv = self.index_to_var(bestj)
         #    self.stats[dv] += 1
                                        
         #    self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         #    self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         #elif(gap>0.03+1e-7 and gap<0.10-1e-7):
         #else:
         if (len( find_max_min_weight_fractional_edges(self))>1e-7):
             for k in find_max_min_weight_fractional_edges(self):
                    if feas[x[k].index] == infeas:
                        xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                        if xj_inf > 0.5:
                            xj_inf = 1.0 - xj_inf
                            
                        if (xj_inf <= maxinf): 
                            bestj = x[k].index
                            maxinf = xj_inf
                            maxobj = abs(obj[x[k].index])
             if bestj < 0:
                    return
             xj_lo = math.floor(x_val[bestj])
                                    
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                    
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))

class MyBranch2(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         if len(find_fractional_edges_with_max_degree_and_edge_value_1(self))>1e-7:
              for k in find_fractional_edges_with_max_degree_and_edge_value_1(self):
                         if feas[x[k].index] == infeas:
                             xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                             if xj_inf > 0.5:
                                 xj_inf = 1.0 - xj_inf
                                
                             if (xj_inf <= maxinf): 
                                 bestj = x[k].index
                                 maxinf = xj_inf
                                 maxobj = abs(obj[x[k].index])
              if bestj < 0:
                         return
              xj_lo = math.floor(x_val[bestj])
                                        
              dv = self.index_to_var(bestj)
              self.stats[dv] += 1
                                        
              self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
              self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         elif len(find_fractional_edges_with_max_deg(self))>1e-7:
             for k in find_fractional_edges_with_max_deg(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v>1e-7): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
                 if len(component) < n and len(component)>=3:
                     edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                     tour=0
                     for e in edge_in_component:
                         tour += x[e]
                     ct = tour <=  len(component)- 1
                     unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                     for ct, cpx_lhs, sense, cpx_rhs in unsats:
        
                         self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
#m.parameters.mip.strategy.heuristicfreq = -1
#m.parameters.mip.tolerances.mipgap(1e-8)
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch2)
m.branch_callback=branch_cb
m.register_callback(CustomIncumbentCallback)
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
values = {}
values.update({x[e]: 1 for e in edge_from_tour(init_tour)})
warmstart=m.new_solution()
warmstart.update( values )
m.add_mip_start(warmstart)
solution1=m.solve(log_output=True)
        

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLID
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Processing 1 MIP starts.
1> found incumbent with objective value 1702865.0, coming from: mipstart
MIP start 'm1' defined solution with objective 1702865.0000.
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 1702865.0000.
Dual steepest-edge pricing selected.
MIP emphasis: ba

In [489]:
from cplex.callbacks import UserCutCallback, LazyConstraintCallback, BranchCallback
from docplex.mp.callbacks.cb_mixin import *
from collections import defaultdict, namedtuple
import cplex.callbacks as cpx_cb
import cplex._internal._constants as cpxcst
import math
import time
import random
def vertex_list(edges):
    vertices = set()
    for u, v in edges:
        vertices.add(u)
        vertices.add(v)
    
    # Optional: convert to sorted list
    vertex_list = sorted(vertices)
    return vertex_list
def match_sep(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
print(solution.get_objective_value())
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    #chosen_edges = [ e for e in graph_new.edges if solution.get_value(x[e]) > 0.5+1e-7 ]
    edges_sub,component_sub=separate_heuristic(solution)
   
    for i in range(len(component_sub)):#nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
            #if len(component) < n and len(component)%2==0:
               #boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
               if sum(solution.get_value(x[e]) for e in edges_sub[i])>len(component_sub[i])-1+1e-7:
                  m.add_constraint(m.sum(x[e] for e in edges_sub[i])<= len(component_sub[i])-1 )
  
    for subtour in getFracSubtourCons(solution):
            m.add_constraint(m.sum(x[e] for e in subtour)>= 2 )
        
              
    solution=m.solve(log_output=True)
              
    comb_graph = nx.Graph()
    comb_graph.add_nodes_from(graph_new.nodes)
    
    for e in graph_new.edges:
        val = solution.get_value(x[e])
        if val > 1e-7:
            comb_graph.add_edge(e[0], e[1], weight=val)
    
    # Step 2: Get max weight matching
    raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
    
    # Step 3: Normalize and filter matching edges
    matching = []
    for e in raw_matching:
        edge = (min(e), max(e))
        if edge in graph_new.edges and solution.get_value(x[edge])>1-1e-7:
            matching.append(edge)
    
    # Step 4: Print matching
    
    
    # Step 5: Build meta-graph
    meta_graph = nx.Graph()
    meta_graph.add_nodes_from(matching)
    
    for i in range(len(matching)):
        for j in range(i + 1, len(matching)):
            e1 = matching[i]
            e2 = matching[j]
            nodes_e1 = set(e1)
            nodes_e2 = set(e2)
            connecting_weight = 0
            for u in nodes_e1:
                for v in nodes_e2:
                    if graph_new.has_edge(u, v):
                        key = (u, v) if (u, v) in x else (v, u)
                        connecting_weight += solution.get_value(x[key])
            if connecting_weight > 0:
                meta_graph.add_edge(e1, e2, weight=min(connecting_weight,1))
    
    # Step 6: Process weakly connected meta components (comb candidates)
    chosen_meta = [(u, v) for u, v, data in meta_graph.edges(data=True) if data['weight'] < 1 - 1e-7]
    
    for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta)):
         handle_edges = []
         teeth_edges = []
         teeth_vertex = []
    
         internal_edges = [(u, v) for (u, v) in meta_graph.edges() if u in component and v in component]
         cut_edges = [(u, v) for (u, v) in meta_graph.edges() if (u in component) != (v in component)]
    
    #     # Handle edges
         for (e1, e2) in internal_edges:
             for u in e1:
                 for v in e2:
                     if graph_new.has_edge(u, v):
                         handle_edges.append((min(u, v), max(u, v)))
             handle_edges.append(e1)
             handle_edges.append(e2)
    
    #     # Teeth edges
         for (e1, e2) in cut_edges:
             teeth = []
             for u in e1:
                 for v in e2:
                     if graph_new.has_edge(u, v):
                         teeth.append((min(u, v), max(u, v)))
             teeth.append(e1)
             teeth.append(e2)
             teeth_edges.append(teeth)
             teeth_vertex.append(vertex_list(teeth))
            
         if len(cut_edges) % 2 == 1 and len(cut_edges) >= 3 and similar_edges(cut_edges)==True:
    #         # Comb constraint
             teeth_sum = 0
             teeth_sum_val =0
             for teeth in teeth_vertex:
                 print('Teeth',teeth)
                 teeth_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in teeth and v in teeth]
                 teeth_sum += m.sum(x[e] for e in teeth_edges_in_graph if e in x)
                 teeth_sum_val+=sum(solution.get_value(x[e]) for e in teeth_edges_in_graph if e in x )
    
             handle_vertices_set = vertex_list(handle_edges)
             print('Handle',handle_vertices_set)
             handle_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in handle_vertices_set and v in handle_vertices_set]
             handle_sum = m.sum(x[e] for e in handle_edges_in_graph if e in x)
             rhs = len(handle_vertices_set) + sum(len(tv) for tv in teeth_vertex) - 0.5 * (3 * len(teeth_vertex) + 1)
             print('Comb RHS:',rhs)
             if sum(solution.get_value(x[e]) for e in handle_edges_in_graph if e in x)+teeth_sum_val>rhs+1e-7:
                 m.add_constraint(handle_sum + teeth_sum <= rhs)  
    #             comb_constr+=1
    #     #if len(cut_edges) % 2 == 1 and len(cut_edges) >= 3 and similar_edges(cut_edges)!=True:
            
    solution=m.solve(log_output=True) 
    #for cons in m.iter_constraints():
    #     if cons.slack_value>1e-7:
    #         m.remove_constraint(cons)
    print(solution.get_objective_value())

1164241.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1164748.000000
Teeth [75, 78, 81, 84]
Teeth [82, 85, 86, 87]
Teeth [74, 77, 80, 83]
Handle [80, 81, 83, 84, 86, 87]
Comb RHS: 13.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1186601.000000
1186601.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1186611.5000

In [345]:
def has_edge(i,e,graph_new,solution):
    if graph_new.has_edge(i, e[0]) and solution.get_value(x[(min(i,e[0]),max(i,e[0]))])>1e-7:
       return (i, e[0]),solution.get_value(x[(min(i,e[0]),max(i,e[0]))])
        
    elif graph_new.has_edge(i, e[1]) and solution.get_value(x[(min(i,e[1]),max(i,e[1]))])>1e-7:
       return (i, e[1]), solution.get_value(x[(min(i,e[1]),max(i,e[1]))])
    else:
       return None   
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
print(solution.get_objective_value())
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    #chosen_edges = [ e for e in graph_new.edges if solution.get_value(x[e]) > 0.5+1e-7 ]
    edges_sub,component_sub=separate_heuristic(solution)
    for i in range(len(component_sub)):
           if sum(solution.get_value(x[e]) for e in edges_sub[i])>len(component_sub[i])-1+1e-7:
              m.add_constraint(m.sum(x[e] for e in edges_sub[i])<= len(component_sub[i])-1 )
              
    solution=m.solve(log_output=True)
    comb_graph = nx.Graph()
    comb_graph.add_nodes_from(graph_new.nodes)
    
    for e in graph_new.edges:
        val = solution.get_value(x[e])
        if val > 1e-7:
            comb_graph.add_edge(e[0], e[1], weight=val)
        
        # Step 2: Get max weight matching
    raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
        
        # Step 3: Normalize and filter matching edges
    matching = []
    for e in raw_matching:
        edge = (min(e), max(e))
        if edge in graph_new.edges and solution.get_value(x[edge])>0.5+1e-7:
                matching.append(edge)
        
        # Step 4: Print matching
      
        
        # Step 5: Build meta-graph
    meta_graph = nx.Graph()
    normal_vertex=[]
    meta_graph.add_nodes_from(matching)
    for v in comb_graph.nodes():
        if v not in vertex_list(matching):
           meta_graph.add_node((v,v))
           normal_vertex.append(v)
    for i in range(len(matching)):
        for j in range(i + 1, len(matching)):
            e1 = matching[i]
            e2 = matching[j]
            nodes_e1 = set(e1)
            nodes_e2 = set(e2)
            connecting_weight = 0
            for u in nodes_e1:
                for v in nodes_e2:
                    if graph_new.has_edge(u, v):
                            key = (u, v) if (u, v) in x else (v, u)
                            connecting_weight += solution.get_value(x[key])
            if connecting_weight > 0:
                    meta_graph.add_edge(e1, e2, weight=min(connecting_weight,1))
    for e in comb_graph.edges:
        if e[0] in normal_vertex and e[1] in normal_vertex:
            meta_graph.add_edge((e[0],e[0]), (e[1],e[1]), weight=solution.get_value(x[e]))
    for i in normal_vertex:
        for e in matching:
            if has_edge(i,e,graph_new,solution)!=None:
                edge,weight_sol=has_edge(i,e,graph_new,solution)
                meta_graph.add_edge((i,i),e,weight=weight_sol )
    #print(meta_graph.edges(data=True))
    chosen_meta = [(u, v) for u, v, data in meta_graph.edges(data=True) if data['weight'] < 1 - 1e-7]
    for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta)):
             handle_edges = []
             teeth_edges = []
             teeth_vertex = []
        
             internal_edges = [(u, v) for (u, v) in meta_graph.edges() if u in component and v in component]
             cut_edges = [(u, v) for (u, v) in meta_graph.edges() if (u in component) != (v in component)]
        
        #     # Handle edges
             for (e1, e2) in internal_edges:
                 for u in e1:
                     for v in e2:
                         if graph_new.has_edge(u, v):
                             handle_edges.append((min(u, v), max(u, v)))
                 handle_edges.append(e1)
                 handle_edges.append(e2)
        
        #     # Teeth edges
             for (e1, e2) in cut_edges:
                 teeth = []
                 for u in e1:
                     for v in e2:
                         if graph_new.has_edge(u, v):
                             teeth.append((min(u, v), max(u, v)))
                 teeth.append(e1)
                 teeth.append(e2)
                 teeth_edges.append(teeth)
                 teeth_vertex.append(vertex_list(teeth))
                
             if len(cut_edges) % 2 == 1 and len(cut_edges) >= 3 and similar_edges(cut_edges)==True:
        #         # Comb constraint
                 teeth_sum = 0
                 teeth_sum_val =0
                 for teeth in teeth_vertex:
                     print('Teeth',teeth)
                     teeth_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in teeth and v in teeth]
                     teeth_sum += m.sum(x[e] for e in teeth_edges_in_graph if e in x)
                     teeth_sum_val+=sum(solution.get_value(x[e]) for e in teeth_edges_in_graph if e in x )
        
                 handle_vertices_set = vertex_list(handle_edges)
                 print('Handle',handle_vertices_set)
                 handle_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in handle_vertices_set and v in handle_vertices_set]
                 handle_sum = m.sum(x[e] for e in handle_edges_in_graph if e in x)
                 rhs = len(handle_vertices_set) + sum(len(tv) for tv in teeth_vertex) - 0.5 * (3 * len(teeth_vertex) + 1)
                 print('Comb RHS:',rhs)
                 if sum(solution.get_value(x[e]) for e in handle_edges_in_graph if e in x)+teeth_sum_val>rhs+1e-7:
                     m.add_constraint(handle_sum + teeth_sum <= rhs)      
    
    solution=m.solve(log_output=True)
    print(solution.get_objective_value())
              

1439993.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1441337.000000
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
1446039.0


In [477]:
from cplex.callbacks import UserCutCallback, LazyConstraintCallback, BranchCallback
from docplex.mp.callbacks.cb_mixin import *
from collections import defaultdict, namedtuple
import cplex.callbacks as cpx_cb
import cplex._internal._constants as cpxcst
import math
import time
import random
def vertex_list(edges):
    vertices = set()
    for u, v in edges:
        vertices.add(u)
        vertices.add(v)
    
    # Optional: convert to sorted list
    vertex_list = sorted(vertices)
    return vertex_list
def match_sep(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets

def removed_graph(G,edge_set):
    for e in edge_set:
        G.remove_edge(e[0],e[1])
    return G
        
def construct_hamiltonian_tour(cplex_solution, nodes):
    adjacency = {node: [] for node in nodes}
    for i, j in cplex_solution:
        adjacency[i].append(j)
        adjacency[j].append(i)
    
    # Start with an arbitrary node
    start_node = nodes[0]
    current_node = start_node
    tour = [current_node]
    visited = set([current_node])
    
    while len(visited) < len(nodes):
        # Visit the next unvisited node
        for neighbor in adjacency[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                tour.append(neighbor)
                current_node = neighbor
                break
    
    # Close the tour
    tour.append(start_node)
    return tour
def k_factor(solution):
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    #k_factor(sep_graph, 2, matching_weight='weight')
    print(k_factor(sep_graph, 2, matching_weight='weight'))
    
def match_sep_1(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                matching_1=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                for e in nx.max_weight_matching(removed_graph(G_comp,matching),weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching_1.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True or nx.is_perfect_matching(G_comp, matching_1)==True  and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets

def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree == 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph
def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_fractional_edges_with_max_degree_and_edge_value_1(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    max_degree = max(dict(graph.degree).values(), default=0)
    max_degree_vertices = [node for node, degree in graph.degree if degree == max_degree]

    if not max_degree_vertices:
        return []

    fractional_edges_max = []

    for vertex in max_degree_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1/2+1e-7< edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

def find_fractional_edges_with_max_deg(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    vertice_edge_1=[]
    fractional_edges_max = []

    for vertex in graph.nodes():
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition
        vertice_edge_1.append(vertex)
        # Process fractional edges
    max_deg_vertice=[vertex for vertex in vertice_edge_1 if graph.degree(vertex)==max([graph.degree(vertex) for vertex in vertice_edge_1])]
    for vertex in max_deg_vertice:
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1/2+1e-7< edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class CustomHeuristicCallback(ModelCallbackMixin, cpx_cb.HeuristicCallback):
    def __init__(self, env):
        # Initialize the parent classes
        cpx_cb.HeuristicCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_heuristics = 0

    def __call__(self):
        x_val= self.make_solution_from_vars(x.values())
        self.nb_heuristics += 1
        tour_edges=[]
        for e in graph_new.edges:
            if x_val.get_value(x[e])>1-1e-7:
                tour_edges.append(e)
        
        # Example heuristic to construct a feasible solution
        gap=self.get_MIP_relative_gap()
        nodes = graph_new.nodes()
        if len(tour_edges)==len(nodes):
            initial_tour = construct_hamiltonian_tour(tour_edges,list(nodes))
            obj_value = total_distance(initial_tour, mat)
    
            # Check feasibility of the solution (optional, depends on your logic)
            tour1=two_opt(initial_tour,mat)
            if total_distance(tour1,mat)<total_distance(initial_tour,mat):
                var_edges=edge_from_tour(tour1)
                
                var_index = [x[e] for e in var_edges]
                self.set_solution(var_index, [1] * len(var_index))
        
            else:
                var_edges=edge_from_tour(initial_tour)
                var_index = [x[e] for e in var_edges]
                self.set_solution(var_index, [1] * len(var_index))
class CustomIncumbentCallback(ModelCallbackMixin, cpx_cb.IncumbentCallback):

    incumbent_sources = {
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_NODESOLN: 'node',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_HEURSOLN: 'heuristic',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_USERSOLN: 'user',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_MIPSTART: 'mipstart'
    }

    def __init__(self, env):
        # non public...
        cpx_cb.IncumbentCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_incumbents = 0

    def __call__(self):
        self.nb_incumbents += 1
        obj = self.get_objective_value()
        src = self.get_solution_source()
        src_name = self.incumbent_sources.get(src, ' unknown???')
        print('{0}> found incumbent with objective value {1}, coming from: {2}'.format(self.nb_incumbents, obj, src_name))       
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         #if len(set(find_min_weight_fractional_edges(self))&set(min_matching(self)))!=0:
             
         #    for k in set(find_max_weight_fractional_edges(self))&set(max_matching(self)):
         #               if feas[x[k].index] == infeas:
         #                   xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
         #                   if xj_inf > 0.5:
         #                       xj_inf = 1.0 - xj_inf
                                
         #                   if (xj_inf <= maxinf): 
         #                       bestj = x[k].index
         #                       maxinf = xj_inf
         #                       maxobj = abs(obj[x[k].index])
         #    if bestj < 0: 
         #             return 
         #    xj_lo = math.floor(x_val[bestj])
                                        
         #    dv = self.index_to_var(bestj)
         #    self.stats[dv] += 1
                                        
         #    self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         #    self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         #elif(gap>0.03+1e-7 and gap<0.10-1e-7):
         #else:
         if (len( find_max_min_weight_fractional_edges(self))>1e-7):
             for k in find_max_min_weight_fractional_edges(self):
                    if feas[x[k].index] == infeas:
                        xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                        if xj_inf > 0.5:
                            xj_inf = 1.0 - xj_inf
                            
                        if (xj_inf <= maxinf): 
                            bestj = x[k].index
                            maxinf = xj_inf
                            maxobj = abs(obj[x[k].index])
             if bestj < 0:
                    return
             xj_lo = math.floor(x_val[bestj])
                                    
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                    
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))

class MyBranch2(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         if len(find_fractional_edges_with_max_degree_and_edge_value_1(self))>1e-7:
             for k in find_fractional_edges_with_max_degree_and_edge_value_1(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         elif len(find_fractional_edges_with_max_deg(self))>1e-7:
             for k in find_fractional_edges_with_max_deg(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
#m.parameters.mip.tolerances.mipgap(1e-8)
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch2)
m.branch_callback=branch_cb
m.register_callback(CustomIncumbentCallback)
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
values = {}
values.update({x[e]: 1 for e in edge_from_tour(init_tour)})
warmstart=m.new_solution()
warmstart.update( values )
m.parameters.mip.strategy.heuristicfreq = -1
m.parameters.mip.strategy.rinsheur = -1
m.add_mip_start(warmstart)
solution1=m.solve(log_output=True)
        

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_MIP_Strategy_HeuristicFreq              -1
CPXPARAM_MIP_Strategy_RINSHeur                   -1
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLID
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Processing 1 MIP starts.
1> found incumbent with objective value 1352392.0, coming from: mipstart
MIP start 'm1' defined solution with objective 1352392.0000.
1 of 1 MIP starts provided solutions.
MIP start 'm1' def

In [1355]:
import networkx as nx
def vertex_list(edges):
    vertices = set()
    for u, v in edges:
        vertices.add(u)
        vertices.add(v)
    
    # Optional: convert to sorted list
    vertex_list = sorted(vertices)
    return vertex_list
# Step 1: Build comb_graph from solution
comb_graph = nx.Graph()
comb_graph.add_nodes_from(graph_new.nodes)
for e in graph_new.edges:
    val = solution.get_value(x[e])
    if val > 1e-7:
        comb_graph.add_edge(e[0], e[1], weight=val)

# Step 2: Get max weight matching
raw_matching = nx.max_weight_matching(comb_graph, weight='weight')

# Normalize edge representation
matching = []
for e in raw_matching:
    edge = (min(e), max(e))
    if edge in graph_new.edges and solution.get_value(x[edge])==1:
        matching.append(edge)

# Step 3: If perfect matching, print
#if nx.is_perfect_matching(comb_graph, matching):
for e in matching:
        print(e, solution.get_value(x[e]))

    # Step 4: Build meta-graph where each node is a matching edge
meta_graph = nx.Graph()
meta_graph.add_nodes_from(matching)

    # Step 5: Connect matching edges that share neighbors in original graph
for i in range(len(matching)):
        for j in range(i + 1, len(matching)):
            e1 = matching[i]
            e2 = matching[j]

            # Get all 4 nodes
            nodes_e1 = set(e1)
            nodes_e2 = set(e2)

            # Check all connections between nodes of e1 and e2
            connecting_weight = 0
            for u in nodes_e1:
                for v in nodes_e2:
                    if graph_new.has_edge(u, v):
                        connecting_weight += solution.get_value(x[(u, v)]) if (u, v) in x else solution.get_value(x[(v, u)])

            if connecting_weight > 0:
                meta_graph.add_edge(e1, e2, weight=connecting_weight)

    # Step 6: Print meta-graph edges
chosen_meta=[(u,v) for u, v, data in meta_graph.edges(data=True) if data['weight']<1-1e-7]
def is_in_handle_vertices(e, vertex_pairs):
    return any(u in e and v in e for u, v in vertex_pairs)
for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta) ):
    handle_vertices=[]
    handle_edges = []
    teeth_edges =[]
    teeth_vertex=[]
    print(component)
        
    
    internal_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v in component)]
    cut_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v not in component) or (u not in component and v in component)]
    for (e1,e2) in internal_edge:
        for u in e1:
            for v in e2:
                if graph_new.has_edge(u, v):
                    handle_edges.append((min(u, v),max(u,v)))
        handle_edges.append(e1)
        handle_edges.append(e2)
    for (e1,e2) in cut_edge:
        teeth=[]
        for u in e1:
            for v in e2:
                if graph_new.has_edge(u, v):
                    teeth.append((min(u, v),max(u,v)))
        teeth.append(e1)
        teeth.append(e2)
        teeth_edges.append(teeth)
        teeth_vertex.append(vertex_list(teeth))
        print(teeth)
    print(teeth_vertex)
    if len(cut_edge)%2==1 and len(cut_edge)>=3:
        teeth_sum=0
        for teeth in teeth_vertex:
            teeth_sum+=m.sum(x[e] for e in graph_new.edges if e[0] in vertex_list(teeth) and e[1] in vertex_list(teeth))
        m.add_constraint(m.sum(x[e] for e in graph_new.edges if e[0] in vertex_list(handle_edges) and e[1] in vertex_list(handle_edges))+teeth_sum <= len(vertex_list(handle_edges))+sum(len(teeth_vertices) for teeth_vertices in teeth_vertex )-0.5*(3*len(teeth_vertex)+1) )
solution=m.solve(log_output=True)
print(solution.get_objective_value())

(104, 107) 1.0
(84, 95) 1.0
(49, 52) 1.0
(35, 38) 1.0
(100, 103) 1.0
(67, 70) 1.0
(77, 80) 1.0
(30, 33) 1.0
(54, 57) 1.0
(36, 39) 1.0
(31, 34) 1.0
(72, 75) 1.0
(73, 76) 1.0
(59, 62) 1.0
(20, 26) 1.0
(13, 16) 1.0
(87, 90) 1.0
(78, 81) 1.0
(55, 58) 1.0
(105, 108) 1.0
(41, 44) 1.0
(18, 21) 1.0
(110, 113) 1.0
(37, 40) 1.0
(19, 22) 1.0
(111, 114) 1.0
(65, 68) 1.0
(106, 109) 1.0
(42, 45) 1.0
(24, 27) 1.0
(60, 63) 1.0
(61, 64) 1.0
(47, 50) 1.0
(29, 32) 1.0
(79, 82) 1.0
(48, 51) 1.0
(98, 101) 1.0
(2, 91) 1.0
(66, 69) 1.0
(11, 14) 1.0
(17, 23) 1.0
(43, 46) 1.0
(12, 15) 1.0
(94, 97) 1.0
(6, 9) 1.0
(0, 3) 1.0
(89, 92) 1.0
(71, 74) 1.0
(7, 10) 1.0
(112, 115) 1.0
(25, 28) 1.0
(99, 102) 1.0
(85, 88) 1.0
(53, 56) 1.0
{(79, 82), (85, 88)}
[(73, 82), (76, 79), (73, 76), (79, 82)]
[(2, 85), (88, 91), (2, 91), (85, 88)]
[[73, 76, 79, 82], [2, 85, 88, 91]]
{(7, 10), (13, 16), (98, 101)}
[(101, 104), (98, 107), (104, 107), (98, 101)]
[(13, 19), (16, 22), (13, 16), (19, 22)]
[[98, 101, 104, 107], [13, 16, 1

TypeError: cannot unpack non-iterable int object

In [15]:
import networkx as nx
from docplex.mp.model import Model

def vertex_list(edges):
    vertices = set()
    for u, v in edges:
        vertices.add(u)
        vertices.add(v)
    return sorted(vertices)

# Assuming graph_new, x, and solution are defined somewhere before

# Step 1: Build comb_graph from solution
comb_graph = nx.Graph()
comb_graph.add_nodes_from(graph_new.nodes)

for e in graph_new.edges:
    val = solution.get_value(x[e])
    if val > 1e-7:
        comb_graph.add_edge(e[0], e[1], weight=val)

# Step 2: Get max weight matching
raw_matching = nx.max_weight_matching(comb_graph, weight='weight')

# Step 3: Normalize and filter matching edges
matching = []
for e in raw_matching:
    edge = (min(e), max(e))
    if edge in graph_new.edges and solution.get_value(x[edge]) == 1:
        matching.append(edge)

# Step 4: Print matching
for e in matching:
    print(e, solution.get_value(x[e]))

# Step 5: Build meta-graph
meta_graph = nx.Graph()
meta_graph.add_nodes_from(matching)

for i in range(len(matching)):
    for j in range(i + 1, len(matching)):
        e1 = matching[i]
        e2 = matching[j]
        nodes_e1 = set(e1)
        nodes_e2 = set(e2)
        connecting_weight = 0
        for u in nodes_e1:
            for v in nodes_e2:
                if graph_new.has_edge(u, v):
                    key = (u, v) if (u, v) in x else (v, u)
                    connecting_weight += solution.get_value(x[key])
        if connecting_weight > 0:
            meta_graph.add_edge(e1, e2, weight=min(connecting_weight,1))

# Step 6: Process weakly connected meta components (comb candidates)
chosen_meta = [(u, v) for u, v, data in meta_graph.edges(data=True) if data['weight'] < 1 - 1e-7]

def is_in_handle_vertices(e, vertex_pairs):
    return any(u in e and v in e for u, v in vertex_pairs)

for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta)):
    handle_edges = []
    teeth_edges = []
    teeth_vertex = []

    internal_edges = [(u, v) for (u, v) in meta_graph.edges() if u in component and v in component]
    cut_edges = [(u, v) for (u, v) in meta_graph.edges() if (u in component) != (v in component)]

    # Handle edges
    for (e1, e2) in internal_edges:
        for u in e1:
            for v in e2:
                if graph_new.has_edge(u, v):
                    handle_edges.append((min(u, v), max(u, v)))
        handle_edges.append(e1)
        handle_edges.append(e2)

    # Teeth edges
    for (e1, e2) in cut_edges:
        teeth = []
        for u in e1:
            for v in e2:
                if graph_new.has_edge(u, v):
                    teeth.append((min(u, v), max(u, v)))
        teeth.append(e1)
        teeth.append(e2)
        teeth_edges.append(teeth)
        teeth_vertex.append(vertex_list(teeth))
        print("Teeth edges:", teeth)

    print("Teeth vertex sets:", teeth_vertex)

    if len(cut_edges) % 2 == 1 and len(cut_edges) >= 3:
        # Comb constraint
        teeth_sum = 0
        for teeth in teeth_vertex:
            teeth_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in teeth and v in teeth]
            teeth_sum += m.sum(x[e] for e in teeth_edges_in_graph if e in x)

        handle_vertices_set = vertex_list(handle_edges)
        handle_edges_in_graph = [(u, v) for (u, v) in graph_new.edges if u in handle_vertices_set and v in handle_vertices_set]
        handle_sum = m.sum(x[e] for e in handle_edges_in_graph if e in x)
        rhs = len(handle_vertices_set) + sum(len(tv) for tv in teeth_vertex) - 0.5 * (3 * len(teeth_vertex) + 1)
        m.add_constraint(handle_sum + teeth_sum <= rhs)
solution=m.solve(log_output=True)       
print(solution.get_objective_value())

(1, 69) 1.0
(43, 52) 1.0
(55, 61) 1.0
(26, 29) 1.0
(67, 70) 1.0
(33, 39) 1.0
(3, 76) 1.0
(63, 66) 1.0
(54, 57) 1.0
(86, 89) 1.0
(8, 11) 1.0
(2, 5) 1.0
(82, 85) 1.0
(91, 92) 1.0
(7, 16) 1.0
(68, 71) 1.0
(50, 53) 1.0
(22, 25) 1.0
(78, 81) 1.0
(32, 35) 1.0
(18, 21) 1.0
(34, 40) 1.0
(14, 17) 1.0
(46, 49) 1.0
(30, 36) 1.0
(42, 45) 1.0
(24, 27) 1.0
(10, 13) 1.0
(0, 73) 1.0
(56, 59) 1.0
(38, 41) 1.0
(48, 51) 1.0
(84, 87) 1.0
(58, 75) 1.0
(4, 74) 1.0
(20, 23) 1.0
(6, 9) 1.0
(19, 28) 1.0
(80, 83) 1.0
(62, 65) 1.0
(31, 37) 1.0
(90, 93) 1.0
(79, 88) 1.0
(12, 15) 1.0
(64, 72) 1.0
(60, 77) 1.0
(44, 47) 1.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
1256408.0


In [1363]:
chosen_meta=[(u,v) for u, v, data in meta_graph.edges(data=True) if data['weight']<1-1e-7]
def is_in_handle_vertices(e, vertex_pairs):
    return any(u in e and v in e for u, v in vertex_pairs)
for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta) ):
    handle_vertices=[]
    handle_edges = []
    teeth_edges =[]
    teeth_vertex= []
    for e in component:
        handle_vertices.append(e)
        
    
    internal_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v in component)]
    cut_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v not in component) or (u not in component and v in component)]
    
    for (e1,e2) in internal_edge:
        for u in e1:
            for v in e2:
                if graph_new.has_edge(u, v):
                    handle_edges.append((min(u, v),max(u,v)))
    
    for (e1,e2) in cut_edge:
        for u in e1:
            for v in e2:
                if graph_new.has_edge(u, v):
                    teeth_edges.append((min(u, v),max(u,v)))
    
    for e in cut_edge:
        if e[0] not in component:
            teeth_vertex.append(e[0])
        if e[1] not in component:
            teeth_vertex.append(e[1])
    if len(component)==3 and len(teeth_edges)==6 and len(handle_edges)==6 and len(teeth_vertex)==3:
        all_edges = handle_edges + teeth_edges+teeth_vertex
        m.add_constraint(m.sum(2*x[e] for e in handle_vertices)+m.sum(x[e] for e in all_edges) <= 13 )
solution=m.solve(log_output=True)
print(solution.get_objective_value())

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
1651981.666666667


In [670]:
comb_graph=nx.Graph()
comb_graph.add_nodes_from(graph_new.nodes)
for e in graph_new.edges:
    if solution.get_value(x[e])>1e-7:
       comb_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
fractional_edge=[e for e in comb_graph.edges() if 1/2+1e-7<solution.get_value(x[e])]
for component in nx.connected_components(comb_graph.edge_subgraph(fractional_edge) ):
            G_comp=nx.Graph()
            G_comp.add_nodes_from(component)
            internal_edge=[e for e in comb_graph.edges if e[0] in component and e[1] in component]
            boundary_edge=[e for e in graph_new.edges if e[0] in component and e[1] in component]
            G_comp.add_edges_from(internal_edge)
            matching=[]
            for e in nx.max_weight_matching(G_comp,weight='weight'):
                if (min(e),max(e)) in internal_edge:
                            matching.append((min(e),max(e)))
            if nx.is_perfect_matching(G_comp, matching)==True and len(component)>2:
                print(matching)
                m.add_constraint(m.sum(2*x[e] for e in matching)+m.sum(x[e] for e in boundary_edge if e not in matching)<= int(0.5*(3*len(component)-2) ))
                     #vertex_not_in_component=[]
                     #for e in cut_edges:
                     #    if e[0] not in component:
                     #        vertex_not_in_component.append(e[0])
                     #    if e[1] not in component:
                     #        vertex_not_in_component.append(e[1])
                     #edge_not_in=[e for e in graph_new.edges if e[0] in vertex_not_in_component and e[1] in vertex_not_in_component]
                #    print(sum(solution.get_value(x[e]) for e in boundary_edge))
                #    print(sum(solution.get_value(x[e]) for e in cut_edges))
                #    m.add_constraint(m.sum(x[e] for e in boundary_edge)+m.sum(x[e] for e in cut_edges)<= len(component)+int(0.5*len(cut_edges))-1 )
        
                     #print(sum(solution.get_value(x[e]) for e in boundary_edge))
                     #print(sum(solution.get_value(x[e]) for e in cut_edges))
                     #m.add_constraint(m.sum(x[e] for e in boundary_edge)+m.sum(x[e] for e in cut_edges)<= len(component)+int(0.5*len(cut_edges)) )
            

solution=m.solve(log_output=True)
print(solution.get_objective_value())

[(37, 43), (1, 69), (58, 61), (26, 29), (63, 66), (86, 89), (8, 75), (82, 85), (56, 62), (50, 53), (92, 93), (68, 73), (5, 72), (22, 25), (32, 35), (64, 67), (7, 77), (28, 31), (34, 40), (14, 17), (46, 49), (88, 91), (0, 71), (10, 13), (52, 55), (38, 41), (76, 79), (11, 78), (57, 60), (4, 74), (20, 23), (59, 65), (80, 83), (16, 19), (2, 70), (3, 6), (44, 47)]
[(9, 18), (12, 15)]
[(21, 30), (24, 27)]
[(33, 42), (36, 39)]
[(48, 51), (45, 54)]
[(81, 90), (84, 87)]
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
1271196.25


In [345]:
from typing import Dict, List, Optional, Union
from qiskit_optimization.applications import Maxcut, Tsp
import networkx as nx
import numpy as np
from docplex.mp.model import Model
from qiskit_algorithms.utils import algorithm_globals

from qiskit_optimization.algorithms import OptimizationResult
from qiskit_optimization.exceptions import QiskitOptimizationError
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
import random

def random_consecutive_slice(lst, count):
    if len(lst) < count:
        raise ValueError("List is too short for the requested number of consecutive elements.")
    start = random.randint(0, len(lst) - count)
    return lst[start:start + count]
def subprob_graph(index_quantum,mat):
    G_quantum=nx.Graph()
    for i in index_quantum:
        G_quantum.add_node(index_quantum.index(i))
    for i in index_quantum:
      for j in index_quantum:
        if i!=j:
          G_quantum.add_edge(index_quantum.index(i),index_quantum.index(j),weight=1)
    return G_quantum

def subproblem(G1):
        mdl = Model(name="TSP_path")
        n = len(G1.nodes())
        x = {(i, k): mdl.binary_var(name=f"x_{i}_{k}") for i in range(n) for k in range(n)}
        tsp_func = mdl.sum(
            G1.edges[i, j]["weight"] * x[(i, k)] * x[(j, (k + 1) % n)]
            for i in range(n)
            for j in range(n)
            for k in range(n-1)
            if i != j
        )
        mdl.minimize(tsp_func)
        for i in range(n):
            mdl.add_constraint(mdl.sum(x[(i, k)] for k in range(n)) == 1)
        for k in range(n):
            mdl.add_constraint(mdl.sum(x[(i, k)] for i in range(n)) == 1)
        op = from_docplex_mp(mdl)
        return op
def create_black_box_circ(G, theta):


    N=len(G.nodes())
    edge=G.edges()
    nodes=G.nodes()
    nqubits = len(G.nodes())
    p = len(theta)//2
    qc = QuantumCircuit(nqubits)

    beta = theta[:p]
    gamma = theta[p:]


    #for i in range(0, nqubits):
    #    qc.h(i)


    for irep in range(0, p):


        for i,j in edge:
            if (i!=j):
                qc.ryy(2*gamma[irep], i, j)
            else:
                qc.ry(2 * beta[irep], i)

    qc.measure_all()
    return qc

def circuit_graph(G1):
        n = len(G1.nodes())
        qp2qubo = QuadraticProgramToQubo()
        qubo=qp2qubo.convert(subproblem(G1))
        
        a=qubo.objective.quadratic.to_dict()
        b=qubo.objective.linear.to_dict()
        Q_dict={}
        n=len(index_quantum)
        for i in range(n**2):
          if i in b.keys():
             Q_dict[(i,i)]=b[i]+a[(i,i)]
          for j in range(n**2):
            if (i<j and (i,j) in (a.keys()) ):
               Q_dict[(i,j)]=a[(i,j)]     
        h, J, offset = dimod.qubo_to_ising(Q_dict)
        G=nx.complete_graph(n**2)
        nodes=G.nodes()
        N=len(nodes)
        for i in nodes:
            G.add_edge(i,i)
            for (i, j) in G.edges():
              if i==j:
                if i in h.keys():
                   G.edges[i,i]['weight'] = h[i]
                else:
                   G.edges[i,i]['weight'] = 0
              else:
                if (i,j) in Q_dict.keys():
                  G.edges[i,j]['weight'] = J[(i,j)]
                else:
                  G.edges[i,j]['weight'] = 0
        return G
def interpret(x):
    n = int(np.sqrt(len(x)))
    route = []  # type: List[Union[int, List[int]]]
    for p__ in range(n):
        p_step = []
        for i in range(n):
            if x[i * n + p__]:
                    p_step.append(i)
        if len(p_step) == 1:
                route.extend(p_step)
        else:
                route.append(p_step)
    return route
def create_new_sol(bitstring, merge_order, index_quantum):
    list_of_ints = [int(i) for i in bitstring]
    x1 = np.array(list_of_ints)
    TSP_array = interpret(x1) 
    reordered_quantum = [index_quantum[i] for i in TSP_array if not isinstance(i, list)]

    new_merge_order = merge_order.copy()

    target_indices = [i for i, val in enumerate(merge_order) if val in index_quantum]

    for i, new_val in zip(target_indices, reordered_quantum):
        new_merge_order[i] = new_val

    return new_merge_order
def feasible_check_1(bitstring, merge_order, index_quantum):
    list_of_ints = [int(i) for i in bitstring]
    x1 = np.array(list_of_ints)
    TSP_array = interpret(x1)
    num_list=0
    for i in TSP_array:
        if isinstance(i, list):
            num_list+=1
    if num_list>1e-7:
        return False
    else: 
        return True
            
def merge_hamiltonian_paths_by_order(ham_path, U, V, merge_order):
    if not ham_path:
        raise ValueError("ham_path is empty.")

    if not merge_order:
        raise ValueError("merge_order is empty.")

    if any(i >= len(ham_path) or i < 0 for i in merge_order):
        raise IndexError(f"merge_order contains invalid indices. "f"Valid range: 0 to {len(ham_path)-1}, got: {merge_order}")
    ordered_paths = [ham_path[i] for i in merge_order]
    cycle = ordered_paths[0][:]
    current_end = cycle[-1]
    for path in ordered_paths[1:]:
        start, end = path[0], path[-1]
        if (current_end in U and start in U) or (current_end in V and start in V):
            path = path[::-1]
            start, end = end, start
        cycle.extend(path)
        current_end = end
    first_start = cycle[0]
    if (current_end in U and first_start in V) or (current_end in V and first_start in U):
        cycle.append(first_start)

    return cycle   
    
    
def feasible_check(order):
    return len(order) == len(set(order))
def cost_func_tsp(ham_path,x,merge_order,index_quantum):
    order=create_new_sol(x, merge_order, index_quantum)
    list_of_ints = [int(i) for i in x]
    x1 = np.array(list_of_ints)
    TSP_array = interpret(x1)
    if feasible_check(order)== True and feasible_check_1(x, merge_order, index_quantum)==True and len(TSP_array) == len(set(TSP_array)):
        merge_path=merge_hamiltonian_paths_by_order(ham_path, bottom_nodes, top_nodes, order)
        #init_tour= two_opt(merge_path,mat)
        cost=total_distance(merge_path,mat)
    else:
        cost=10000000000
    return cost 
def cost_func_tsp_path(x,merge_order,index_quantum):
    order=create_new_sol(x, merge_order, index_quantum)
    list_of_ints = [int(i) for i in x]
    x1 = np.array(list_of_ints)
    TSP_array = interpret(x1)
    if feasible_check(order)== True and feasible_check_1(x, merge_order, index_quantum)==True and len(TSP_array) == len(set(TSP_array)):
        cost=total_distance(create_new_sol(x, merge_order, index_quantum),mat)
    else:
        cost=10000000000
    return cost 
def quantum_2_opt(ham_path,merge_order,count,index_quantum):
    feasible_sol = [create_new_sol(x[::-1], merge_order, index_quantum) for x in count.keys() if feasible_check(create_new_sol(x[::-1], merge_order, index_quantum))== True 
                    and feasible_check_1(x[::-1], merge_order, index_quantum)==True 
                    and len(interpret(np.array([int(i) for i in x[::-1]]))) == len(set(interpret(np.array([int(i) for i in x[::-1]]))))]
    if len(feasible_sol)>1e-7:
        avg=0
        item_sum=0
        for x, counts in count.items():
            if feasible_check(create_new_sol(x[::-1], merge_order, index_quantum))== True and feasible_check_1(x[::-1], merge_order, index_quantum)==True and len(interpret(np.array([int(i) for i in x[::-1]]))) == len(set(interpret(np.array([int(i) for i in x[::-1]])))):
                item_sum+=counts
            
        for x, counts in count.items():
            if feasible_check(create_new_sol(x[::-1], merge_order, index_quantum))== True and feasible_check_1(x[::-1], merge_order, index_quantum)==True and len(interpret(np.array([int(i) for i in x[::-1]]))) == len(set(interpret(np.array([int(i) for i in x[::-1]])))):
                sol=create_new_sol(x[::-1], merge_order, index_quantum)
                init_tour=two_opt(merge_hamiltonian_paths_by_order(ham_path, bottom_nodes, top_nodes, sol),mat)
                avg+=total_distance(init_tour,mat)*counts/item_sum
        return avg
    else:
        init_tour=two_opt(merge_hamiltonian_paths_by_order(ham_path, bottom_nodes, top_nodes, merge_order),mat)
        return total_distance(init_tour,mat)
    
def compute_cvar(ham_path,merge_order,count,index_quantum, alpha):
    probability1=[]
    for i in count.values():
      probs=i/sum(count.values())
      probability1.append(probs)
    value1=[]
    for x in count.keys():
      value1.append(cost_func_tsp(ham_path,x[::-1],merge_order,index_quantum))
     
    sorted_indices1 = np.argsort(value1)
    probs = np.array(probability1)[sorted_indices1]
    vals = np.array(value1)[sorted_indices1]
    cvar = 0
    total_prob = 0
    for i, (p, v) in enumerate(zip(probs, vals)):
        done = False
        if p >= alpha - total_prob:
            p = alpha - total_prob
            done = True
        total_prob += p
        cvar += p * v
    cvar /= total_prob
    return cvar
def get_expectation_cvar(G, ham_path,merge_order,index_quantum,alpha):
    backend = Aer.get_backend('qasm_simulator')
    backend.shots = 512

    def execute_circ(theta):

        qc = create_black_box_circ(G, theta)
        counts = backend.run(qc, seed_simulator=10,nshots=512).result().get_counts()

        return compute_cvar(ham_path,merge_order,counts, index_quantum, alpha)

    return execute_circ

def get_expectation_2opt(G, ham_path,merge_order,index_quantum):
    backend = Aer.get_backend('qasm_simulator')
    backend.shots = 512

    def execute_circ(theta):

        qc = create_black_box_circ(G, theta)
        counts = backend.run(qc, seed_simulator=10,nshots=512).result().get_counts()

        return quantum_2_opt(ham_path,merge_order,count,index_quantum)

    return execute_circ

In [365]:
import qiskit
from qiskit_aer import Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit_algorithms import SamplingVQE, NumPyMinimumEigensolver,QAOA
from qiskit_algorithms.optimizers import SPSA, COBYLA
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit.library import RealAmplitudes
from scipy.optimize import differential_evolution, dual_annealing
import dimod
from scipy.optimize import minimize
def local_search(ham_path,merge_order,index_quantum,mat):
    bounds = [(0, 1) for _ in range(2)]
        
    expectation = get_expectation_cvar(subprob_graph(index_quantum,mat), ham_path,merge_order,index_quantum,0.001)
    res1 = differential_evolution(expectation, bounds, maxiter=20)
    backend = Aer.get_backend('qasm_simulator')
    qc_res1 = create_black_box_circ(
            circuit_graph(subprob_graph(index_quantum, mat)),
            res1.x
        )
        
    count = backend.run(qc_res1, seed_simulator=10).result().get_counts()
    sum_prob = sum(count.values())
    probability1 = [i / sum_prob for i in count.values()]
    feasible_sol = [create_new_sol(x[::-1], merge_order, index_quantum) for x in count.keys() if feasible_check(create_new_sol(x[::-1], merge_order, index_quantum))== True 
                    and feasible_check_1(x[::-1], merge_order, index_quantum)==True 
                    and len(interpret(np.array([int(i) for i in x[::-1]]))) == len(set(interpret(np.array([int(i) for i in x[::-1]]))))]
    if len(feasible_sol)>1e-7:
        for sol in feasible_sol:
            init_tour_quantum=two_opt(merge_hamiltonian_paths_by_order(ham_path, bottom_nodes, top_nodes, sol),mat)
            print('initial tour:', init_tour_quantum)
            print('length of initial tour:',total_distance(init_tour_quantum,mat))
        
    
    

In [367]:
merge_order=[i for i in range(len(ham_path))]
for i in range(20):
    print('Sampling')
    index_quantum=random.sample(merge_order, 2)
    merge_new=local_search(ham_path,merge_order,index_quantum,mat)
    #print("Quantum construct tour length:",total_distance(merge_hamiltonian_paths_by_order(ham_path, bottom_nodes, top_nodes, merge_new),mat))
    #init_tour_1=merge_hamiltonian_paths_by_order(ham_path, bottom_nodes, top_nodes, merge_new)
    #new_tour=two_opt(init_tour_1,mat)
    #print("2 opt length:", total_distance(new_tour,mat))
    #merge_order=merge_new

Sampling
initial tour: [86, 89, 92, 95, 98, 99, 97, 94, 91, 88, 85, 82, 65, 62, 68, 71, 74, 77, 0, 79, 6, 3, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 48, 45, 51, 54, 57, 60, 63, 83, 66, 69, 72, 75, 1, 80, 4, 7, 10, 13, 16, 19, 22, 25, 34, 46, 52, 55, 58, 61, 64, 67, 70, 73, 76, 2, 5, 78, 81, 8, 11, 14, 17, 20, 23, 26, 32, 29, 35, 59, 56, 53, 50, 47, 44, 41, 38, 96, 93, 90, 87, 84, 49, 43, 40, 37, 31, 28, 86]
length of initial tour: 1543968.0
initial tour: [23, 20, 35, 62, 68, 71, 74, 77, 0, 79, 6, 3, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 48, 45, 51, 54, 57, 60, 63, 83, 66, 69, 72, 75, 1, 80, 4, 7, 10, 13, 16, 19, 22, 25, 31, 28, 34, 37, 40, 43, 49, 46, 52, 55, 58, 61, 64, 67, 70, 73, 76, 2, 5, 78, 81, 8, 11, 14, 17, 84, 87, 90, 93, 96, 86, 89, 92, 95, 98, 99, 97, 94, 91, 88, 85, 82, 65, 59, 56, 53, 50, 47, 44, 41, 38, 29, 32, 26, 23]
length of initial tour: 1542693.0
Sampling
initial tour: [23, 20, 35, 62, 68, 71, 74, 77, 0, 79, 6, 3, 9, 12, 15, 18, 21, 24, 27, 30, 33, 3

In [357]:
def spanning_tree_sep(self):
    Graph_NN=nx.Graph()
    Graph_NN.add_nodes_from(graph_new.nodes())
    mat_nn=[[0 for i in range(n)] for j in range(n)]
    for e in graph_new.edges():
        if self.get_values(x[e].name)>1e-7:
            Graph_NN.add_edge(e[0],e[1],weight=self.get_values(x[e].name))
    tree_1=tree.maximum_spanning_edges(Graph_NN, weight='weight', algorithm='kruskal', data=False,ignore_nan=False)
    #print(tree.edges(data=True))
    edgelist=[]
    component_cons=[]
    for e in tree_1:
        edgelist.append(e)
    numb_comp=0
    for component in nx.connected_components( graph_new.edge_subgraph(edgelist) ):
        numb_comp+=1
    #chosen_edges = [ e for e in graph_new.edges if solution.get_value(x[e]) > 0.5+1e-7 ]
    #for component in match_sep(solution):#nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
    if numb_comp>=1+1e-7:
        
        for component in nx.connected_components( graph_new.edge_subgraph(edgelist) ):
            
            if len(component) < n and len(component)%2==0:
                component_cons.append(component)
               
    else:
        chosen_edges = [ e for e in edgelist if self.get_values(x[e].name) > min([self.get_values(x[e].name) for e in edgelist]) ]
        for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
            if len(component) < n and len(component)%2==0:
                component_cons.append(component)
    return component_cons

In [139]:
def nodes2edge(u, v):
    return (min(u, v), max(u, v))

def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons

def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
class SubtourUserCallback_heu(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           edges_sub,component_sub= separate_heuristic(sol)
           for i in range(len(component_sub)):
               tour=0
               for e in edges_sub[i]:
                   tour+=x[e]
               ct=tour<=len(component_sub[i])-1
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
           
          
from docplex.mp.callbacks.cb_mixin import *
import numpy as np
def add_violated_cuts(callback, solution):
    fracSubtours, distances = getFracSubtourCons(solution, callback.x, callback.nodes)
    for subtour in fracSubtours:
        expr = []
        for edge in subtour:
            expr.append("x.{}.{}".format(*edge))
        thecoefs = [1] * len(subtour)
        callback.add(cplex.SparsePair(expr, thecoefs), sense="G", rhs=2)
    return fracSubtours, len((fracSubtours)), max(distances) if len(distances) > 0 else 0
class SubtourUserCallback(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           subtourCons= getFracSubtourCons(sol)
           for subtour in subtourCons:
               tour=0
               for e in subtour:
                   tour+=x[e]
               ct=tour>=2
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
                   
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v>1e-7): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)



from collections import defaultdict, namedtuple
import math
import random

def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph
        
def find_max_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = max(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree == 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))


def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_branch = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = max(edge[2] for edge in fractional_edges)
            #min_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            #min_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == min_weight]
            for e in max_weight_edges:
                fractional_edges_branch.append(e)
            #for e in min_weight_edges:
            #    fractional_edges_branch.append(e)

    return fractional_edges_branch
    spanning_tree_sep

class SubtourUserCallback_tree(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           for component in spanning_tree_sep(self):
               boundary_edge=[e for e in graph_new.edges if e[0] in component and e[1] in component]
               tour=0
               for e in boundary_edge:
                   tour+=x[e]
               ct=tour<=len(component)-1
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         for k in find_max_min_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0: 
              return 
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         #elif(gap>0.03+1e-7 and gap<0.10-1e-7):
         #   for k in find_min_weight_fractional_edges(self):
         #       if feas[x[k].index] == infeas:
         #           xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
         #           if xj_inf > 0.5:
         #               xj_inf = 1.0 - xj_inf
                        
         #           if (xj_inf <= maxinf): 
         #               bestj = x[k].index
         #               maxinf = xj_inf
         #               maxobj = abs(obj[x[k].index])
         #   if bestj < 0:return
         #   xj_lo = math.floor(x_val[bestj])
                                
         #   dv = self.index_to_var(bestj)
         #   self.stats[dv] += 1
                                
         #   self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         #   self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         
def new_branch(self):
    comb_graph = branching_graph(self)
    x_bin_max = []
    x_bin_other = []
    frac_edge_branch = []
    min_branch = []

    chosen_edges_x_bin = [e for e in comb_graph.edges  if 1 - 1e-7 < self.get_values(x[e].name)]
    components = list(nx.connected_components(comb_graph.edge_subgraph(chosen_edges_x_bin)))
    largest_cc = max(components, key=len) if components else set()
    #for component in nx.connected_components(G.edge_subgraph(chosen_edges_x_bin_y_frac)):
    for i in largest_cc:
            x_bin_max.append(i)
    for component in components:
        if component!=largest_cc:
            for i in component:
                x_bin_other.append(i)
                
            
    for e in comb_graph.edges:
        if (1e-7 < self.get_values(x[e].name) < 1 - 1e-7 and ((e[0] in x_bin_max and e[1] in x_bin_other) or(e[1] in x_bin_max and e[0] in x_bin_other))):
            frac_edge_branch.append(e)
    min_edge=[]
    for e in frac_edge_branch:
        if c[e[0]][e[1]]==min([c[e[0]][e[1]] for e in frac_edge_branch]):
            min_edge.append(e)
        
    return min_edge          
        
class MyBranch2(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         if len(find_fractional_edges_with_max_degree_and_edge_value_1(self))>1e-7:
             for k in find_fractional_edges_with_max_degree_and_edge_value_1(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         elif len(find_fractional_edges_with_max_deg(self))>1e-7:
             for k in find_fractional_edges_with_max_deg(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN")) 
         
        

from docplex.mp.model import Model
#from docplex.mp.model import Priority
m=Model()
x = {(i,j):m.binary_var(name="x_{0}_{1}".format(i,j)) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
#m.parameters.mip.strategy.search.set(1)
#m.parameters.mip.interval.set(1)
m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch2)
m.branch_callback=branch_cb
#m.parameters.mip.tolerances.mipgap(0.3)
#m.register_callback(CustomHeuristicCallback)
cut_cb = m.register_callback(SubtourUserCallback)
m.cut_callback = cut_cb
values = {}
values.update({x[e]: 1 for e in edge_from_tour([71, 68, 65, 62, 59, 56, 50, 53, 47, 44, 41, 38, 35, 29, 32, 26, 23, 20, 17, 14, 11, 8, 5, 81, 2, 79, 76, 73, 70, 84, 87, 67, 64, 61, 58, 55, 52, 90, 93, 96, 99, 102, 105, 103, 100, 104, 101, 98, 46, 49, 43, 40, 37, 34, 31, 28, 25, 22, 19, 16, 13, 10, 86, 7, 4, 83, 1, 78, 75, 72, 69, 66, 89, 92, 95, 63, 60, 57, 54, 51, 48, 45, 42, 39, 36, 33, 30, 27, 24, 21, 18, 15, 12, 9, 6, 3, 0, 80, 77, 74, 82, 85, 88, 91, 94, 97, 71])})
warmstart=m.new_solution()
warmstart.update( values )
m.add_mip_start(warmstart)
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution=m.solve(log_output=True) 

KeyError: (50, 56)

In [255]:
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
print(solution.get_objective_value())
constr_count=0
sep_time=0
iteration=0
for component in match_sep(solution):#nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
    if len(component) < n and len(component)%2==0:
           boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
           if sum(solution.get_value(x[e]) for e in boundary_edges)>len(component)-1+1e-7:
              m.add_constraint(m.sum(x[e] for e in boundary_edges)<= len(component)-1 )
              constr_count+=1
solution=m.solve(log_output=True)
print(solution.get_objective_value())
Graph_NN=nx.Graph()
Graph_NN.add_nodes_from(graph_new.nodes())
mat_nn=[[0 for i in range(n)] for j in range(n)]
for e in graph_new.edges():
    if solution.get_value(x[e])>1e-7:
        Graph_NN.add_edge(e[0],e[1],weight=-solution.get_value(x[e]))
tree=nx.minimum_spanning_tree(Graph_NN, weight='weight', algorithm='kruskal', ignore_nan=False)
print(tree.edges(data=True))
for component in nx.connected_components( graph_new.edge_subgraph(tree.edges) ):
    if len(component) < n and len(component)%2==0:
           print(component)
           boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
           if sum(solution.get_value(x[e]) for e in boundary_edges)>len(component)-1+1e-7:
              m.add_constraint(m.sum(x[e] for e in boundary_edges)<= len(component)-1 )
              constr_count+=1
solution=m.solve(log_output=True)
print(solution.get_objective_value())


1572396.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1572882.000000
1575720.0
[(0, 3, {'weight': -1.0}), (0, 80, {'weight': -1.0}), (1, 75, {'weight': -1.0}), (1, 78, {'weight': -1.0}), (2, 79, {'weight': -1.0}), (2, 81, {'weight': -1.0}), (3, 6, {'weight': -1.0}), (4, 83, {'weight': -1.0}), (4, 86, {'weight': -1.0}), (5, 8, {'weight': -1.0}), (5, 81, {'weight': -1.0}), (6, 9, {'weight': -1.0}), (7, 10, {'weight': -1.0}), (7, 86, {'weight': -1.0}), (8, 14, {'weight': -1.0}), (9, 12, {'weight': -1.0}), (10, 13, {'weight': -1.0}), (11, 14, {'weight': -1.0}), (11, 17, {'weight': -1.0}), (12, 15, {'weight': -1.0}), (13, 16, {'weight': -1.0}), (15, 18, {'weight': -1.0}), (16, 89, {'weight': -1.0}), (17, 20, {'weight': -1.0}), (18, 21, {'weight': -1.0}), (19, 22, {

In [269]:
from cplex.callbacks import UserCutCallback, LazyConstraintCallback, BranchCallback
from docplex.mp.callbacks.cb_mixin import *
def nodes2edge(u, v):
    return (min(u, v), max(u, v))
def removed_graph(G,edge_set):
    for e in edge_set:
        G.remove_edge(e[0],e[1])
    return G
        
def match_sep(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                Matching=nx.max_weight_matching(sep_graph,weight='weight')
                for e in Matching:
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets     
def match_sep_1(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                matching_1=[]
                M=nx.max_weight_matching(sep_graph,weight='weight')
                for e in M:
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                M1=nx.max_weight_matching(removed_graph(G_comp,matching),weight='weight')
                for e in nx.max_weight_matching(removed_graph(G_comp,matching),weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching_1.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True or nx.is_perfect_matching(G_comp, matching_1)==True  and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons

from docplex.mp.callbacks.cb_mixin import *
import numpy as np
def add_violated_cuts(callback, solution):
    fracSubtours, distances = getFracSubtourCons(solution, callback.x, callback.nodes)
    for subtour in fracSubtours:
        expr = []
        for edge in subtour:
            expr.append("x.{}.{}".format(*edge))
        thecoefs = [1] * len(subtour)
        callback.add(cplex.SparsePair(expr, thecoefs), sense="G", rhs=2)
    return fracSubtours, len((fracSubtours)), max(distances) if len(distances) > 0 else 0
class SubtourUserCallback(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           subtourCons= getFracSubtourCons(sol)
           for subtour in subtourCons:
               tour=0
               for e in subtour:
                   tour+=x[e]
               ct=tour>=2
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v>1e-7): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)



from collections import defaultdict, namedtuple
import math
import random
def min_matching(self):
    branching_graph=nx.Graph()
    branching_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
        if 1-1e-7>self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
    matching=nx.min_weight_matching(branching_graph,weight='weight')
    branching_edge=[]
    for e in matching:
        branching_edge.append((min(e),max(e)))
    return branching_edge
def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph

def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub

class SubtourUserCallback_sep(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           if len(match_sep(sol))>1+1e-7:
               for component in match_sep(sol):
                   tour=0
                   edges=[e for e in graph_new.edges if e[0] in component and e[1] in component]
                   for e in edges:
                       tour+=x[e]
                   ct=tour<=len(component)-1
                   unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                   for ct, cpx_lhs, sense, cpx_rhs in unsats:
                        self.add(cpx_lhs, sense, cpx_rhs)

class SubtourUserCallback_heu(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           edges_sub,component_sub= separate_heuristic(sol)
           for i in range(len(component_sub)):
               tour=0
               for e in edges_sub[i]:
                   tour+=x[e]
               ct=tour<=len(component_sub[i])-1
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
           comb_graph=nx.Graph()
           comb_graph.add_nodes_from(graph_new.nodes)
        
           for e in graph_new.edges:
              if sol.get_value(x[e])>1e-7:
                  comb_graph.add_edge(e[0],e[1])
           fractional_edge=[e for e in comb_graph.edges() if 1-1e-7>sol.get_value(x[e])>1e-7]
           for component in nx.connected_components(comb_graph.edge_subgraph(fractional_edge) ):
               number_of_u=0
               number_of_v=0
               for i in component:
                   if i in bottom_nodes:
                      number_of_u+=1       
                   elif i in top_nodes:
                      number_of_v+=1
               if number_of_u == number_of_v:       
                        boundary_edge=[e for e in graph_new.edges if e[0] in component and e[1] in component]
                        cut_edges=[e for e in comb_graph.edges if ((e[0] in component and e[1] not in component) or (e[0] not in component and e[1] in component)) and sol.get_value(x[e])>1e-7]
                        if len(cut_edges)%4==2 and len(cut_edges)>1e-7 and similar_edges(cut_edges)==True:
                            cut_edges_u=[e for e in graph_new.edges if ((e[0] in component and e[1] not in component and e[0] in bottom_nodes) or (e[0] not in component and e[1] in component and e[1] in bottom_nodes)) and sol.get_value(x[e])>1e-7]
                            cut_edges_v=[e for e in graph_new.edges if ((e[0] in component and e[1] not in component and e[0] in top_nodes) or (e[0] not in component and e[1] in component and e[1] in top_nodes)) and sol.get_value(x[e])>1e-7]
                            if len(cut_edges_u)==len(cut_edges_v):
                                tour_1=0
                                for e in boundary_edge:
                                    tour_1+=x[e]
                                for e1 in cut_edges:
                                    tour_1+=x[e1]
                                    
                                ct=tour_1<=len(component)+0.5*len(cut_edges)-1
                                unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                                for ct, cpx_lhs, sense, cpx_rhs in unsats:
                                        self.add(cpx_lhs, sense, cpx_rhs)
                            
def find_max_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = max(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))


def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_branch = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = max(edge[2] for edge in fractional_edges)
            #min_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            #min_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == min_weight]
            for e in max_weight_edges:
                fractional_edges_branch.append(e)
            #for e in min_weight_edges:
            #    fractional_edges_branch.append(e)

    return fractional_edges_branch
    
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         if len(set(find_min_weight_fractional_edges(self)))!=0:
             
            for k in find_min_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
            if bestj < 0:
                return
            xj_lo = math.floor(x_val[bestj])
                                
            dv = self.index_to_var(bestj)
            self.stats[dv] += 1
                                
            self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
            self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         
             
        
  
         
        

from docplex.mp.model import Model
#from docplex.mp.model import Priority
m=Model()
x = {(i,j):m.binary_var(name="x_{0}_{1}".format(i,j)) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
#m.parameters.mip.strategy.search.set(1)
#m.parameters.mip.interval.set(1)
m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
#branch_cb=m.register_callback(MyBranch2)
#m.branch_callback=branch_cb
m.parameters.mip.tolerances.mipgap(1e-7)
cut_cb = m.register_callback(SubtourUserCallback_heu)
m.cut_callback = cut_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
values = {}
values.update({x[e]: 1 for e in edge_from_tour(init_tour)})
warmstart=m.new_solution()
warmstart.update( values )
m.add_mip_start(warmstart)
solution=m.solve(log_output=True) 

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Tolerances_MIPGap                   9.9999999999999995e-08
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  LUD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Processing 1 MIP starts.
MIP start 'm1' defined solution with objective 1415532.0000.
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 1415532.0000.
Dual steepest-edge pricing selected.
MIP emphasis: balan

In [9]:
from cplex.callbacks import UserCutCallback, LazyConstraintCallback, BranchCallback
from docplex.mp.callbacks.cb_mixin import *
from collections import defaultdict, namedtuple
import cplex.callbacks as cpx_cb
import cplex._internal._constants as cpxcst
import math
import time
import random
def match_sep(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
def removed_graph(G,edge_set):
    for e in edge_set:
        G.remove_edge(e[0],e[1])
    return G
        
def construct_hamiltonian_tour(cplex_solution, nodes):
    adjacency = {node: [] for node in nodes}
    for i, j in cplex_solution:
        adjacency[i].append(j)
        adjacency[j].append(i)
    
    # Start with an arbitrary node
    start_node = nodes[0]
    current_node = start_node
    tour = [current_node]
    visited = set([current_node])
    
    while len(visited) < len(nodes):
        # Visit the next unvisited node
        for neighbor in adjacency[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                tour.append(neighbor)
                current_node = neighbor
                break
    
    # Close the tour
    tour.append(start_node)
    return tour
def k_factor(solution):
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    #k_factor(sep_graph, 2, matching_weight='weight')
    print(k_factor(sep_graph, 2, matching_weight='weight'))
    
def match_sep_1(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                matching_1=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                for e in nx.max_weight_matching(removed_graph(G_comp,matching),weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching_1.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True or nx.is_perfect_matching(G_comp, matching_1)==True  and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
print(solution.get_objective_value())
constr_count=0
sep_time=0
iteration=0
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    #chosen_edges = [ e for e in graph_new.edges if solution.get_value(x[e]) > 0.5+1e-7 ]
    for component in match_sep(solution):#nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
        if len(component) < n and len(component)%2==0:
           boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
           if sum(solution.get_value(x[e]) for e in boundary_edges)>len(component)-1+1e-7:
              m.add_constraint(m.sum(x[e] for e in boundary_edges)<= len(component)-1 )
              constr_count+=1
    end=time.time()
    sep_time+=end-start
    iteration+=1
    solution=m.solve(log_output=True)
    print(solution.get_objective_value())
print(constr_count) 
print(sep_time)
print(iteration)
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree == 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph
def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_fractional_edges_with_max_degree_and_edge_value_1(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    max_degree = max(dict(graph.degree).values(), default=0)
    max_degree_vertices = [node for node, degree in graph.degree if degree == max_degree]

    if not max_degree_vertices:
        return []

    fractional_edges_max = []

    for vertex in max_degree_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

def find_fractional_edges_with_max_deg(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    vertice_edge_1=[]
    fractional_edges_max = []

    for vertex in graph.nodes():
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition
        vertice_edge_1.append(vertex)
        # Process fractional edges
    max_deg_vertice=[vertex for vertex in vertice_edge_1 if graph.degree(vertex)==max([graph.degree(vertex) for vertex in vertice_edge_1])]
    for vertex in max_deg_vertice:
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class CustomHeuristicCallback(ModelCallbackMixin, cpx_cb.HeuristicCallback):
    def __init__(self, env):
        # Initialize the parent classes
        cpx_cb.HeuristicCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_heuristics = 0

    def __call__(self):
        x_val= self.make_solution_from_vars(x.values())
        self.nb_heuristics += 1
        tour_edges=[]
        for e in graph_new.edges:
            if x_val.get_value(x[e])>1-1e-7:
                tour_edges.append(e)
        
        # Example heuristic to construct a feasible solution
        gap=self.get_MIP_relative_gap()
        nodes = graph_new.nodes()
        if len(tour_edges)==len(nodes):
            initial_tour = construct_hamiltonian_tour(tour_edges,list(nodes))
            obj_value = total_distance(initial_tour, mat)
    
            # Check feasibility of the solution (optional, depends on your logic)
            tour1=two_opt(initial_tour,mat)
            if total_distance(tour1,mat)<total_distance(initial_tour,mat):
                var_edges=edge_from_tour(tour1)
                
                var_index = [x[e] for e in var_edges]
                self.set_solution(var_index, [1] * len(var_index))
        
            else:
                var_edges=edge_from_tour(initial_tour)
                var_index = [x[e] for e in var_edges]
                self.set_solution(var_index, [1] * len(var_index))
            
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         #if len(set(find_min_weight_fractional_edges(self))&set(min_matching(self)))!=0:
             
         #    for k in set(find_max_weight_fractional_edges(self))&set(max_matching(self)):
         #               if feas[x[k].index] == infeas:
         #                   xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
         #                   if xj_inf > 0.5:
         #                       xj_inf = 1.0 - xj_inf
                                
         #                   if (xj_inf <= maxinf): 
         #                       bestj = x[k].index
         #                       maxinf = xj_inf
         #                       maxobj = abs(obj[x[k].index])
         #    if bestj < 0: 
         #             return 
         #    xj_lo = math.floor(x_val[bestj])
                                        
         #    dv = self.index_to_var(bestj)
         #    self.stats[dv] += 1
                                        
         #    self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         #    self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         #elif(gap>0.03+1e-7 and gap<0.10-1e-7):
         #else:
         if len(find_max_min_weight_fractional_edges(self))!=0:
             for k in find_max_min_weight_fractional_edges(self):
                    if feas[x[k].index] == infeas:
                        xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                        if xj_inf > 0.5:
                            xj_inf = 1.0 - xj_inf
                            
                        if (xj_inf <= maxinf): 
                            bestj = x[k].index
                            maxinf = xj_inf
                            maxobj = abs(obj[x[k].index])
             if bestj < 0:
                    return
             xj_lo = math.floor(x_val[bestj])
                                    
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                    
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))

class MyBranch2(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         if len(find_fractional_edges_with_max_degree_and_edge_value_1(self))>1e-7:
             for k in find_fractional_edges_with_max_degree_and_edge_value_1(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         elif len(find_fractional_edges_with_max_deg(self))>1e-7:
             for k in find_fractional_edges_with_max_deg(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=0
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution1=m.solve(log_output=True)
        

1643618.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1648183.000000
1648587.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
1648587.0
8
0.4285390377044678
2
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLD
Lazy constraint(s) or lazy constrai

KeyboardInterrupt: 

In [19]:
from cplex.callbacks import UserCutCallback, LazyConstraintCallback, BranchCallback
from docplex.mp.callbacks.cb_mixin import *
from collections import defaultdict, namedtuple
import cplex.callbacks as cpx_cb
import cplex._internal._constants as cpxcst
import math
import time
import random
def match_sep(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
def removed_graph(G,edge_set):
    for e in edge_set:
        G.remove_edge(e[0],e[1])
    return G
        
def construct_hamiltonian_tour(cplex_solution, nodes):
    adjacency = {node: [] for node in nodes}
    for i, j in cplex_solution:
        adjacency[i].append(j)
        adjacency[j].append(i)
    
    # Start with an arbitrary node
    start_node = nodes[0]
    current_node = start_node
    tour = [current_node]
    visited = set([current_node])
    
    while len(visited) < len(nodes):
        # Visit the next unvisited node
        for neighbor in adjacency[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                tour.append(neighbor)
                current_node = neighbor
                break
    
    # Close the tour
    tour.append(start_node)
    return tour
def k_factor(solution):
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    #k_factor(sep_graph, 2, matching_weight='weight')
    print(k_factor(sep_graph, 2, matching_weight='weight'))
    
def match_sep_1(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                matching_1=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                for e in nx.max_weight_matching(removed_graph(G_comp,matching),weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching_1.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True or nx.is_perfect_matching(G_comp, matching_1)==True  and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
print(solution.get_objective_value())
constr_count=0
sep_time=0
iteration=0
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    #chosen_edges = [ e for e in graph_new.edges if solution.get_value(x[e]) > 0.5+1e-7 ]
    for component in match_sep(solution):#nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
        if len(component) < n and len(component)%2==0:
           boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
           if sum(solution.get_value(x[e]) for e in boundary_edges)>len(component)-1+1e-7:
              m.add_constraint(m.sum(x[e] for e in boundary_edges)<= len(component)-1 )
              constr_count+=1
    end=time.time()
    sep_time+=end-start
    iteration+=1
    solution=m.solve(log_output=True)
    print(solution.get_objective_value())
print(constr_count) 
print(sep_time)
print(iteration)
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree == 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph
def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_fractional_edges_with_max_degree_and_edge_value_1(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    max_degree = max(dict(graph.degree).values(), default=0)
    max_degree_vertices = [node for node, degree in graph.degree if degree == max_degree]

    if not max_degree_vertices:
        return []

    fractional_edges_max = []

    for vertex in max_degree_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

def find_fractional_edges_with_max_deg(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    vertice_edge_1=[]
    fractional_edges_max = []

    for vertex in graph.nodes():
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition
        vertice_edge_1.append(vertex)
        # Process fractional edges
    max_deg_vertice=[vertex for vertex in vertice_edge_1 if graph.degree(vertex)==max([graph.degree(vertex) for vertex in vertice_edge_1])]
    for vertex in max_deg_vertice:
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class CustomHeuristicCallback(ModelCallbackMixin, cpx_cb.HeuristicCallback):
    def __init__(self, env):
        # Initialize the parent classes
        cpx_cb.HeuristicCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_heuristics = 0

    def __call__(self):
        x_val= self.make_solution_from_vars(x.values())
        self.nb_heuristics += 1
        tour_edges=[]
        for e in graph_new.edges:
            if x_val.get_value(x[e])>1-1e-7:
                tour_edges.append(e)
        
        # Example heuristic to construct a feasible solution
        gap=self.get_MIP_relative_gap()
        nodes = graph_new.nodes()
        if len(tour_edges)==len(nodes):
            initial_tour = construct_hamiltonian_tour(tour_edges,list(nodes))
            obj_value = total_distance(initial_tour, mat)
    
            # Check feasibility of the solution (optional, depends on your logic)
            tour1=two_opt(initial_tour,mat)
            if total_distance(tour1,mat)<total_distance(initial_tour,mat):
                var_edges=edge_from_tour(tour1)
                
                var_index = [x[e] for e in var_edges]
                self.set_solution(var_index, [1] * len(var_index))
        
            else:
                var_edges=edge_from_tour(initial_tour)
                var_index = [x[e] for e in var_edges]
                self.set_solution(var_index, [1] * len(var_index))
            
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         #if len(set(find_min_weight_fractional_edges(self))&set(min_matching(self)))!=0:
             
         #    for k in set(find_max_weight_fractional_edges(self))&set(max_matching(self)):
         #               if feas[x[k].index] == infeas:
         #                   xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
         #                   if xj_inf > 0.5:
         #                       xj_inf = 1.0 - xj_inf
                                
         #                   if (xj_inf <= maxinf): 
         #                       bestj = x[k].index
         #                       maxinf = xj_inf
         #                       maxobj = abs(obj[x[k].index])
         #    if bestj < 0: 
         #             return 
         #    xj_lo = math.floor(x_val[bestj])
                                        
         #    dv = self.index_to_var(bestj)
         #    self.stats[dv] += 1
                                        
         #    self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         #    self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         #elif(gap>0.03+1e-7 and gap<0.10-1e-7):
         #else:
         for k in find_min_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0:
                return
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))

class MyBranch2(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         if len(find_fractional_edges_with_max_degree_and_edge_value_1(self))>1e-7:
             for k in find_fractional_edges_with_max_degree_and_edge_value_1(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         elif len(find_fractional_edges_with_max_deg(self))>1e-7:
             for k in find_fractional_edges_with_max_deg(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=0
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
#values = {}
#values.update({x[e]: 1 for e in edge_from_tour(init_tour)})
#warmstart=m.new_solution()
#warmstart.update( values )
#m.add_mip_start(warmstart)
solution1=m.solve(log_output=True)
        

1164241.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1166996.000000
1185916.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1185926.500000
1185926.5
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
1185926.5
12
0.12427258491516113
3
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                      

In [672]:
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
def k_factors(solution):
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    #k_factor(sep_graph, 2, matching_weight='weight')
    G=nx.k_factor(sep_graph, 2,matching_weight='weight')
    subtour=[]
    
    for component in nx.connected_components( sep_graph.edge_subgraph([e for e in G.edges() if G[e[0]][e[1]]['weight']>1/2+1e-7]) ):
        subtour.append(component)
    return subtour
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    #chosen_edges = [ e for e in graph_new.edges if solution.get_value(x[e]) > 0.5+1e-7 ]
    for component in k_factors(solution):
        if len(component) < n and len(component)%2==0:
           boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
           if sum(solution.get_value(x[e]) for e in boundary_edges)>len(component)-1+1e-7:
              m.add_constraint(m.sum(x[e] for e in boundary_edges)<= len(component)-1 )
              constr_count+=1
    end=time.time()
    sep_time+=end-start
    iteration+=1
    solution=m.solve(log_output=True)
    print(solution.get_objective_value())

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1268853.000000
1274875.5
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1274876.500000
1274876.5
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
1274876.5


In [1499]:
from cplex.callbacks import UserCutCallback, LazyConstraintCallback, BranchCallback
from docplex.mp.callbacks.cb_mixin import *
import cplex
from collections import defaultdict, namedtuple
import cplex.callbacks as cpx_cb
from docplex.mp.model import Model
import cplex._internal._constants as cpxcst
import math
import time
import random
from networkx.algorithms import approximation as approx
def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons

def match_sep(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
def removed_graph(G,edge_set):
    for e in edge_set:
        G.remove_edge(e[0],e[1])
    return G
def edge_from_tour(tour):
    edge_set=[]
    for i in range(len(tour)):
        if i>=1:
            edge_set.append((min(tour[i-1],tour[i]),max(tour[i-1],tour[i])))
    return edge_set        
def construct_hamiltonian_tour(cplex_solution, nodes):
    adjacency = {node: [] for node in nodes}
    for i, j in cplex_solution:
        adjacency[i].append(j)
        adjacency[j].append(i)
    
    # Start with an arbitrary node
    start_node = nodes[0]
    current_node = start_node
    tour = [current_node]
    visited = set([current_node])
    
    while len(visited) < len(nodes):
        # Visit the next unvisited node
        for neighbor in adjacency[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                tour.append(neighbor)
                current_node = neighbor
                break
    
    # Close the tour
    tour.append(start_node)
    return tour
    
def match_sep_1(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    match=[]
    match_1=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                matching_1=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                
                for e in G_comp.edges():
                    if (min(e),max(e)) in internal_edge and (min(e),max(e)) not in matching:
                        matching_1.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True:
                      match.append(matching)
                      match_1.append(matching_1)
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets,match,match_1
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
print(solution.get_objective_value())
constr_count=0
sep_time=0
iteration=0
iter_G_H=0
G_H_cons=0
#print(solution.get_objective_value())
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    for component in match_sep(solution):
        print(component)
        boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
        m.add_constraint(m.sum(x[e] for e in boundary_edges)<= len(component)-1 )
        constr_count+=1
    end=time.time()
    sep_time+=end-start
    iteration+=1
    solution=m.solve(log_output=True)
    print(solution.get_objective_value())
subtourCons=getFracSubtourCons(solution)
for subtour in subtourCons:
    m.add_constraint(m.sum(x[e] for e in subtour)>=2)
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    subtourCons=getFracSubtourCons(solution)
    for subtour in subtourCons:
        m.add_constraint(m.sum(x[e] for e in subtour)>=2)
        constr_count+=1
    end=time.time()
    sep_time+=end-start
    solution=m.solve()
    iteration+=1
    G_H_cons+=1
    iter_G_H+=1
    solution=m.solve()

print(iter_G_H)
print(constr_count) 
print(sep_time)
print(iteration)
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph
def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class CustomHeuristicCallback(ModelCallbackMixin, cpx_cb.HeuristicCallback):
    def __init__(self, env):
        # Initialize the parent classes
        cpx_cb.HeuristicCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_heuristics = 0

    def __call__(self):
        x_val= self.make_solution_from_vars(x.values())
        self.nb_heuristics += 1
        tour_edges=[]
        for e in graph_new.edges:
            if x_val.get_value(x[e])>1-1e-7:
                tour_edges.append(e)

        # Example heuristic to construct a feasible solution
        gap=self.get_MIP_relative_gap()
        nodes = graph_new.nodes()
        if len(tour_edges)==len(nodes) and gap>0.05:
            initial_tour = construct_hamiltonian_tour(tour_edges,list(nodes))
            obj_value = total_distance(initial_tour, mat)
            tour1=two_opt(initial_tour,mat)
            obj_new=total_distance(tour1,mat)
            if obj_new<obj_value:
                print("2-opt algorithm")
                var_edges=edge_from_tour(tour1)
                
                var_index = [x[e].index for e in var_edges]
                self.set_solution([var_index, [1] * len(var_index)])
        
            else:
                var_edges=edge_from_tour(initial_tour)
                var_index = [x[e].index for e in var_edges]
                self.set_solution([var_index, [1] * len(var_index)])

class MetaHeuristicCallback(ModelCallbackMixin, cpx_cb.HeuristicCallback):
    def __init__(self, env):
        # Initialize the parent classes
        cpx_cb.HeuristicCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_heuristics = 0

    def __call__(self):
        x_val= self.make_solution_from_vars(x.values())
        self.nb_heuristics += 1
        tour_edges=[]
        for e in graph_new.edges:
            if x_val.get_value(x[e])>1-1e-7:
                tour_edges.append(e)

        # Example heuristic to construct a feasible solution
        gap=self.get_MIP_relative_gap()
        nodes = graph_new.nodes()
        if len(tour_edges)==len(nodes):
            initial_tour = construct_hamiltonian_tour(tour_edges,list(nodes))
            obj_value = total_distance(initial_tour, mat)
            tour1=approx.simulated_annealing_tsp(G, initial_tour)
            #tour1=two_opt(initial_tour,mat)
            obj_new=total_distance(tour1,mat)
            if obj_new<obj_value:
                print("simulated annealing algorithm")
                var_edges=edge_from_tour(tour1)
                
                var_index = [x[e].index for e in var_edges]
                self.set_solution([var_index, [1] * len(var_index)])
        
            else:
                var_edges=edge_from_tour(initial_tour)
                var_index = [x[e].index for e in var_edges]
                self.set_solution([var_index, [1] * len(var_index)])

def find_fractional_edges_with_max_degree_and_edge_value_1(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    max_degree = max(dict(graph.degree).values(), default=0)
    max_degree_vertices = [node for node, degree in graph.degree if degree == max_degree]

    if not max_degree_vertices:
        return []

    fractional_edges_max = []

    for vertex in max_degree_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

def find_fractional_edges_with_max_deg(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    vertice_edge_1=[]
    fractional_edges_max = []

    for vertex in graph.nodes():
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition
        vertice_edge_1.append(vertex)
        # Process fractional edges
    max_deg_vertice=[vertex for vertex in vertice_edge_1 if graph.degree(vertex)==max([graph.degree(vertex) for vertex in vertice_edge_1])]
    for vertex in max_deg_vertice:
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         if len(find_max_min_weight_fractional_edges(self))>1e-7:
             for k in find_max_min_weight_fractional_edges(self):
                    if feas[x[k].index] == infeas:
                        xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                        if xj_inf > 0.5:
                            xj_inf = 1.0 - xj_inf
                            
                        if (xj_inf <= maxinf): 
                            bestj = x[k].index
                            maxinf = xj_inf
                            maxobj = abs(obj[x[k].index])
             if bestj < 0:
                    return
             xj_lo = math.floor(x_val[bestj])
                                    
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                    
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class MyBranch2(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         if len(find_fractional_edges_with_max_degree_and_edge_value_1(self))>1e-7:
             for k in find_fractional_edges_with_max_degree_and_edge_value_1(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         elif len(find_fractional_edges_with_max_deg(self))>1e-7:
             for k in find_fractional_edges_with_max_deg(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
             
             
class CustomIncumbentCallback(ModelCallbackMixin, cpx_cb.IncumbentCallback):

    incumbent_sources = {
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_NODESOLN: 'node',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_HEURSOLN: 'heuristic',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_USERSOLN: 'user',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_MIPSTART: 'mipstart'
    }

    def __init__(self, env):
        # non public...
        cpx_cb.IncumbentCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_incumbents = 0

    def __call__(self):
        self.nb_incumbents += 1
        obj = self.get_objective_value()
        src = self.get_solution_source()
        src_name = self.incumbent_sources.get(src, ' unknown???')
        print('{0}> found incumbent with objective value {1}, coming from: {2}'.format(self.nb_incumbents, obj, src_name))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=3
#m.parameters.mip.strategy.search = 1
#m.parameters.mip.strategy.heuristicfreq = -1
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch2)
m.branch_callback=branch_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
values = {}
init_tour=[13, 10, 7, 4, 71, 60, 63, 1, 68, 57, 54, 51, 48, 45, 42, 39, 36, 33, 30, 27, 24, 21, 18, 15, 12, 9, 6, 3, 0, 65, 62, 67, 59, 56, 70, 73, 53, 50, 47, 44, 41, 38, 35, 32, 29, 26, 23, 20, 17, 14, 11, 8, 5, 2, 66, 64, 61, 58, 52, 55, 69, 72, 75, 49, 46, 43, 40, 37, 34, 31, 28, 78, 81, 84, 76, 79, 82, 85, 87, 86, 83, 80, 77, 74, 25, 22, 19, 16, 13]
values.update({x[e]: 1 for e in edge_from_tour(init_tour)})
warmstart=m.new_solution()
warmstart.update( values )
m.add_mip_start(warmstart)

solution1=m.solve(log_output=True)
        

1133387.0
{34, 28, 37, 31}
{1, 68, 60, 63}
{16, 25, 19, 22}
{40, 49, 43, 46}
{82, 76, 85, 79}
{0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 20, 21, 23, 24, 26, 27, 29, 30, 32, 33, 35, 36, 38, 39, 41, 42, 44, 45, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 77, 78, 80, 81, 83, 84, 86, 87}
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1135110.500000
1145184.5
{25, 28, 22, 31}
{40, 34, 43, 37}
{16, 10, 19, 13}
{49, 52, 46, 55}
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1

In [547]:
from cplex.callbacks import UserCutCallback, LazyConstraintCallback, BranchCallback
from docplex.mp.callbacks.cb_mixin import *
import cplex
from collections import defaultdict, namedtuple
import cplex.callbacks as cpx_cb
from docplex.mp.model import Model
import cplex._internal._constants as cpxcst
import math
import time
import random
from networkx.algorithms import approximation as approx
def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons

def match_sep(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
def removed_graph(G,edge_set):
    for e in edge_set:
        G.remove_edge(e[0],e[1])
    return G
def edge_from_tour(tour):
    edge_set=[]
    for i in range(len(tour)):
        if i>=1:
            edge_set.append((min(tour[i-1],tour[i]),max(tour[i-1],tour[i])))
    return edge_set        
def construct_hamiltonian_tour(cplex_solution, nodes):
    adjacency = {node: [] for node in nodes}
    for i, j in cplex_solution:
        adjacency[i].append(j)
        adjacency[j].append(i)
    
    # Start with an arbitrary node
    start_node = nodes[0]
    current_node = start_node
    tour = [current_node]
    visited = set([current_node])
    
    while len(visited) < len(nodes):
        # Visit the next unvisited node
        for neighbor in adjacency[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                tour.append(neighbor)
                current_node = neighbor
                break
    
    # Close the tour
    tour.append(start_node)
    return tour
    
def match_sep_1(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                matching_1=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                for e in nx.max_weight_matching(removed_graph(G_comp,matching),weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching_1.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True or nx.is_perfect_matching(G_comp, matching_1)==True  and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
print(solution.get_objective_value())
constr_count=0
sep_time=0
iteration=0
iter_G_H=0
G_H_cons=0
#print(solution.get_objective_value())
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    for component in match_sep(solution):
        print(component)
        boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
        m.add_constraint(m.sum(x[e] for e in boundary_edges)<= len(component)-1 )
        constr_count+=1
    end=time.time()
    sep_time+=end-start
    iteration+=1
    solution=m.solve(log_output=True)
    print(solution.get_objective_value())
subtourCons=getFracSubtourCons(solution)
for subtour in subtourCons:
    m.add_constraint(m.sum(x[e] for e in subtour)>=2)
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    subtourCons=getFracSubtourCons(solution)
    for subtour in subtourCons:
        m.add_constraint(m.sum(x[e] for e in subtour)>=2)
        constr_count+=1
    end=time.time()
    sep_time+=end-start
    solution=m.solve()
    iteration+=1
    G_H_cons+=1
    iter_G_H+=1
    solution=m.solve()

print(iter_G_H)
print(constr_count) 
print(sep_time)
print(iteration)
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph
def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class CustomHeuristicCallback(ModelCallbackMixin, cpx_cb.HeuristicCallback):
    def __init__(self, env):
        # Initialize the parent classes
        cpx_cb.HeuristicCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_heuristics = 0

    def __call__(self):
        x_val= self.make_solution_from_vars(x.values())
        self.nb_heuristics += 1
        tour_edges=[]
        for e in graph_new.edges:
            if x_val.get_value(x[e])>1-1e-7:
                tour_edges.append(e)

        # Example heuristic to construct a feasible solution
        gap=self.get_MIP_relative_gap()
        nodes = graph_new.nodes()
        if len(tour_edges)==len(nodes) and gap>0.05:
            initial_tour = construct_hamiltonian_tour(tour_edges,list(nodes))
            obj_value = total_distance(initial_tour, mat)
            tour1=two_opt(initial_tour,mat)
            obj_new=total_distance(tour1,mat)
            if obj_new<obj_value:
                print("2-opt algorithm")
                var_edges=edge_from_tour(tour1)
                
                var_index = [x[e].index for e in var_edges]
                self.set_solution([var_index, [1] * len(var_index)])
        
            else:
                var_edges=edge_from_tour(initial_tour)
                var_index = [x[e].index for e in var_edges]
                self.set_solution([var_index, [1] * len(var_index)])

class MetaHeuristicCallback(ModelCallbackMixin, cpx_cb.HeuristicCallback):
    def __init__(self, env):
        # Initialize the parent classes
        cpx_cb.HeuristicCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_heuristics = 0

    def __call__(self):
        x_val= self.make_solution_from_vars(x.values())
        self.nb_heuristics += 1
        tour_edges=[]
        for e in graph_new.edges:
            if x_val.get_value(x[e])>1-1e-7:
                tour_edges.append(e)

        # Example heuristic to construct a feasible solution
        gap=self.get_MIP_relative_gap()
        nodes = graph_new.nodes()
        if len(tour_edges)==len(nodes):
            initial_tour = construct_hamiltonian_tour(tour_edges,list(nodes))
            obj_value = total_distance(initial_tour, mat)
            tour1=approx.simulated_annealing_tsp(G, initial_tour)
            #tour1=two_opt(initial_tour,mat)
            obj_new=total_distance(tour1,mat)
            if obj_new<obj_value:
                print("simulated annealing algorithm")
                var_edges=edge_from_tour(tour1)
                
                var_index = [x[e].index for e in var_edges]
                self.set_solution([var_index, [1] * len(var_index)])
        
            else:
                var_edges=edge_from_tour(initial_tour)
                var_index = [x[e].index for e in var_edges]
                self.set_solution([var_index, [1] * len(var_index)])

def find_fractional_edges_with_max_degree_and_edge_value_1(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    max_degree = max(dict(graph.degree).values(), default=0)
    max_degree_vertices = [node for node, degree in graph.degree if degree == max_degree]

    if not max_degree_vertices:
        return []

    fractional_edges_max = []

    for vertex in max_degree_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

def find_fractional_edges_with_max_deg(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    vertice_edge_1=[]
    fractional_edges_max = []

    for vertex in graph.nodes():
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition
        vertice_edge_1.append(vertex)
        # Process fractional edges
    max_deg_vertice=[vertex for vertex in vertice_edge_1 if graph.degree(vertex)==max([graph.degree(vertex) for vertex in vertice_edge_1])]
    for vertex in max_deg_vertice:
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         if len(find_max_min_weight_fractional_edges(self))>1e-7:
             for k in find_max_min_weight_fractional_edges(self):
                    if feas[x[k].index] == infeas:
                        xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                        if xj_inf > 0.5:
                            xj_inf = 1.0 - xj_inf
                            
                        if (xj_inf <= maxinf): 
                            bestj = x[k].index
                            maxinf = xj_inf
                            maxobj = abs(obj[x[k].index])
             if bestj < 0:
                    return
             xj_lo = math.floor(x_val[bestj])
                                    
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                    
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class MyBranch2(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         if len(find_fractional_edges_with_max_degree_and_edge_value_1(self))>1e-7:
             for k in find_fractional_edges_with_max_degree_and_edge_value_1(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         elif len(find_fractional_edges_with_max_deg(self))>1e-7:
             for k in find_fractional_edges_with_max_deg(self):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0:
                        return
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
             
             
class CustomIncumbentCallback(ModelCallbackMixin, cpx_cb.IncumbentCallback):

    incumbent_sources = {
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_NODESOLN: 'node',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_HEURSOLN: 'heuristic',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_USERSOLN: 'user',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_MIPSTART: 'mipstart'
    }

    def __init__(self, env):
        # non public...
        cpx_cb.IncumbentCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_incumbents = 0

    def __call__(self):
        self.nb_incumbents += 1
        obj = self.get_objective_value()
        src = self.get_solution_source()
        src_name = self.incumbent_sources.get(src, ' unknown???')
        print('{0}> found incumbent with objective value {1}, coming from: {2}'.format(self.nb_incumbents, obj, src_name))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=0
#m.parameters.mip.strategy.search = 1
#m.parameters.mip.strategy.heuristicfreq = -1
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb


solution1=m.solve(log_output=True)
        

1797134.0
{41, 44, 38, 47}
{65, 68, 62, 71}
{32, 26, 35, 29}
{17, 20, 14, 23}
{56, 50, 59, 53}
{113, 116, 110, 119}
{80, 74, 83, 77}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18, 19, 21, 22, 24, 25, 27, 28, 30, 31, 33, 34, 36, 37, 39, 40, 42, 43, 45, 46, 48, 49, 51, 52, 54, 55, 57, 58, 60, 61, 63, 64, 66, 67, 69, 70, 72, 73, 75, 76, 78, 79, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 114, 115, 117, 118, 120, 121, 122, 123}
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1797406.000000
1798809.0
{64, 58, 67, 61}
{40, 34, 43, 37}
{73, 76, 70, 79}
{49, 52, 46, 55}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 26, 27, 29, 30, 32

In [211]:
from cplex.callbacks import UserCutCallback, LazyConstraintCallback, BranchCallback
from docplex.mp.callbacks.cb_mixin import *
from collections import defaultdict, namedtuple
import cplex.callbacks as cpx_cb
import cplex._internal._constants as cpxcst
import math
import random
def match_sep(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
def removed_graph(G,edge_set):
    for e in edge_set:
        G.remove_edge(e[0],e[1])
    return G
def edge_from_tour(tour):
    edge_set=[]
    for i in range(len(tour)):
        if i>=1:
            edge_set.append((min(tour[i-1],tour[i]),max(tour[i-1],tour[i])))
    return edge_set        
def construct_hamiltonian_tour(cplex_solution, nodes):
    adjacency = {node: [] for node in nodes}
    for i, j in cplex_solution:
        adjacency[i].append(j)
        adjacency[j].append(i)
    
    # Start with an arbitrary node
    start_node = nodes[0]
    current_node = start_node
    tour = [current_node]
    visited = set([current_node])
    
    while len(visited) < len(nodes):
        # Visit the next unvisited node
        for neighbor in adjacency[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                tour.append(neighbor)
                current_node = neighbor
                break
    
    # Close the tour
    tour.append(start_node)
    return tour
def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub 
def match_sep_1(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                matching_1=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                for e in nx.max_weight_matching(removed_graph(G_comp,matching),weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching_1.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True or nx.is_perfect_matching(G_comp, matching_1)==True  and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
print(solution.get_objective_value())
constr_count=0
sep_time=0
iteration=0
#print(solution.get_objective_value())
while m.solve_details.nb_iterations>1e-7:
    edges_sub,component_sub=separate_heuristic(solution)
    for i in range (len(component_sub)):
        if len(component) < n and len(component)%2==0:
           #boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
           m.add_constraint(m.sum(x[e] for e in edges_sub[i])<= len(component_sub[i])-1 )
           constr_count+=1
        #end=time.time()
    #sep_time+=end-start
    iteration+=1
    solution=m.solve(log_output=True)
    print(solution.get_objective_value())
print(constr_count) 
print(sep_time)
print(iteration)
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree == 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph
def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class CustomHeuristicCallback(ModelCallbackMixin, cpx_cb.HeuristicCallback):
    def __init__(self, env):
        # Initialize the parent classes
        cpx_cb.HeuristicCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_heuristics = 0

    def __call__(self):
        x_val= self.make_solution_from_vars(x.values())
        self.nb_heuristics += 1
        tour_edges=[]
        for e in graph_new.edges:
            if x_val.get_value(x[e])>1-1e-7:
                tour_edges.append(e)

        # Example heuristic to construct a feasible solution
        gap=self.get_MIP_relative_gap()
        nodes = graph_new.nodes()
        if len(tour_edges)==len(nodes) and gap>0.05:
            initial_tour = construct_hamiltonian_tour(tour_edges,list(nodes))
            obj_value = total_distance(initial_tour, mat)
            tour1=two_opt(initial_tour,mat)
            obj_new=total_distance(tour1,mat)
            if obj_new<obj_value:
                print("2-opt algorithm")
                var_edges=edge_from_tour(tour1)
                
                var_index = [x[e].index for e in var_edges]
                self.set_solution([var_index, [1] * len(var_index)])
        
            else:
                var_edges=edge_from_tour(initial_tour)
                var_index = [x[e].index for e in var_edges]
                self.set_solution([var_index, [1] * len(var_index)])

def find_fractional_edges_with_max_degree_and_edge_value_1(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    max_degree = max(dict(graph.degree).values(), default=0)
    max_degree_vertices = [node for node, degree in graph.degree if degree == max_degree]

    if not max_degree_vertices:
        return []

    fractional_edges_max = []

    for vertex in max_degree_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         if (len(find_fractional_edges_with_max_degree_and_edge_value_1(self)))>1e-7:
             for k in find_fractional_edges_with_max_degree_and_edge_value_1(self):
                    if feas[x[k].index] == infeas:
                        xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                        if xj_inf > 0.5:
                            xj_inf = 1.0 - xj_inf
                            
                        if (xj_inf <= maxinf): 
                            bestj = x[k].index
                            maxinf = xj_inf
                            maxobj = abs(obj[x[k].index])
             if bestj < 0:
                    return
             xj_lo = math.floor(x_val[bestj])
                                    
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                    
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))

class CustomIncumbentCallback(ModelCallbackMixin, cpx_cb.IncumbentCallback):

    incumbent_sources = {
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_NODESOLN: 'node',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_HEURSOLN: 'heuristic',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_USERSOLN: 'user',
        cpxcst.CPX_CALLBACK_MIP_INCUMBENT_MIPSTART: 'mipstart'
    }

    def __init__(self, env):
        # non public...
        cpx_cb.IncumbentCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_incumbents = 0

    def __call__(self):
        self.nb_incumbents += 1
        obj = self.get_objective_value()
        src = self.get_solution_source()
        src_name = self.incumbent_sources.get(src, ' unknown???')
        print('{0}> found incumbent with objective value {1}, coming from: {2}'.format(self.nb_incumbents, obj, src_name))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=3
#m.parameters.mip.strategy.search = 0
#m.parameters.mip.strategy.heuristicfreq = -1
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
cb=m.register_callback(SubtourLazyCallback1)
values = {}
values.update({x[e]: 1 for e in edge_from_tour(init_tour)})
warmstart=m.new_solution()
warmstart.update( values )
m.add_mip_start(warmstart)
solution1=m.solve(log_output=True)

1133387.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1135110.500000
1145184.5
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1146003.500000
1146003.5
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
1146003.5
26
0
3
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Thr

In [336]:
solution1.solve_details

docplex.mp.SolveDetails(time=44.594,status='integer optimal solution')

In [179]:
def nodes2edge(u, v):
    return (min(u, v), max(u, v))

def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons

def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub

def removed_graph(G,edge_set):
    for e in edge_set:
        G.remove_edge(e[0],e[1])
    return G
        
        
def match_sep_1(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                matching_1=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                for e in nx.max_weight_matching(removed_graph(G_comp,matching),weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching_1.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True or nx.is_perfect_matching(G_comp, matching_1)==True  and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
class SubtourUserCallback_heu(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           edges_sub,component_sub= separate_heuristic(sol)
           for i in range(len(component_sub)):
               tour=0
               for e in edges_sub[i]:
                   tour+=x[e]
               ct=tour<=len(component_sub[i])-1
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
from docplex.mp.callbacks.cb_mixin import *
import numpy as np
def add_violated_cuts(callback, solution):
    fracSubtours, distances = getFracSubtourCons(solution, callback.x, callback.nodes)
    for subtour in fracSubtours:
        expr = []
        for edge in subtour:
            expr.append("x.{}.{}".format(*edge))
        thecoefs = [1] * len(subtour)
        callback.add(cplex.SparsePair(expr, thecoefs), sense="G", rhs=2)
    return fracSubtours, len((fracSubtours)), max(distances) if len(distances) > 0 else 0
class SubtourUserCallback(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           subtourCons= getFracSubtourCons(sol)
           for subtour in subtourCons:
               tour=0
               for e in subtour:
                   tour+=x[e]
               ct=tour>=2
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
                   
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v>1e-7): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)



from collections import defaultdict, namedtuple
import math
import random

def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph
        
def find_max_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = max(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree == 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

def find_fractional_edges_with_max_degree_and_edge_value_1(self):
    graph = branching_graph(self)

    # Find vertices with maximum degree
    max_degree = max(dict(graph.degree).values(), default=0)
    max_degree_vertices = [node for node, degree in graph.degree if degree == max_degree]

    if not max_degree_vertices:
        return []

    fractional_edges_max = []

    for vertex in max_degree_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))

def max_matching(self):
    branching_graph=nx.Graph()
    branching_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
        if 1-1e-7>self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
    matching=
    return branching_graph
    
def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_branch = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = max(edge[2] for edge in fractional_edges)
            #min_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            #min_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == min_weight]
            for e in max_weight_edges:
                fractional_edges_branch.append(e)
            #for e in min_weight_edges:
            #    fractional_edges_branch.append(e)

    return fractional_edges_branch
    
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         for k in find_fractional_edges_with_max_degree_and_edge_value_1(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0: 
              return 
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         #elif(gap>0.03+1e-7 and gap<0.10-1e-7):
         #   for k in find_min_weight_fractional_edges(self):
         #       if feas[x[k].index] == infeas:
         #           xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
         #           if xj_inf > 0.5:
         #               xj_inf = 1.0 - xj_inf
                        
         #           if (xj_inf <= maxinf): 
         #               bestj = x[k].index
         #               maxinf = xj_inf
         #               maxobj = abs(obj[x[k].index])
         #   if bestj < 0:return
         #   xj_lo = math.floor(x_val[bestj])
                                
         #   dv = self.index_to_var(bestj)
         #   self.stats[dv] += 1
                                
         #   self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         #   self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         
             
        
  
         
        

from docplex.mp.model import Model
#from docplex.mp.model import Priority
m=Model()
x = {(i,j):m.binary_var(name="x_{0}_{1}".format(i,j)) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
#m.parameters.mip.strategy.search.set(1)
#m.parameters.mip.interval.set(1)
m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
#m.parameters.mip.tolerances.mipgap(0.3)
cut_cb = m.register_callback(SubtourUserCallback_heu)
m.cut_callback = cut_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution=m.solve(log_output=True) 

SyntaxError: invalid syntax (2960230620.py, line 281)

In [120]:
def match_sep(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
def removed_graph(G,edge_set):
    for e in edge_set:
        G.remove_edge(e[0],e[1])
    return G
        
        
def vertex_cover_sep(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
            
                matching=nx.bipartite.maximum_matching(G_comp)
                vertex_cover = nx.bipartite.to_vertex_cover(G_comp, matching)
                if len(component)/2-1e-7<len(vertex_cover)<len(component)/2+1e-7 and len(component)>2+1e-7:
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
#from networkx.algorithms import bipartite
#even,odd=bipartite.sets(graph_new)
#print(even)
#print(odd)
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
constr_count=0
print(solution.get_objective_value())
while m.solve_details.nb_iterations>1e-7:
    for component in vertex_cover_sep(solution):
        #print(component)
        boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
        m.add_constraint(m.sum(x[e] for e in boundary_edges)<= len(component)-1 )
        constr_count+=1
    #end=time.time()
    #separation_time+=end-start
    solution=m.solve(log_output=True)
    print(solution.get_objective_value())
print(constr_count)    #iteration+=1
def max_matching(self):
    branching_graph=nx.Graph()
    branching_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
        if 1-1e-7>self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
    matching=sorted(nx.max_weight_matching(branching_graph))
    branching_edges=[]
    for e in matching:
        branching_edges.append((min(e),max(e)))
        
    return branching_edges
def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         
         for k in find_min_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0:
                return
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=0
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
values = {}
values.update({x[e]: 1 for e in edge_from_tour(init_tour)})
warmstart=m.new_solution()
warmstart.update( values )
m.add_mip_start(warmstart)
solution1=m.solve(log_output=True)

1572396.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1572882.000000
1575720.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1575720.500000
1578273.75
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
1578273.75
15
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threa

In [41]:
def match_sep(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
def removed_graph(G,edge_set):
    for e in edge_set:
        G.remove_edge(e[0],e[1])
    return G
        
        
def match_sep_1(solution):
    def unique_sets_in_list(list_of_sets):
            return list({frozenset(s) for s in list_of_sets})
    sep_graph=nx.Graph()
    sep_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
            if solution.get_value(x[e])>1e-7:
                sep_graph.add_edge(e[0],e[1],weight=solution.get_value(x[e]))
    subgraph=[e for e in sep_graph.edges if solution.get_value(x[e])>1/2+1e-7]
    connected_comp=[]
    for component in nx.connected_components( sep_graph.edge_subgraph(subgraph) ):
                G_comp=nx.Graph()
                G_comp.add_nodes_from(component)
                internal_edge=[e for e in sep_graph.edges if e[0] in component and e[1] in component]
                G_comp.add_edges_from(internal_edge)
                matching=[]
                matching_1=[]
                for e in nx.max_weight_matching(sep_graph,weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching.append((min(e),max(e)))
                for e in nx.max_weight_matching(removed_graph(G_comp,matching),weight='weight'):
                    if (min(e),max(e)) in internal_edge:
                        matching_1.append((min(e),max(e)))
                if nx.is_perfect_matching(G_comp, matching)==True or nx.is_perfect_matching(G_comp, matching_1)==True  and len(component)>2:
                    
                      connected_comp.append(component)
        #print(unique_sets_in_list(connected_comp))
    unique_sets = [set(s) for s in unique_sets_in_list(connected_comp)]
    return unique_sets
#from networkx.algorithms import bipartite
#even,odd=bipartite.sets(graph_new)
#print(even)
#print(odd)
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
constr_count=0
print(solution.get_objective_value())
while m.solve_details.nb_iterations>1e-7:
    for component in match_sep_1(solution):
        #print(component)
        boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
        m.add_constraint(m.sum(x[e] for e in boundary_edges)<= len(component)-1 )
        constr_count+=1
    #end=time.time()
    #separation_time+=end-start
    solution=m.solve(log_output=True)
    print(solution.get_objective_value())
print(constr_count)    #iteration+=1
def max_matching(self):
    branching_graph=nx.Graph()
    branching_graph.add_nodes_from(graph_new.nodes)
    for e in graph_new.edges:
        if 1-1e-7>self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
    matching=sorted(nx.max_weight_matching(branching_graph))
    branching_edges=[]
    for e in matching:
        branching_edges.append((min(e),max(e)))
        
    return branching_edges
def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         if len(set(find_min_weight_fractional_edges(self))&set(min_matching(self)))!=0:
             
             for k in set(find_max_weight_fractional_edges(self))&set(max_matching(self)):
                        if feas[x[k].index] == infeas:
                            xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                            if xj_inf > 0.5:
                                xj_inf = 1.0 - xj_inf
                                
                            if (xj_inf <= maxinf): 
                                bestj = x[k].index
                                maxinf = xj_inf
                                maxobj = abs(obj[x[k].index])
             if bestj < 0: 
                      return 
             xj_lo = math.floor(x_val[bestj])
                                        
             dv = self.index_to_var(bestj)
             self.stats[dv] += 1
                                        
             self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
             self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         #elif(gap>0.03+1e-7 and gap<0.10-1e-7):
         else:
            for k in find_min_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
            if bestj < 0:
                return
            xj_lo = math.floor(x_val[bestj])
                                
            dv = self.index_to_var(bestj)
            self.stats[dv] += 1
                                
            self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
            self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=0
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution1=m.solve(log_output=True)
        

1439993.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1442680.000000
1446039.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1446516.000000
1451987.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1451987.000000
1451987.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve        

In [519]:
from math import sqrt
from cplex.callbacks import UserCutCallback, LazyConstraintCallback, BranchCallback
from docplex.mp.callbacks.cb_mixin import *
import time
epsilon=1e-7
def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons

m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
iteration=0
constr_count=0
separation_time=0

while m.solve_details.nb_iterations>1e-7:
#    start=time.time()
    subtourCons=getFracSubtourCons(solution)
    for subtour in subtourCons:
        m.add_constraint(m.sum(x[e] for e in subtour)>=2)
        constr_count+=1
#    end=time.time()
#    separation_time+=end-start
    solution=m.solve(log_output=True)
    iteration+=1
    #for cons in m.iter_constraints():
    #    if cons.slack_value>1e-7:
    #        m.remove_constraint(cons)
        
def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         for k in find_max_min_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0: 
              return 
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution1=m.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1690646.000000
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1700928.500000
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM

In [53]:
def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub

m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
#m.add_constraints(x[e]<=y[e[0]]+y[e[1]] for e in G.edges)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
iteration=0
constr_count=0
separation_time=0

while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    chosen_edges = [ e for e in graph_new.edges if solution.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                m.add_constraint(m.sum(x[e] for e in boundary_edges)<= len(component)-1 )
                constr_count+=1
    end=time.time()
    separation_time+=end-start
    solution=m.solve()
    iteration+=1
        
def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         for k in find_max_min_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0: 
              return 
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=0
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution1=m.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Dual steepest-edge pricing selected.
Clique table members: 14.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Initializing primal norms rowwise . . .
Initializing primal norms rowwise . . .
Root relaxation solution time = 0.02 sec. (1.58 ticks)

        Nodes                         

In [50]:
import tsplib95
import networkx as nx
import numpy as np
# load the tsplib problem

problem = tsplib95.load('pcb442.tsp')

# convert into a networkx.Graph
graph = problem.get_graph()

# convert into a numpy distance matrix
distance_matrix = nx.to_numpy_array(graph)
c = distance_matrix
n = len(c)
B=nx.complete_graph(n)
for (i,j) in B.edges():
      B.add_edge(i,j,weight=c[i][j])

opt = tsplib95.load('pcb442.opt.tour')

opt_sol=opt.tours[0]
graph_new=nx.Graph()
graph_new.add_nodes_from(B.nodes())
odd=[]
even=[]

for i in range(len(opt_sol)):
  if i%2==0:
    even.append(opt_sol[i]-1)
  else:
    odd.append(opt_sol[i]-1)

for u in odd:
    for v in B.neighbors(u):
        if v in even:
            # Ensure (i, j) with i < j
            i, j = sorted((u, v))
            graph_new.add_edge(i, j, weight=B.edges()[i,j]['weight'])

In [31]:
problem = tsplib95.load('pcb442.tsp')

# convert into a networkx.Graph
graph = problem.get_graph()

# convert into a numpy distance matrix
distance_matrix = nx.to_numpy_array(graph)
c = distance_matrix
n = len(c)
B=nx.complete_graph(n)
for (i,j) in B.edges():
      B.add_edge(i,j,weight=c[i][j])
graph_new=nx.Graph()
odd,even=nx.community.kernighan_lin_bisection(B, weight='weight')
#print(len(odd))
graph_new.add_nodes_from(B.nodes)


for u in odd:
    for v in B.neighbors(u):
       if v in even:
            # Ensure (i, j) with i < j
            i, j = sorted((u, v))
            graph_new.add_edge(i, j, weight=B.edges()[i,j]['weight'])
graph_new.edges()

EdgeView([(0, 1), (0, 3), (0, 4), (0, 7), (0, 9), (0, 11), (0, 14), (0, 15), (0, 18), (0, 21), (0, 23), (0, 26), (0, 27), (0, 29), (0, 30), (0, 32), (0, 34), (0, 38), (0, 40), (0, 42), (0, 44), (0, 45), (0, 47), (0, 49), (0, 50), (0, 52), (0, 54), (0, 55), (0, 57), (0, 58), (0, 61), (0, 64), (0, 65), (0, 67), (0, 68), (0, 69), (0, 71), (0, 72), (0, 74), (0, 76), (0, 78), (0, 80), (0, 82), (0, 84), (0, 85), (0, 87), (0, 88), (0, 91), (0, 93), (0, 96), (0, 100), (0, 102), (0, 105), (0, 106), (0, 107), (0, 108), (0, 110), (0, 111), (0, 112), (0, 114), (0, 116), (0, 117), (0, 120), (0, 121), (0, 125), (0, 126), (0, 129), (0, 130), (0, 132), (0, 133), (0, 134), (0, 137), (0, 138), (0, 141), (0, 142), (0, 143), (0, 144), (0, 148), (0, 149), (0, 151), (0, 153), (0, 157), (0, 158), (0, 159), (0, 161), (0, 162), (0, 163), (0, 165), (0, 166), (0, 167), (0, 171), (0, 173), (0, 176), (0, 181), (0, 183), (0, 187), (0, 189), (0, 191), (0, 192), (0, 194), (0, 197), (0, 199), (0, 200), (0, 201), (0, 2

In [9]:
graph_new = nx.complete_bipartite_graph(int(len(B.nodes)/2), int(len(B.nodes)/2))
#print(B.nodes)
for e in graph_new.edges:
   if e in B.edges:
      graph_new.edges[e]['weight']=B.edges[e]['weight']



In [262]:
nx.write_edgelist(graph_new, "Tnm118_2.txt", data=True)

In [124]:
import tsplib95
import networkx as nx
import numpy as np
# load the tsplib problem

problem = tsplib95.load('Tnm100.tsp')

# convert into a networkx.Graph
graph = problem.get_graph()

# convert into a numpy distance matrix
distance_matrix = nx.to_numpy_array(graph)


c = distance_matrix
n = len(c)
B=nx.Graph()
for i in range(len(c)):
  for j in range(len(c)):
    if i<j:
      B.add_edge(i,j,weight=c[i][j])


def nodes2edge(u, v):
    return (min(u, v), max(u, v))

#import networkx as nx

# Define the file path
file_path = 'Tnm100_1.txt'  # Replace with your file's actual path if needed

# Create an empty graph
graph_new = nx.Graph()
graph_new.add_nodes_from(B.nodes())
# Read the file and add edges to the graph
with open(file_path, 'r') as file:
    for line in file:
        source, target, attributes = line.strip().split(maxsplit=2)
        source, target = int(source), int(target)
        attributes = eval(attributes)  # Convert string representation of dictionary to a Python dictionary
        graph_new.add_edge(source, target, **attributes)

In [667]:
def merge_hamiltonian_paths_fix_largest(paths, U, V):
    # Sort paths by length and fix the largest path
    paths.sort(key=len, reverse=True)
    largest_path = paths[0]
    other_paths = paths[1:]

    # Initialize the cycle with the largest path
    cycle = largest_path[:]
    current_end = largest_path[-1]

    # Connect other paths to the cycle
    for path in other_paths:
        start, end = path[0], path[-1]

        # Determine if reversal is needed
        if current_end in U and start in U or current_end in V and start in V:
            path = path[::-1]  # Reverse the path
            start, end = end, start

        # Add the reversed or original path to the cycle
        cycle.extend(path)
        current_end = end  # Update the current endpoint

    # Close the cycle
    first_start = largest_path[0]
    if current_end in U and first_start in V or current_end in V and first_start in U:
        cycle.append(first_start)

    return cycle
bottom_nodes, top_nodes = bipartite.sets(graph_new)
# Merge paths into a single Hamiltonian cycle
merged_cycle = merge_hamiltonian_paths_fix_largest(ham_path, bottom_nodes, top_nodes)
print(total_distance(two_opt(merged_cycle, mat),mat))

1542693.0


In [669]:
print(two_opt(merged_cycle, mat))

[23, 20, 35, 62, 68, 71, 74, 77, 0, 79, 6, 3, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 48, 45, 51, 54, 57, 60, 63, 83, 66, 69, 72, 75, 1, 80, 4, 7, 10, 13, 16, 19, 22, 25, 31, 28, 34, 37, 40, 43, 49, 46, 52, 55, 58, 61, 64, 67, 70, 73, 76, 2, 5, 78, 81, 8, 11, 14, 17, 84, 87, 90, 93, 96, 86, 89, 92, 95, 98, 99, 97, 94, 91, 88, 85, 82, 65, 59, 56, 53, 50, 47, 44, 41, 38, 29, 32, 26, 23]


In [76]:
import time
def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub

m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
iteration=0
constr_count=0
separation_time=0
start=time.time()
edges_sub,component_sub=separate_heuristic(solution)
for i in range(len(component_sub)):
    m.add_constraint(m.sum(x[e] for e in edges_sub[i])<=len(component_sub[i])-1)
    constr_count+=1
end=time.time()
separation_time+=end-start
solution=m.solve()
iteration=1
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    #subtourCons=getFracSubtourCons(solution)
    edges_sub,component_sub=separate_heuristic(solution)
    for i in range(len(component_sub)):
        m.add_constraint(m.sum(x[e] for e in edges_sub[i])<=len(component_sub[i])-1)
        constr_count+=1
    end=time.time()
    separation_time+=end-start
    solution=m.solve(log_output=True)
    iteration+=1
Gomory_Hu_constr=0
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    subtourCons=getFracSubtourCons(solution)
    for subtour in subtourCons:
        m.add_constraint(m.sum(x[e] for e in subtour)>=2)
        Gomory_Hu_constr+=1
    
    solution=m.solve()
    iteration+=1

def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         for k in find_max_min_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0: 
              return 
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch2)
m.branch_callback=branch_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution1=m.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Dual steepest-edge pricing selected.
Clique table members: 15.
MIP emphasis: balance optimality and feasibility.


KeyboardInterrupt: None

In [251]:
print(solution)

solution for: docplex_model147
objective: 1.52876e+06
status: OPTIMAL_SOLUTION(2)
x_0_3=1
x_0_80=1
x_1_78=1
x_1_83=1
x_2_79=1
x_2_81=1
x_3_82=1
x_4_7=1
x_4_83=1
x_5_8=1
x_5_81=1
x_6_9=1
x_6_85=1
x_7_86=1
x_8_11=1
x_9_12=1
x_10_13=1
x_10_86=1
x_11_14=1
x_12_15=1
x_13_16=1
x_14_17=1
x_15_18=1
x_16_19=1
x_17_20=1
x_18_21=1
x_19_22=1
x_20_23=1
x_21_24=1
x_22_25=1
x_23_26=1
x_24_27=1
x_25_28=1
x_26_32=1
x_27_30=1
x_28_31=1
x_29_32=1
x_29_35=1
x_30_33=1
x_31_34=1
x_33_36=1
x_34_37=1
x_35_38=1
x_36_39=1
x_37_40=1
x_38_41=1
x_39_42=1
x_40_43=1
x_41_44=1
x_42_45=1
x_43_92=1
x_44_47=1
x_45_48=1
x_46_49=1
x_46_90=1
x_47_53=1
x_48_51=1
x_49_52=1
x_50_53=1
x_50_103=1
x_51_54=1
x_52_55=1
x_54_57=1
x_55_58=1
x_56_59=1
x_56_88=1
x_57_60=1
x_58_61=1
x_59_62=1
x_60_63=1
x_61_64=1
x_62_65=1
x_63_89=1
x_64_67=1
x_65_68=1
x_66_69=1
x_66_89=1
x_67_87=1
x_68_71=1
x_69_72=1
x_70_73=1
x_70_84=1
x_71_74=1
x_72_75=1
x_73_76=1
x_74_77=1
x_75_78=1
x_76_79=1
x_77_80=1
x_82_85=1
x_84_87=1
x_88_91=1
x_90_93=1
x_91_94

In [81]:
def nodes2edge(u, v):
    return (min(u, v), max(u, v))

def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons
def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in m._G.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
    
from docplex.mp.callbacks.cb_mixin import *
import numpy as np

class SubtourUserCallback_heu(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           edges_sub,component_sub= separate_heuristic(sol)
           for i in range(len(component_sub)):
               tour=0
               for e in edges_sub[i]:
                   tour+=x[e]
               ct=tour<=len(component_sub[i])-1
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
                   
class SubtourUserCallback(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           subtourCons= getFracSubtourCons(sol)
           for subtour in subtourCons:
               tour=0
               for e in subtour:
                   tour+=x[e]
               ct=tour>=2
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v>1e-7): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)



from collections import defaultdict, namedtuple
import math
import random

def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph

def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub

class SubtourUserCallback_heu(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           edges_sub,component_sub= separate_heuristic(sol)
           for i in range(len(component_sub)):
               tour=0
               for e in edges_sub[i]:
                   tour+=x[e]
               ct=tour<=len(component_sub[i])-1
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
def find_max_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = max(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree == 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))


def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_branch = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = max(edge[2] for edge in fractional_edges)
            #min_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            #min_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == min_weight]
            for e in max_weight_edges:
                fractional_edges_branch.append(e)
            #for e in min_weight_edges:
            #    fractional_edges_branch.append(e)

    return fractional_edges_branch
    
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         for k in find_max_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0: 
              return 
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         #elif(gap>0.03+1e-7 and gap<0.10-1e-7):
         #   for k in find_min_weight_fractional_edges(self):
         #       if feas[x[k].index] == infeas:
         #           xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
         #           if xj_inf > 0.5:
         #               xj_inf = 1.0 - xj_inf
                        
         #           if (xj_inf <= maxinf): 
         #               bestj = x[k].index
         #               maxinf = xj_inf
         #               maxobj = abs(obj[x[k].index])
         #   if bestj < 0:return
         #   xj_lo = math.floor(x_val[bestj])
                                
         #   dv = self.index_to_var(bestj)
         #   self.stats[dv] += 1
                                
         #   self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         #   self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         
             
        
  
         
        

from docplex.mp.model import Model
#from docplex.mp.model import Priority
m=Model()
x = {(i,j):m.binary_var(name="x_{0}_{1}".format(i,j)) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
#m.parameters.mip.strategy.search.set(1)
#m.parameters.mip.interval.set(1)
m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
#m.parameters.mip.tolerances.mipgap(0.3)
cut_cb = m.register_callback(SubtourUserCallback_heu)
m.cut_callback = cut_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution=m.solve(log_output=True) 

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLUD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Dual steepest-edge pricing selected.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =       1468377.000000
Iteration:    62   Dual object

In [82]:
def nodes2edge(u, v):
    return (min(u, v), max(u, v))

def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons
def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in m._G.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
    
from docplex.mp.callbacks.cb_mixin import *
import numpy as np

class SubtourUserCallback_heu(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           edges_sub,component_sub= separate_heuristic(sol)
           for i in range(len(component_sub)):
               tour=0
               for e in edges_sub[i]:
                   tour+=x[e]
               ct=tour<=len(component_sub[i])-1
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
                   
class SubtourUserCallback(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           subtourCons= getFracSubtourCons(sol)
           for subtour in subtourCons:
               tour=0
               for e in subtour:
                   tour+=x[e]
               ct=tour>=2
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v>1e-7): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)



from collections import defaultdict, namedtuple
import math
import random

def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph

def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub

class SubtourUserCallback_heu(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           edges_sub,component_sub= separate_heuristic(sol)
           for i in range(len(component_sub)):
               tour=0
               for e in edges_sub[i]:
                   tour+=x[e]
               ct=tour<=len(component_sub[i])-1
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
def find_max_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = max(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree == 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))


def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_branch = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = max(edge[2] for edge in fractional_edges)
            #min_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            #min_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == min_weight]
            for e in max_weight_edges:
                fractional_edges_branch.append(e)
            #for e in min_weight_edges:
            #    fractional_edges_branch.append(e)

    return fractional_edges_branch
    
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         for k in find_max_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0: 
              return 
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         #elif(gap>0.03+1e-7 and gap<0.10-1e-7):
         #   for k in find_min_weight_fractional_edges(self):
         #       if feas[x[k].index] == infeas:
         #           xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
         #           if xj_inf > 0.5:
         #               xj_inf = 1.0 - xj_inf
                        
         #           if (xj_inf <= maxinf): 
         #               bestj = x[k].index
         #               maxinf = xj_inf
         #               maxobj = abs(obj[x[k].index])
         #   if bestj < 0:return
         #   xj_lo = math.floor(x_val[bestj])
                                
         #   dv = self.index_to_var(bestj)
         #   self.stats[dv] += 1
                                
         #   self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         #   self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         
             
        
  
         
        

from docplex.mp.model import Model
#from docplex.mp.model import Priority
m=Model()
x = {(i,j):m.binary_var(name="x_{0}_{1}".format(i,j)) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
#m.parameters.mip.strategy.search.set(1)
#m.parameters.mip.interval.set(1)
m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch2)
m.branch_callback=branch_cb
#m.parameters.mip.tolerances.mipgap(0.3)
cut_cb = m.register_callback(SubtourUserCallback)
m.cut_callback = cut_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution=m.solve(log_output=True) 

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLUD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Dual steepest-edge pricing selected.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =       1468377.000000
Iteration:    62   Dual object

In [339]:
def nodes2edge(u, v):
    return (min(u, v), max(u, v))

def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons
def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
    
from docplex.mp.callbacks.cb_mixin import *
import numpy as np

class SubtourUserCallback_heu(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           edges_sub,component_sub= separate_heuristic(sol)
           for i in range(len(component_sub)):
               tour=0
               for e in edges_sub[i]:
                   tour+=x[e]
               ct=tour<=len(component_sub[i])-1
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
                   
class SubtourUserCallback(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           subtourCons= getFracSubtourCons(sol)
           for subtour in subtourCons:
               tour=0
               for e in subtour:
                   tour+=x[e]
               ct=tour>=2
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v>1e-7): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)



from collections import defaultdict, namedtuple
import math
import random

def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph

def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub

class SubtourUserCallback_heu(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           edges_sub,component_sub= separate_heuristic(sol)
           for i in range(len(component_sub)):
               tour=0
               for e in edges_sub[i]:
                   tour+=x[e]
               ct=tour<=len(component_sub[i])-1
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
def find_max_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = max(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree == 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))


def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_branch = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = max(edge[2] for edge in fractional_edges)
            #min_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            #min_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == min_weight]
            for e in max_weight_edges:
                fractional_edges_branch.append(e)
            #for e in min_weight_edges:
            #    fractional_edges_branch.append(e)

    return fractional_edges_branch
    
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         for k in find_max_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0: 
              return 
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         #elif(gap>0.03+1e-7 and gap<0.10-1e-7):
         #   for k in find_min_weight_fractional_edges(self):
         #       if feas[x[k].index] == infeas:
         #           xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
         #           if xj_inf > 0.5:
         #               xj_inf = 1.0 - xj_inf
                        
         #           if (xj_inf <= maxinf): 
         #               bestj = x[k].index
         #               maxinf = xj_inf
         #               maxobj = abs(obj[x[k].index])
         #   if bestj < 0:return
         #   xj_lo = math.floor(x_val[bestj])
                                
         #   dv = self.index_to_var(bestj)
         #   self.stats[dv] += 1
                                
         #   self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         #   self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         
             
        
  
         
        

from docplex.mp.model import Model
#from docplex.mp.model import Priority
m=Model()
x = {(i,j):m.binary_var(name="x_{0}_{1}".format(i,j)) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
#m.parameters.mip.strategy.search.set(1)
#m.parameters.mip.interval.set(1)
m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch2)
m.branch_callback=branch_cb
#m.parameters.mip.tolerances.mipgap(0.3)
cut_cb = m.register_callback(SubtourUserCallback_heu)
m.cut_callback = cut_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
values = {}
values.update({x[e]: 1 for e in edge_from_tour(init_tour)})
warmstart=m.new_solution()
warmstart.update( values )
m.add_mip_start(warmstart)
solution=m.solve(log_output=True) 

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLUD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Processing 1 MIP starts.
MIP start 'm1' defined solution with objective 1936032.0000.
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 1936032.0000.
Dual steepest-edge pricing selected.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-a

In [341]:
def nodes2edge(u, v):
    return (min(u, v), max(u, v))

def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons

def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
class SubtourUserCallback_heu(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           edges_sub,component_sub= separate_heuristic(sol)
           for i in range(len(component_sub)):
               tour=0
               for e in edges_sub[i]:
                   tour+=x[e]
               ct=tour<=len(component_sub[i])-1
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
from docplex.mp.callbacks.cb_mixin import *
import numpy as np
def add_violated_cuts(callback, solution):
    fracSubtours, distances = getFracSubtourCons(solution, callback.x, callback.nodes)
    for subtour in fracSubtours:
        expr = []
        for edge in subtour:
            expr.append("x.{}.{}".format(*edge))
        thecoefs = [1] * len(subtour)
        callback.add(cplex.SparsePair(expr, thecoefs), sense="G", rhs=2)
    return fracSubtours, len((fracSubtours)), max(distances) if len(distances) > 0 else 0
class SubtourUserCallback(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           subtourCons= getFracSubtourCons(sol)
           for subtour in subtourCons:
               tour=0
               for e in subtour:
                   tour+=x[e]
               ct=tour>=2
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
                   
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v>1e-7): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)



from collections import defaultdict, namedtuple
import math
import random

def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph
        
def find_max_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = max(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree == 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))


def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_branch = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = max(edge[2] for edge in fractional_edges)
            #min_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            #min_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == min_weight]
            for e in max_weight_edges:
                fractional_edges_branch.append(e)
            #for e in min_weight_edges:
            #    fractional_edges_branch.append(e)

    return fractional_edges_branch
    
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         for k in find_max_min_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0: 
              return 
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         #elif(gap>0.03+1e-7 and gap<0.10-1e-7):
         #   for k in find_min_weight_fractional_edges(self):
         #       if feas[x[k].index] == infeas:
         #           xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
         #           if xj_inf > 0.5:
         #               xj_inf = 1.0 - xj_inf
                        
         #           if (xj_inf <= maxinf): 
         #               bestj = x[k].index
         #               maxinf = xj_inf
         #               maxobj = abs(obj[x[k].index])
         #   if bestj < 0:return
         #   xj_lo = math.floor(x_val[bestj])
                                
         #   dv = self.index_to_var(bestj)
         #   self.stats[dv] += 1
                                
         #   self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         #   self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         
             
        
  
         
        

from docplex.mp.model import Model
#from docplex.mp.model import Priority
m=Model()
x = {(i,j):m.binary_var(name="x_{0}_{1}".format(i,j)) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
#m.parameters.mip.strategy.search.set(1)
#m.parameters.mip.interval.set(1)
m.parameters.mip.strategy.variableselect=0
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
#m.parameters.mip.tolerances.mipgap(0.3)
cut_cb = m.register_callback(SubtourUserCallback)
m.cut_callback = cut_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution=m.solve(log_output=True) 

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLUD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Dual steepest-edge pricing selected.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =       1589716.000000
Iteration:    62   Dual objective     =       1771884.000000
Root relaxation solu

In [110]:
def nodes2edge(u, v):
    return (min(u, v), max(u, v))

def getFracSubtourCons(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons

def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
class SubtourUserCallback_heu(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           edges_sub,component_sub= separate_heuristic(sol)
           for i in range(len(component_sub)):
               tour=0
               for e in edges_sub[i]:
                   tour+=x[e]
               ct=tour<=len(component_sub[i])-1
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
from docplex.mp.callbacks.cb_mixin import *
import numpy as np
def add_violated_cuts(callback, solution):
    fracSubtours, distances = getFracSubtourCons(solution, callback.x, callback.nodes)
    for subtour in fracSubtours:
        expr = []
        for edge in subtour:
            expr.append("x.{}.{}".format(*edge))
        thecoefs = [1] * len(subtour)
        callback.add(cplex.SparsePair(expr, thecoefs), sense="G", rhs=2)
    return fracSubtours, len((fracSubtours)), max(distances) if len(distances) > 0 else 0
class SubtourUserCallback(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
           subtourCons= getFracSubtourCons(sol)
           for subtour in subtourCons:
               tour=0
               for e in subtour:
                   tour+=x[e]
               ct=tour>=2
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
                   
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v>1e-7): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)



from collections import defaultdict, namedtuple
import math
import random

def branching_graph(self):
      branching_graph=nx.Graph()
      branching_graph.add_nodes_from(graph_new.nodes)
      for e in graph_new.edges:
        if self.get_values(x[e].name)>1e-7:
          branching_graph.add_edge(e[0],e[1],weight=c[e[0]][e[1]])
      return branching_graph
        
def find_max_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = max(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree == 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))


def find_max_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_branch = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            #max_weight = max(edge[2] for edge in fractional_edges)
            #min_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges]
            #min_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2] == min_weight]
            for e in max_weight_edges:
                fractional_edges_branch.append(e)
            #for e in min_weight_edges:
            #    fractional_edges_branch.append(e)

    return fractional_edges_branch
    
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         for k in find_max_min_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0: 
              return 
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         #elif(gap>0.03+1e-7 and gap<0.10-1e-7):
         #   for k in find_min_weight_fractional_edges(self):
         #       if feas[x[k].index] == infeas:
         #           xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
         #           if xj_inf > 0.5:
         #               xj_inf = 1.0 - xj_inf
                        
         #           if (xj_inf <= maxinf): 
         #               bestj = x[k].index
         #               maxinf = xj_inf
         #               maxobj = abs(obj[x[k].index])
         #   if bestj < 0:return
         #   xj_lo = math.floor(x_val[bestj])
                                
         #   dv = self.index_to_var(bestj)
         #   self.stats[dv] += 1
                                
         #   self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         #   self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
         
             
        
  
         
        

from docplex.mp.model import Model
#from docplex.mp.model import Priority
m=Model()
x = {(i,j):m.binary_var(name="x_{0}_{1}".format(i,j)) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
#m.parameters.mip.strategy.search.set(1)
#m.parameters.mip.interval.set(1)
m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
#m.parameters.mip.tolerances.mipgap(0.3)
cut_cb = m.register_callback(SubtourUserCallback_heu)
m.cut_callback = cut_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution=m.solve(log_output=True) 

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLUD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Dual steepest-edge pricing selected.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.

Iteration log . . .
Iteration:     1   Dual objective     =         20000.000000
Iteration:    77   Dual objective     =       1165943.000000
Root 

In [281]:
import time
def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub

m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
iteration=0
constr_count=0
separation_time=0
start=time.time()
edges_sub,component_sub=separate_heuristic(solution)
for i in range(len(component_sub)):
    m.add_constraint(m.sum(x[e] for e in edges_sub[i])<=len(component_sub[i])-1)
    constr_count+=1
end=time.time()
separation_time+=end-start
solution=m.solve()
iteration=1
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    #subtourCons=getFracSubtourCons(solution)
    edges_sub,component_sub=separate_heuristic(solution)
    for i in range(len(component_sub)):
        m.add_constraint(m.sum(x[e] for e in edges_sub[i])<=len(component_sub[i])-1)
        constr_count+=1
    end=time.time()
    separation_time+=end-start
    solution=m.solve(log_output=True)
    iteration+=1
Gomory_Hu_constr=0
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    subtourCons=getFracSubtourCons(solution)
    for subtour in subtourCons:
        m.add_constraint(m.sum(x[e] for e in subtour)>=2)
        Gomory_Hu_constr+=1
    
    solution=m.solve()
    iteration+=1

def find_min_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = min(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         for k in find_min_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0: 
              return 
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution1=m.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1690968.000000
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLD
Lazy constraint(s) or lazy constraint

In [523]:
def separate_heuristic_1(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) >1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
iteration=0
constr_count=0
separation_time=0
start=time.time()
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    #subtourCons=getFracSubtourCons(solution)
    edges_sub,component_sub=separate_heuristic_1(solution)
    for i in range(len(component_sub)):
        m.add_constraint(m.sum(x[e] for e in edges_sub[i])<=len(component_sub[i])-1)
        constr_count+=1
    end=time.time()
    separation_time+=end-start
    solution=m.solve(log_output=True)
    print

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1797135.000000
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1798815.000000
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1


In [521]:
import time
def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
def separate_heuristic_1(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) >1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
iteration=0
constr_count=0
separation_time=0
start=time.time()
edges_sub,component_sub=separate_heuristic_1(solution)
for i in range(len(component_sub)):
    m.add_constraint(m.sum(x[e] for e in edges_sub[i])<=len(component_sub[i])-1)
    constr_count+=1
end=time.time()
separation_time+=end-start
solution=m.solve()
iteration=1
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    #subtourCons=getFracSubtourCons(solution)
    edges_sub,component_sub=separate_heuristic_1(solution)
    for i in range(len(component_sub)):
        m.add_constraint(m.sum(x[e] for e in edges_sub[i])<=len(component_sub[i])-1)
        constr_count+=1
    end=time.time()
    separation_time+=end-start
    solution=m.solve(log_output=True)
    iteration+=1
Gomory_Hu_constr=0
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    subtourCons=getFracSubtourCons(solution)
    for subtour in subtourCons:
        m.add_constraint(m.sum(x[e] for e in subtour)>=2)
        Gomory_Hu_constr+=1
    
    solution=m.solve()
    iteration+=1

def find_max_weight_fractional_edges(self):
    graph = branching_graph(self)
    degree_3_vertices = [node for node, degree in graph.degree if degree >= 3]
    if not degree_3_vertices:
        return []

    fractional_edges_max = []

    for vertex in degree_3_vertices:
        # Check if the vertex has at least one edge with self.get_values(x[(u, v)].name) == 1
        has_edge_with_value_1 = any(
            self.get_values(x[(min(edge[0], edge[1]), max(edge[0], edge[1]))].name) == 1
            for edge in graph.edges(vertex)
        )
        if not has_edge_with_value_1:
            continue  # Skip this vertex if it doesn't meet the condition

        # Process fractional edges
        fractional_edges = []
        for edge in graph.edges(vertex, data=True):
            u, v, data = edge
            u, v = sorted((u, v))
            weight = data.get('weight', 0)  # Default weight to 0 if not present
            edge_value = self.get_values(x[(u, v)].name)
            if 1e-7 < edge_value < 1 - 1e-7:
                fractional_edges.append((u, v, weight))  # Use tuples

        if fractional_edges:  # Ensure we have edges to process
            max_weight = max(edge[2] for edge in fractional_edges)
            max_weight_edges = [(edge[0], edge[1]) for edge in fractional_edges if edge[2]==max_weight]
            for e in max_weight_edges:
                fractional_edges_max.append(e)  # Convert to tuples

    return list(set(fractional_edges_max))
class MyBranch1(ModelCallbackMixin, BranchCallback):

    brtype_map = {'0': 'var', '1': 'sos1', '2': 'sos2', 'X': 'user'}
    def __init__(self, env):
        # non public...
        BranchCallback.__init__(self, env)
        ModelCallbackMixin.__init__(self)
        self.nb_called = 0
        self.stats = defaultdict(int)

    def __call__(self):
         x_val = self.get_values()
         #sol=self.make_solution_from_vars(x.values())
         bipartite_sum=0
         obj_TSP=0
         for e in graph_new.edges():
             obj_TSP+=c[e[0]][e[1]]*self.get_values(x[e].name)
         #objval = self.get_objective_value()
         obj = self.get_objective_coefficients()
         feas = self.get_feasibilities()
         maxobj = cplex.infinity
         maxinf = cplex.infinity
         bestj = -1
         infeas = self.feasibility_status.infeasible
         fractional_node=[]
         number_of_int=[]
         gap=self.get_MIP_relative_gap()
         max_node=[]
         int_node=[]
         weight_node=[]
         int_edge=[]
         
         #print(x_val) 
         #if gap>0.10+1e-7:
         for k in find_max_weight_fractional_edges(self):
                if feas[x[k].index] == infeas:
                    xj_inf = x_val[x[k].index] - math.floor(x_val[x[k].index])
                    if xj_inf > 0.5:
                        xj_inf = 1.0 - xj_inf
                        
                    if (xj_inf <= maxinf): 
                        bestj = x[k].index
                        maxinf = xj_inf
                        maxobj = abs(obj[x[k].index])
         if bestj < 0: 
              return 
         xj_lo = math.floor(x_val[bestj])
                                
         dv = self.index_to_var(bestj)
         self.stats[dv] += 1
                                
         self.make_branch(obj_TSP, variables=[(bestj, "L", xj_lo + 1)],node_data=(bestj, xj_lo, "UP"))
         self.make_branch(obj_TSP, variables=[(bestj, "U", xj_lo)],node_data=(bestj, xj_lo, "DOWN"))
class SubtourLazyCallback1(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v==1): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
for e in graph_new.edges:
    x[e].set_vartype('B')

m.parameters.mip.strategy.variableselect=3
m.parameters.mip.limits.cutsfactor.set(0) 
m.parameters.mip.display.set(4)
m.set_time_limit(600)
branch_cb=m.register_callback(MyBranch1)
m.branch_callback=branch_cb
cb=m.register_callback(SubtourLazyCallback1)
m.lazy_callback=cb
solution1=m.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1798815.000000
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Strategy_VariableSelect             3
CPXPARAM_TimeLimit                               600
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  BbLD
Lazy constraint(s) or lazy constraint

KeyboardInterrupt: None

In [533]:
import time
def separate_heuristic_1(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) >1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub

m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve(log_output=True)
print(solution.get_objective_value())
iteration=0
constr_count=0
while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    #subtourCons=getFracSubtourCons(solution)
    edges_sub,component_sub=separate_heuristic(solution)
    for i in range(len(component_sub)):
        m.add_constraint(m.sum(x[e] for e in edges_sub[i])<=len(component_sub[i])-1)
        constr_count+=1
    end=time.time()
    separation_time+=end-start
    solution=m.solve(log_output=True)
    print(solution.get_objective_value())
    iteration+=1

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =       1589716.000000
Iteration:    62   Dual objective     =       1771884.000000
1797134.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1797135.000000
1798809.0
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1

Iteration log . . .
Iteration:     1   Dual objective     =       1798815.000000
1798815.0
Versio

In [535]:
def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub

m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j),lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
iteration=0
constr_count=0
separation_time=0
Gomory_Hu_constr=0
#while m.solve_details.nb_iterations>1e-7:
#    start=time.time()
    #subtourCons=getFracSubtourCons(solution)
#    edges_sub,component_sub=separate_heuristic(solution)
#    for i in range(len(component_sub)):
#        m.add_constraint(m.sum(x[e] for e in edges_sub[i])<=len(component_sub[i])-1)
#        constr_count+=1
#    end=time.time()
#    separation_time+=end-start
#    solution=m.solve()
#    iteration+=1
#Gomory_Hu_constr=0
while m.solve_details.nb_iterations>1+1e-7:
    start=time.time()
    subtourCons=getFracSubtourCons(solution)
    for subtour in subtourCons:
        m.add_constraint(m.sum(x[e] for e in subtour)>=2)
        Gomory_Hu_constr+=1
    end=time.time()
    separation_time+=end-start
    solution=m.solve()
    iteration+=1
print(constr_count)
print(Gomory_Hu_constr)
print(separation_time)
print(solution.get_objective_value())
print(iteration)
for e in graph_new.edges:
    if solution.get_value(x[e])>1e-7:
        print(e,solution.get_value(x[e]))

0
12
0.18684077262878418
1798815.0
2
(0, 3) 1.0
(0, 98) 1.0
(1, 96) 1.0
(1, 101) 1.0
(2, 5) 1.0
(2, 97) 1.0
(3, 100) 1.0
(4, 7) 1.0
(4, 101) 1.0
(5, 99) 1.0
(6, 9) 1.0
(6, 100) 1.0
(7, 10) 1.0
(8, 11) 1.0
(8, 99) 1.0
(9, 12) 1.0
(10, 13) 1.0
(11, 14) 1.0
(12, 15) 1.0
(13, 16) 0.5
(13, 104) 0.5
(14, 17) 1.0
(15, 18) 1.0
(16, 19) 1.0
(16, 25) 0.5
(17, 20) 1.0
(18, 21) 1.0
(19, 22) 0.5
(19, 104) 0.5
(20, 23) 1.0
(21, 24) 1.0
(22, 25) 1.0
(22, 31) 0.5
(23, 26) 1.0
(24, 27) 1.0
(25, 28) 0.5
(26, 29) 1.0
(27, 30) 1.0
(28, 31) 1.0
(28, 37) 0.5
(29, 32) 1.0
(30, 33) 1.0
(31, 34) 0.5
(32, 35) 1.0
(33, 39) 1.0
(34, 37) 1.0
(34, 43) 0.5
(35, 38) 1.0
(36, 39) 1.0
(36, 42) 1.0
(37, 40) 0.5
(38, 41) 1.0
(40, 43) 1.0
(40, 49) 0.5
(41, 44) 1.0
(42, 45) 1.0
(43, 46) 0.5
(44, 47) 1.0
(45, 48) 1.0
(46, 49) 1.0
(46, 55) 0.5
(47, 50) 1.0
(48, 51) 1.0
(49, 52) 0.5
(50, 53) 1.0
(51, 57) 1.0
(52, 55) 1.0
(52, 61) 0.5
(53, 56) 1.0
(54, 57) 1.0
(54, 60) 1.0
(55, 58) 0.5
(56, 59) 1.0
(58, 61) 1.0
(58, 67) 0.5
(5

In [349]:
m=Model()
x = {(i,j):m.continuous_var(name="x_{0}_{1}".format(i,j), lb=0.0,ub=1.0) for (i,j) in graph_new.edges}
m.minimize(m.sum(c[i][j]*x[i,j] for (i,j) in graph_new.edges))
m.add_constraints(m.sum(x[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m.parameters.preprocessing.presolve.set(0)
m.parameters.threads.set(1) 
solution=m.solve() 
iteration=0
constr_count=0
separation_time=0

while m.solve_details.nb_iterations>1e-7:
    start=time.time()
    subtourCons=getFracSubtourCons(solution)
    for subtour in subtourCons:
        m.add_constraint(m.sum(x[e] for e in subtour)>=2)
        constr_count+=1
    
    solution=m.solve()
    iteration+=1
    end=time.time()
    separation_time+=end-start
print(constr_count)
print(separation_time)
print(solution.get_objective_value())
print(iteration)

16
0.4640843868255615
1848843.0
5


In [1386]:
def nodes2edge(u, v):
    return (min(u, v), max(u, v))
def add_violated_cuts_BTSP(callback, solution):
    fracSubtours, distances = getFracSubtourCons(solution, callback.x, callback.nodes)
    for subtour in fracSubtours:
        expr = []
        for edge in subtour:
            expr.append("x.{}.{}".format(*edge))
        thecoefs = [1] * len(subtour)
        callback.add(cplex.SparsePair(expr, thecoefs), sense="G", rhs=2)
    return fracSubtours, len((fracSubtours)), max(distances) if len(distances) > 0 else 0
def separate_heuristic(sol):
    edges_sub=[]
    component_sub=[]
    chosen_edges = [ e for e in graph_new.edges if sol.get_value(x1[e]) > 0.5+1e-7 ]
    for component in nx.connected_components( graph_new.edge_subgraph(chosen_edges) ):
          if len(component) < n and len(component)%2==0:
                boundary_edges = [(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                edges_sub.append(boundary_edges)
                component_sub.append(component)
    return edges_sub,component_sub
def getFracSubtourCons_BTSP(sol):
    graph = nx.Graph()
    for e in graph_new.edges():
        if (sol.get_value(x1[e])>0):
            graph.add_edge(e[0], e[1], capacity=sol.get_value(x1[e]))
    
    tree = nx.gomory_hu_tree(graph)
    subtourCons = []
    #distances = []

    for edge in tree.edges:
        if (2 - tree[edge[0]][edge[1]]['weight'] > 1e-7):
            w = tree[edge[0]][edge[1]]['weight']
            tree.remove_edge(edge[0], edge[1])
            V1 = nx.node_connected_component(tree, edge[0])
            V2 = nx.node_connected_component(tree, edge[1])
            cutEdges = []
            tree.add_edge(edge[0], edge[1], weight=w)
            for u in V1:
                for v in V2:
                    if (u != v and nodes2edge(u, v) in graph_new.edges()):
                        cutEdges.append(nodes2edge(u, v))

            subtourCons.append(cutEdges)
            #distances.append((2 - w) / (sqrt(len(V1) * len(V2))))
    return subtourCons

class SubtourUserCallback_BTSP(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x1.values())
           subtourCons= getFracSubtourCons_BTSP(sol)
           for subtour in subtourCons:
               tour=0
               for e in subtour:
                   tour+=x1[e]
               ct=tour>=2
               unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
               for ct, cpx_lhs, sense, cpx_rhs in unsats:
                    self.add(cpx_lhs, sense, cpx_rhs)
           #if len(subtourCons)!=0:
           comb_graph=nx.Graph()
           comb_graph.add_nodes_from(graph_new.nodes)
           for e in graph_new.edges:
                  if sol.get_value(x1[e])>1e-7:
                      comb_graph.add_edge(e[0],e[1])
               
           fractional_edge=[e for e in comb_graph.edges() if 1-1e-7>sol.get_value(x1[e])>1e-7]
           for component in nx.connected_components(comb_graph.edge_subgraph(fractional_edge) ):
                   number_of_u=0
                   number_of_v=0
                   for i in component:
                       if i in bottom_nodes:
                          number_of_u+=1       
                       elif i in top_nodes:
                          number_of_v+=1
                   if number_of_u == number_of_v:       
                            boundary_edge=[e for e in graph_new.edges if e[0] in component and e[1] in component]
                            cut_edges=[e for e in comb_graph.edges if ((e[0] in component and e[1] not in component) or (e[0] not in component and e[1] in component))]
                            cut_edges_u=[e for e in comb_graph.edges if ((e[0] in component and e[1] not in component and e[0] in bottom_nodes) or (e[0] not in component and e[1] in component and e[1] in bottom_nodes))]
                            cut_edges_v=[e for e in comb_graph.edges if ((e[0] in component and e[1] not in component and e[0] in top_nodes) or (e[0] not in component and e[1] in component and e[1] in top_nodes))]
                            if  len(cut_edges)>=4 and len(cut_edges)%4==2 and similar_edges(cut_edges)==True and len(cut_edges_u)== len(cut_edges_v) and len(cut_edges_v)>=3:
                                    tour_1=0
                                    for e in boundary_edge:
                                        tour_1+=x1[e]
                                    for e1 in cut_edges:
                                        tour_1+=x1[e1]
                                        
                                    ct=tour_1<=len(component)+len(cut_edges_v)-1
                                    unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                                    for ct, cpx_lhs, sense, cpx_rhs in unsats:
                                            self.add(cpx_lhs, sense, cpx_rhs)
class SubtourUserCallback_heu(ConstraintCallbackMixin,UserCutCallback): 
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)
        self.subtour_cut_count = 0
    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x1.values())
           edges_sub,component_sub= separate_heuristic(sol)
           
           for i in range(len(component_sub)):
                   tour=0
                   for e in edges_sub[i]:
                       tour+=x1[e]
                   ct=tour<=len(component_sub[i])-1
                   unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                   for ct, cpx_lhs, sense, cpx_rhs in unsats:
                        self.add(cpx_lhs, sense, cpx_rhs)
           
                                
           
            
                        # Get all 4 nodes
           comb_graph = nx.Graph()
           comb_graph.add_nodes_from(graph_new.nodes)
           
           for e in graph_new.edges:
               val = sol.get_value(x1[e])
               if val > 1e-7:
                        comb_graph.add_edge(e[0], e[1], weight=val)
                
                # Step 2: Get max weight matching
           raw_matching = nx.max_weight_matching(comb_graph, weight='weight')
                
                # Normalize edge representation
           matching = []
           for e in raw_matching:
                edge = (min(e), max(e))
                if edge in graph_new.edges and sol.get_value(x1[edge])==1:
                        matching.append(edge)
                
                # Step 3: If perfect matching, print
                #if nx.is_perfect_matching(comb_graph, matching):
                
                
                    # Step 4: Build meta-graph where each node is a matching edge
           meta_graph = nx.Graph()
           meta_graph.add_nodes_from(matching)
                
                    # Step 5: Connect matching edges that share neighbors in original graph
           for i in range(len(matching)):
                for j in range(i + 1, len(matching)):
                    e1 = matching[i]
                    e2 = matching[j]
                
                            # Get all 4 nodes
                    nodes_e1 = set(e1)
                    nodes_e2 = set(e2)
                
                            # Check all connections between nodes of e1 and e2
                    connecting_weight = 0
                    for u in nodes_e1:
                        for v in nodes_e2:
                            if graph_new.has_edge(u, v):
                               connecting_weight += sol.get_value(x1[(u, v)]) if (u, v) in x else sol.get_value(x1[(v, u)])
                
                    if connecting_weight > 0:
                        meta_graph.add_edge(e1, e2, weight=connecting_weight)
                
                    # Step 6: Print meta-graph edges
           chosen_meta=[(u,v) for u, v, data in meta_graph.edges(data=True) if data['weight']<1-1e-7]
           for component in nx.connected_components(meta_graph.edge_subgraph(chosen_meta) ):
                            handle_vertices=[]
                            handle_edges = []
                            teeth_edges =[]
                            teeth_vertex=[]
                            internal_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v in component)]
                            cut_edge=[(u,v) for (u,v) in meta_graph.edges() if (u in component and v not in component) or (u not in component and v in component)]
                            for (e1,e2) in internal_edge:
                                for u in e1:
                                    for v in e2:
                                        if graph_new.has_edge(u, v):
                                            handle_edges.append((min(u, v),max(u,v)))
                                handle_edges.append(e1)
                                handle_edges.append(e2)
                            for (e1,e2) in cut_edge:
                                teeth=[]
                                for u in e1:
                                    for v in e2:
                                        if graph_new.has_edge(u, v):
                                            teeth.append((min(u, v),max(u,v)))
                                teeth.append(e1)
                                teeth.append(e2)
                                teeth_edges.append(teeth)
                                teeth_vertex.append(vertex_list(teeth))
                                
                            if len(cut_edge)%2==1 and len(cut_edge)>=3:
                                tour=0
                                teeth_sum=0
                                for teeth in teeth_vertex:
                                    for e in graph_new.edges:
                                        if e[0] in vertex_list(teeth) and e[1] in vertex_list(teeth):
                                           teeth_sum+=x1[e]
                                for e in graph_new.edges:
                                    if e[0] in vertex_list(handle_edges) and e[1] in vertex_list(handle_edges):
                                        tour+=x1[e]

                                ct=tour+teeth_sum<= len(vertex_list(handle_edges))+sum(len(teeth_vertices) for teeth_vertices in teeth_vertex )-0.5*(3*len(teeth_vertex)+1)
                                unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                                for ct, cpx_lhs, sense, cpx_rhs in unsats:
                                        self.add(cpx_lhs, sense, cpx_rhs)
           
           
class SubtourLazyCallback1_BTSP(ConstraintCallbackMixin,LazyConstraintCallback): 
    def __init__(self, env):
        LazyConstraintCallback.__init__(self, env)
        ConstraintCallbackMixin.__init__(self)

    def __call__(self):
           var_pos_name=[]
           edge_pos=[]
           sol = self.make_solution_from_vars(x1.values())
        # Fetch variable values into a solution object
           for i,v in enumerate(self.get_values()):
              if (v>1e-7): var_pos_name.append(self.index_to_var(i).name)
           for i in range(len(var_pos_name)):
               for e in graph_new.edges:
                   if (x1[e].name==var_pos_name[i]):
                       edge_pos.append(e)
           edge_for_subtour=[]
           for component in nx.connected_components( graph_new.edge_subgraph(edge_pos) ):
             if len(component) < n and len(component)>=3:
                 edge_in_component=[(i,j) for (i,j) in graph_new.edges if (i in component and j in component)]
                 tour=0
                 for e in edge_in_component:
                     tour += x1[e]
                 ct = tour <=  len(component)- 1
                 unsats = self.get_cpx_unsatisfied_cts([ct], sol, tolerance=1e-7)
                 for ct, cpx_lhs, sense, cpx_rhs in unsats:
    
                     self.add(cpx_lhs, sense, cpx_rhs)
                     
from docplex.mp.model import Model
#from docplex.mp.model import Priority
m1=Model()
x1 = {(i,j):m1.binary_var(name="x_{0}_{1}".format(i,j)) for (i,j) in graph_new.edges}
m1.minimize(m1.sum(c[i][j]*x1[i,j] for (i,j) in graph_new.edges))
m1.add_constraints(m1.sum(x1[e] for e in graph_new.edges if e in graph_new.edges(i))==2 for i in graph_new.nodes)
m1.parameters.preprocessing.presolve.set(0)
m1.parameters.threads.set(1) 
#m.parameters.mip.strategy.search.set(1)
#m.parameters.mip.interval.set(1)

m1.parameters.mip.limits.cutsfactor.set(0) 
m1.parameters.mip.display.set(4)
m.parameters.mip.tolerances.mipgap(1e-6)
cut_cb = m1.register_callback(SubtourUserCallback_heu)
m1.cut_callback = cut_cb
cb=m1.register_callback(SubtourLazyCallback1_BTSP)
m1.lazy_callback=cb
solution=m1.solve(log_output=True) 

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Limits_CutsFactor                   0
Legacy callback                                  LUD
Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
Dual steepest-edge pricing selected.
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =       1468377.000000
Iteration:    62   Dual objective     =       1606001.000000
Root relaxation solution time = 0.01 sec. (2.16 ticks)

        Nodes      

In [519]:
print(solution)

solution for: docplex_model139
objective: 1.88442e+06
status: OPTIMAL_SOLUTION(2)
x_0_3=1
x_0_98=1
x_1_96=1
x_1_101=1
x_2_5=1
x_2_97=1
x_3_100=1
x_4_7=1
x_4_101=1
x_5_99=1
x_6_9=1
x_6_100=1
x_7_10=1
x_8_11=1
x_8_99=1
x_9_12=1
x_10_13=1
x_11_102=1
x_12_15=1
x_13_16=1
x_14_17=1
x_14_105=1
x_15_18=1
x_16_19=1
x_17_20=1
x_18_21=1
x_19_22=1
x_20_23=1
x_21_24=1
x_22_25=1
x_23_26=1
x_24_27=1
x_25_28=1
x_26_29=1
x_27_30=1
x_28_31=1
x_29_32=1
x_30_33=1
x_31_34=1
x_32_35=1
x_33_39=1
x_34_37=1
x_35_38=1
x_36_39=1
x_36_42=1
x_37_40=1
x_38_41=1
x_40_43=1
x_41_44=1
x_42_45=1
x_43_110=1
x_44_47=1
x_45_48=1
x_46_49=1
x_46_108=1
x_47_50=1
x_48_51=1
x_49_52=1
x_50_53=1
x_51_57=1
x_52_55=1
x_53_56=1
x_54_57=1
x_54_60=1
x_55_58=1
x_56_59=1
x_58_61=1
x_59_62=1
x_60_63=1
x_61_64=1
x_62_65=1
x_63_66=1
x_64_67=1
x_65_71=1
x_66_69=1
x_67_70=1
x_68_71=1
x_68_115=1
x_69_72=1
x_70_73=1
x_72_75=1
x_73_76=1
x_74_77=1
x_74_83=1
x_75_78=1
x_76_79=1
x_77_80=1
x_78_81=1
x_79_82=1
x_80_86=1
x_81_107=1
x_82_85=1
x_83_103

In [220]:
import networkx
tour_graph=nx.Graph()
for e in graph_new.edges():
    if solution.get_value(x1[e])>1e-7:
        tour_graph.add_edge(e[0],e[1])
print(tour_graph.degree)
pos = nx.circular_layout(tour_graph)
for c in nx.connected_components(tour_graph):
    print(c)

[(0, 2), (3, 2), (92, 2), (1, 2), (4, 2), (87, 2), (2, 2), (5, 2), (91, 2), (94, 2), (7, 2), (93, 2), (6, 2), (9, 2), (97, 2), (10, 2), (8, 2), (11, 2), (96, 2), (12, 2), (16, 2), (14, 2), (15, 2), (13, 2), (98, 2), (17, 2), (18, 2), (23, 2), (21, 2), (19, 2), (22, 2), (113, 2), (20, 2), (99, 2), (24, 2), (25, 2), (27, 2), (28, 2), (26, 2), (29, 2), (100, 2), (30, 2), (31, 2), (32, 2), (33, 2), (34, 2), (35, 2), (36, 2), (37, 2), (38, 2), (39, 2), (40, 2), (41, 2), (42, 2), (43, 2), (44, 2), (45, 2), (46, 2), (47, 2), (48, 2), (49, 2), (50, 2), (51, 2), (52, 2), (53, 2), (54, 2), (55, 2), (56, 2), (57, 2), (58, 2), (59, 2), (60, 2), (61, 2), (62, 2), (63, 2), (64, 2), (65, 2), (66, 2), (70, 2), (68, 2), (69, 2), (67, 2), (73, 2), (71, 2), (72, 2), (74, 2), (75, 2), (76, 2), (77, 2), (78, 2), (79, 2), (80, 2), (81, 2), (82, 2), (83, 2), (84, 2), (85, 2), (86, 2), (95, 2), (88, 2), (89, 2), (90, 2), (101, 2), (102, 2), (103, 2), (104, 2), (105, 2), (106, 2), (107, 2), (108, 2), (109, 2),

In [258]:
for e in graph_new.edges():
    if solution.get_value(x1[e])>1e-7:
        print(e,solution.get_value(x1[e]))

(0, 3) 1.0
(0, 92) 1.0
(1, 4) 1.0
(1, 87) 1.0
(2, 5) 1.0
(2, 91) 1.0
(3, 94) 1.0
(4, 7) 1.0
(5, 93) 1.0
(6, 9) 1.0
(6, 97) 1.0
(7, 10) 1.0
(8, 11) 1.0
(8, 96) 1.0
(9, 12) 1.0
(10, 16) 1.0
(11, 14) 1.0
(12, 15) 1.0
(13, 16) 1.0
(13, 98) 1.0
(14, 17) 1.0
(15, 18) 1.0
(17, 23) 1.0
(18, 21) 1.0
(19, 22) 1.0
(19, 113) 1.0
(20, 23) 1.0
(20, 99) 1.0
(21, 24) 1.0
(22, 25) 1.0
(24, 27) 1.0
(25, 28) 1.0
(26, 29) 1.0
(26, 100) 1.0
(27, 30) 1.0
(28, 31) 1.0
(29, 32) 1.0
(30, 33) 1.0
(31, 34) 1.0
(32, 35) 1.0
(33, 36) 1.0
(34, 37) 1.0
(35, 38) 1.0
(36, 39) 1.0
(37, 40) 1.0
(38, 41) 1.0
(39, 42) 1.0
(40, 43) 1.0
(41, 44) 1.0
(42, 45) 1.0
(43, 46) 1.0
(44, 47) 1.0
(45, 48) 1.0
(46, 49) 1.0
(47, 50) 1.0
(48, 51) 1.0
(49, 52) 1.0
(50, 53) 1.0
(51, 54) 1.0
(52, 55) 1.0
(53, 56) 1.0
(54, 57) 1.0
(55, 58) 1.0
(56, 59) 1.0
(57, 60) 1.0
(58, 61) 1.0
(59, 62) 1.0
(60, 63) 1.0
(61, 64) 1.0
(62, 65) 1.0
(63, 66) 1.0
(64, 70) 1.0
(65, 68) 1.0
(66, 69) 1.0
(67, 70) 1.0
(67, 73) 1.0
(68, 71) 1.0
(69, 72) 1.0
(71,

In [256]:
nx.approximation.traveling_salesman_problem(tour_graph)

[0,
 92,
 89,
 86,
 83,
 80,
 77,
 74,
 71,
 68,
 65,
 62,
 59,
 56,
 53,
 50,
 47,
 44,
 41,
 38,
 35,
 32,
 29,
 26,
 100,
 103,
 106,
 109,
 112,
 115,
 116,
 117,
 114,
 111,
 108,
 105,
 102,
 99,
 20,
 23,
 17,
 14,
 11,
 8,
 96,
 93,
 5,
 2,
 91,
 88,
 85,
 82,
 79,
 76,
 73,
 67,
 70,
 64,
 61,
 58,
 55,
 52,
 49,
 46,
 43,
 40,
 37,
 34,
 31,
 28,
 25,
 22,
 19,
 113,
 110,
 107,
 104,
 101,
 98,
 13,
 16,
 10,
 7,
 4,
 1,
 87,
 90,
 95,
 84,
 81,
 78,
 75,
 72,
 69,
 66,
 63,
 60,
 57,
 54,
 51,
 48,
 45,
 42,
 39,
 36,
 33,
 30,
 27,
 24,
 21,
 18,
 15,
 12,
 9,
 6,
 97,
 94,
 3,
 0]